In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import colors
import csv
import numpy as np
import pandas as pd
import cobra

In [2]:
cobra.__version__

'0.17.1'

In [3]:
Annotation = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Annotation', index_col=0)
Annotation.index = Annotation.index.map(str)
Annotation = Annotation.fillna('')
Transcriptomics = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Transcriptomics', header=[0,1,2,3], index_col=0)
Transcriptomics.index = Transcriptomics.index.map(str)
Proteomics = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Proteomics', header=[0,1,2], index_col=0)
Proteomics.index = Proteomics.index.map(str)
Fitness = pd.read_excel('../../Data/R_toruloides_Data_for_Reconstruction.xlsx',
                          sheet_name='Fitness', index_col=0)
Fitness.index = Fitness.index.map(str)

In [4]:
def background_gradient(s, cmap='seismic', text_color_threshold=0.408):
    lim = max(abs(s.min().min()),abs(s.max().max()))
    rng = 2.0*lim
    norm = colors.Normalize(-lim - (rng * 0.2), lim + (rng * 0.2))
    rgbas = plt.cm.get_cmap(cmap)(norm(s.values))
    def relative_luminance(rgba):
        r, g, b = (x / 12.92 if x <= 0.03928 else ((x + 0.055) / 1.055 ** 2.4) for x in rgba[:3])
        return 0.2126 * r + 0.7152 * g + 0.0722 * b
    def css(rgba):
        dark = relative_luminance(rgba) < text_color_threshold
        text_color = '#f1f1f1' if dark else '#000000'
        return 'background-color: {b};color: {c};'.format(b=colors.rgb2hex(rgba), c=text_color)

    if s.ndim == 1:
        return [css(rgba) for rgba in rgbas]
    else:
        return pd.DataFrame([[css(rgba) for rgba in row] for row in rgbas], index=s.index, columns=s.columns)

def Show_Data(x):
    display(Transcriptomics.loc[x].style.background_gradient(cmap='Reds', low=0.2, high=0.2, axis=None))
    temp = [y for y in x if y in Proteomics.index]
    display(Proteomics.loc[temp].style.background_gradient(cmap='Reds', low=0.2, high=0.2, axis=None))
    temp = [y for y in x if y in Fitness.index]
    display(Fitness.loc[temp].style.apply(background_gradient, cmap='seismic', axis=None))
    return;

In [5]:
eco = cobra.io.load_json_model('../../Data/BiGG_Models/iML1515.json')
sce = cobra.io.load_json_model('../../Data/BiGG_Models/iMM904.json')
hsa = cobra.io.load_json_model('../../Data/BiGG_Models/RECON1.json')
hsa2 = cobra.io.load_json_model('../../Data/BiGG_Models/Recon3D.json')
ptri = cobra.io.load_json_model('../../Data/BiGG_Models/iLB1027_lipid.json')

In [6]:
model = cobra.io.load_json_model("IFO0880_GPR_1b.json")

In [7]:
print(len(model.genes))
print(len([x for x in model.genes if not x.id[0].isalpha()]))
model

1312
1150


Name,R. toruloides
Memory address,0x0102f395e80
Number of metabolites,3283
Number of reactions,3264
Number of groups,0
Objective expression,0
Compartments,"c, x, m, e, r, v, n, g, p, h, s, f, l"


### Fatty acid degradation

In [8]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('ACACT') and r.id.endswith('m'):
        print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('ACACT') and r.id.endswith('r'):
        print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('ACACT') and not (r.id.endswith('m') or r.id.endswith('r')):
        print(r.id, r.reaction, r.gene_reaction_rule)

ACACT10m 2maacoa_m + coa_m <=> accoa_m + ppcoa_m 13813 or 8678
ACACT1m 2.0 accoa_m --> aacoa_m + coa_m 8678 or 8885
ACACT2m accoa_m + btcoa_m --> 3ohcoa_m + coa_m 13813
ACACT3m accoa_m + hxcoa_m --> 3oocoa_m + coa_m 13813
ACACT4m accoa_m + occoa_m <-- 3odcoa_m + coa_m 13813 or 9065
ACACT5m accoa_m + dcacoa_m --> 3oddcoa_m + coa_m 13813
ACACT6m accoa_m + ddcacoa_m <-- 3otdcoa_m + coa_m 13813 or 9065
ACACT7m accoa_m + tdcoa_m <-- 3ohdcoa_m + coa_m 13813 or 9065

ACACT1r 2.0 accoa_c <=> aacoa_c + coa_c 13813 or 8678 or 8885
ACACT2r accoa_c + btcoa_c <=> 3ohcoa_c + coa_c 13813 or 8678 or 8885
ACACT3r accoa_c + hxcoa_c <=> 3oocoa_c + coa_c 13813
ACACT4r accoa_c + occoa_c <=> 3odcoa_c + coa_c 13813
ACACT5r accoa_c + dcacoa_c <=> 3oddcoa_c + coa_c 13813
ACACT6r accoa_c + ddcacoa_c <=> 3otdcoa_c + coa_c 13813
ACACT7r accoa_c + tdcoa_c <=> 3ohdcoa_c + coa_c 13813
ACACT8r 3ohodcoa_c + coa_c <=> accoa_c + pmtcoa_c 13813

ACACT1x 2.0 accoa_x <=> aacoa_x + coa_x 8678 or 8885
ACACT2 3ohcoa_x + coa_x

In [9]:
temp = ['8678','8885','13813','9065','15228','14055','11237']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
8678,"K00626: E2.3.1.9, atoB; acetyl-CoA C-acetyltra...",,ERG10,ACAT1,ERG10,ACAT1,Not Essential,"mito 23.5, cyto_mito 14",QRL*
8885,K07508: ACAA2; acetyl-CoA acyltransferase 2,,,ACAA2,ERG10,ACAA2,Not Essential,"mito 12, cyto 11, pero 3",ERV*
13813,K07513: ACAA1; acetyl-CoA acyltransferase 1,,POT1,ACAA1,POT1,ACAA1,Not Essential,"mito 18.5, cyto_mito 13, cyto 6.5",AEN*
9065,KOG1389: 3-oxoacyl CoA thiolase,,POT1,ACAA1,POT1,ACAA1,Not Essential,"mito 13, cyto 10, nucl 2",VAE*
15228,K07513: ACAA1; acetyl-CoA acyltransferase 1,,,,POT1,ACAA1,Not Essential,"cyto 14, mito 13",VRD*
14055,"K08764: SCP2, SCPX; sterol carrier protein 2",,,SCP2,,SCP2,Not Essential,"cyto_mito 13.333, mito 13, cyto 11.5, cyto_nuc...",ANL*
11237,KOG4170: 2-enoyl-CoA hydratase/3-hydroxyacyl-C...,,,,,,Not Essential,"mito_nucl 10.166, nucl 10, mito 10, cyto_nucl 8",AKL*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
8885,-0.216781,0.0868698,0.107291,-0.144912,-0.0878781,0.256151,-0.409361,-0.996266,-0.417173,-0.594114,0.200075
13813,0.12641,0.107964,0.192915,0.252853,0.192276,0.251335,-2.49127,-1.6891,-0.0652642,-0.2766,0.225342
9065,-0.350988,-0.271906,-0.134182,-0.284826,-5.6088,-3.42636,-0.290069,0.247955,0.258871,0.400966,-0.20317
15228,-0.223091,-0.11819,-0.335936,-0.277735,-4.99507,-4.67996,0.138344,0.163114,0.296363,0.54175,0.402816
14055,0.39435,0.131153,0.25378,0.0630405,0.300374,-0.158759,-0.304284,-0.624025,0.255405,-0.263458,-0.21954
11237,-0.0579288,-0.355731,0.0689474,0.0852672,0.215514,-0.159126,-0.115729,-0.0764419,0.094597,0.206729,-0.335975


8678 mito 23.5, cyto_mito 14 (no sigP) ERG10 Acetyl-CoA acetyltransferase, KEGG acetyl-CoA C-acetyltransferase 2.3.1.9 (ACAT1) QRL*  
8885 mito 12, cyto 11, pero 3 (no sigP) Acetyl-CoA acetyltransferase, KEGG acetyl-CoA acyltransferase 2 2.3.1.16 (ACAA2) -> mito fatty acid beta-oxidation ERV*  
13813 mito 18.5, cyto_mito 13, cyto 6.5 POT1 (sigP) 3-oxoacyl CoA thiolase, KEGG acetyl-CoA acyltransferase 1 2.3.1.16 (ACAA1) -> peroxisomal fatty acid beta-oxidation AEN*  

These two have significant fitness defect in coumarate or ferulate - 4HB-acetyl-CoA to 4HB-CoA / 3-oxoadipyl CoA thiolase  
9065 mito 13, cyto 10, nucl 2 POT1 (no sigP) 3-oxoacyl CoA thiolase, KEGG acetyl-CoA acyltransferase 1 2.3.1.16 (ACAA1) -> 4HB-acetyl-CoA to 4HB-CoA  
15228 cyto 14, mito 13 (no sigP) 3-oxoacyl CoA thiolase, KEGG acetyl-CoA acyltransferase 1 2.3.1.16 (ACAA1), no rxn -> blast hit to 3-oxoadipyl CoA thiolase  

14055 cyto_mito 13.333, mito 13, cyto 11.5, cyto_nucl 7.166 Peroxisomal 3-ketoacyl-CoA-thiolase P-44/SCP2, KEGG propanoyl-CoA C-acyltransferase 2.3.1.176 (SCP2) -> check reactions  
11237 mito_nucl 10.166, nucl 10, mito 10, cyto_nucl 8 sterol-binding domain and related enzymes, shorter only SCP-2 sterol transfer family domain, KEGG 2.3.1.176, no rxn

ERG10 is cyto and human ACAT1 is mito, works on ACACT1 and ACACT10  
Change ACACT1r genes to '8678'  
Remove ACACT2r-8r  
Change ACACT1m genes to '8678', make it reversible  
Change ACACT2m, ACACT3m, ACACT5m irreversible in the reverse direction  
Change ACACT2m-7m genes to '8885'  
Change ACACT10m genes to '8678'  
Reverse ACACT1x  
Change ACACT1x, ACACT2, ACACT3 to ACACT1p, ACACT2p, ACACT3p  
Change ACACTxp genes to '13813'

In [10]:
model.reactions.get_by_id('ACACT1r').gene_reaction_rule = '8678'
model.remove_reactions(['ACACT2r','ACACT3r','ACACT4r','ACACT5r','ACACT6r','ACACT7r','ACACT8r'], remove_orphans=True)
model.reactions.get_by_id('ACACT1m').gene_reaction_rule = '8678'
model.reactions.get_by_id('ACACT1m').lower_bound = -1000.0
model.reactions.get_by_id('ACACT2m').upper_bound = 0.0
model.reactions.get_by_id('ACACT2m').lower_bound = -1000.0
model.reactions.get_by_id('ACACT3m').upper_bound = 0.0
model.reactions.get_by_id('ACACT3m').lower_bound = -1000.0
model.reactions.get_by_id('ACACT5m').upper_bound = 0.0
model.reactions.get_by_id('ACACT5m').lower_bound = -1000.0
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('ACACT') and not r.id.startswith('ACACT1') and r.id.endswith('m'):
        r.gene_reaction_rule = '8885'
model.reactions.get_by_id('ACACT10m').gene_reaction_rule = '8678'
for m in model.reactions.get_by_id('ACACT1x').metabolites:
    model.reactions.get_by_id('ACACT1x').add_metabolites({m.id: -2*model.reactions.get_by_id('ACACT1x').get_coefficient(m.id)})
model.reactions.get_by_id('ACACT1x').id = 'ACACT1p'
model.reactions.get_by_id('ACACT2').id = 'ACACT2p'
model.reactions.get_by_id('ACACT3').id = 'ACACT3p'
model.reactions.get_by_id('ACACT1p').lower_bound = 0.0
model.reactions.get_by_id('ACACT2p').lower_bound = 0.0
model.reactions.get_by_id('ACACT3p').lower_bound = 0.0
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('ACACT') and r.id.endswith('p'):
        r.gene_reaction_rule = '13813'

In [11]:
for r in sorted(model.genes.get_by_id('8678').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8885').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13813').reactions, key=lambda x: x.id):
    if 'FAO' not in r.id:
        print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9065').reactions, key=lambda x: x.id):
    if 'FAO' not in r.id:
        print(r.id, r.reaction, r.gene_reaction_rule)

ACACT10m 2maacoa_m + coa_m <=> accoa_m + ppcoa_m 8678
ACACT1m 2.0 accoa_m <=> aacoa_m + coa_m 8678
ACACT1r 2.0 accoa_c <=> aacoa_c + coa_c 8678

ACACT2m accoa_m + btcoa_m <-- 3ohcoa_m + coa_m 8885
ACACT3m accoa_m + hxcoa_m <-- 3oocoa_m + coa_m 8885
ACACT4m accoa_m + occoa_m <-- 3odcoa_m + coa_m 8885
ACACT5m accoa_m + dcacoa_m <-- 3oddcoa_m + coa_m 8885
ACACT6m accoa_m + ddcacoa_m <-- 3otdcoa_m + coa_m 8885
ACACT7m accoa_m + tdcoa_m <-- 3ohdcoa_m + coa_m 8885

ACACT1p aacoa_x + coa_x --> 2.0 accoa_x 13813
ACACT2p 3ohcoa_x + coa_x --> accoa_x + btcoa_x 13813
ACACT3p 3oocoa_x + coa_x --> accoa_x + hxcoa_x 13813
ACACT4p 3odcoa_x + coa_x --> accoa_x + occoa_x 13813
ACACT5p 3oddcoa_x + coa_x --> accoa_x + dcacoa_x 13813
ACACT6p 3otdcoa_x + coa_x --> accoa_x + ddcacoa_x 13813
ACACT7p 3ohdcoa_x + coa_x --> accoa_x + tdcoa_x 13813
ACACT8p 3ohodcoa_x + coa_x --> accoa_x + pmtcoa_x 13813
ACACT9p 3ohxccoa_x + coa_x --> accoa_x + ttccoa_x 13813
KAT180_m 3ohodcoa_m + coa_m --> accoa_m + pmtcoa_m 138

In [12]:
# MACCOAT is cyto version of ACACT10m, mitochondrial ppcoa from beta oxidation or branched aa degradation
# KAT180_m long chain beta oxidation should be in peroxisome
model.remove_reactions(['KAT180_m','MACCOAT'], remove_orphans=True)
# Other yli mito reactions redundant or use o2 like pero -> remove
# genes are '(12742 or 12752 or 9700)/POX1 and 14805/ECHS1 and (13813 or 9065)/ACAA1', they are all covered below
model.remove_reactions(['yli_R0194','yli_R0195','yli_R0211','yli_R0216','yli_R0217','yli_R0218','yli_R0219','yli_R0220','yli_R0221','yli_R0222','yli_R0223','yli_R0224','yli_R0225'], remove_orphans=True)

In [13]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('ACOAO') and r.id.endswith('p'):
        print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('ACOA') and r.id.endswith('OR'):
        print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('ACOAD') and not r.id.startswith('ACOADAGAT'):
        print(r.id, r.reaction, r.gene_reaction_rule)

ACOAO4p dcacoa_x + o2_x --> dc2coa_x + h2o2_x 12742 or 12752 or 9700
ACOAO5p ddcacoa_x + o2_x --> dd2coa_x + h2o2_x 12742 or 12752 or 9700
ACOAO6p o2_x + tdcoa_x --> h2o2_x + td2coa_x 12742 or 12752 or 9700
ACOAO7p o2_x + pmtcoa_x --> h2o2_x + hdd2coa_x 12742 or 12752 or 9700
ACOAO8p o2_x + stcoa_x --> h2o2_x + od2coa_x 12742 or 12752 or 9700
ACOAO9p hexccoa_x + o2_x --> h2o2_x + hxc2coa_x 12742 or 12752 or 9700

ACOA100OR dcacoa_x + fad_x --> dc2coa_x + fadh2_x 11989 or 12564 or 12742 or 9700
ACOA120OR ddcacoa_x + fad_x --> dd2coa_x + fadh2_x 11989 or 12564 or 12742 or 9700
ACOA140OR fad_x + tdcoa_x --> fadh2_x + td2coa_x 11989 or 12564 or 12742 or 9700
ACOA160OR fad_x + pmtcoa_x --> fadh2_x + hdd2coa_x 11989 or 12564 or 12742 or 9700
ACOA40OR btcoa_x + fad_x --> b2coa_x + fadh2_x 10012 or 11989 or 12564 or 12570 or 12742 or 9700
ACOA60OR fad_x + hxcoa_x --> fadh2_x + hx2coa_x 10012 or 11989 or 12564 or 12570 or 12742 or 9700
ACOA80OR fad_x + occoa_x --> fadh2_x + oc2coa_x 11989 or 12

In [14]:
# Add 8963 acyl-CoA dehydrogenase, citronellyl-CoA dehydrogenase?
Annotation.loc[['8963']]

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
8963,KOG0137: Very-long-chain acyl-CoA dehydrogenase,,,,CYB5,IVDe,Not Essential,"cyto 16.5, cyto_nucl 9, mito 6, pero 4",AVL*


#mito  
10012	mito 25.5, cyto_mito 14 (sigP) IVD, ivd; isovaleryl-CoA dehydrogenase (FAD)  
11989	mito 26.5, cyto_mito 14 (sigP) GCDH, gcdH; glutaryl-CoA dehydrogenase (FAD)  
14070	mito 26, cyto_mito 14.5 (no sigP) ACADSB; short/branched chain acyl-CoA dehydrogenase (branched and C4-C6)  
12570	mito 26 (no sigP) ACADM, acd; medium-chain acyl-CoA dehydrogenase (C4-C16, most active with C6-C12)  
12564	mito 11, cyto 6.5, cyto_nucl 5.5, extr 4, nucl 3.5 (sigP) SRD5A1; 3-oxo-5-alpha-steroid 4-dehydrogenase 1  
#pero  
12742	cyto 7, pero 6, cysk 6, nucl 5, cyto_mito 5 (no sigP) POX1 K00232: E1.3.3.6, ACOX1, ACOX3; acyl-CoA oxidase  
12752	mito 11, cyto 8, pero 4, nucl 3 (no sigP) E1.3.3.6, ACOX1, ACOX3; acyl-CoA oxidase  
9700	cyto 9.5, cyto_nucl 7.5, cysk 7, nucl 4.5, pero 4 (no sigP) POX1 K00232: E1.3.3.6, ACOX1, ACOX3; acyl-CoA oxidase  
12086	cyto 17, cyto_nucl 10, pero 4, cysk 3 (no sigP) ACADS, acd; short-chain acyl-CoA dehydrogenase, mitochondrial, KEGG butyryl-CoA dehydrogenase butyryl hexanoyl (has cytochrome b5 domain), no rxn (C4-C6)  
10408	cyto 17.5, cyto_nucl 10.5, pero 6 (no sigP) ACAD10; acyl-CoA dehydrogenase family member 10, KEGG acyl-CoA dehydrogenase (medium-chain EC, assign C4-C12)  
12989	cyto 15.5, cyto_nucl 8.5, cysk 6, pero 3 (sigP) ACADL, long-chain acyl-CoA dehydrogenase (C8-C22, most active with C14-C16)  
16253	mito 9, cysk 8, cyto 5.5, pero 4, cyto_nucl 3.5 (sigP) ACADVL, very-long-chain acyl-CoA dehydrogenase (C14-C24, most active with C14-C18)  
#peroxisomal acyl-CoA dehydrogenases - use fadh? where is the electron going?

#mito  
short/branched chain, C3-C5  
Change ACOAD8m genes to '10012'  
Change ACOAD9m, ACOAD10m genes to '14070'  
short chain, C4-C6  
Change ACOAD1fm to ACOAD1m, and change genes to '12570 or 14070'  
Remove ACOAD1, ACOAD1f, ACOAD1fr  
Change ACOAD2f to ACOAD2m, change metabolites to mito, make it irreversible  
Change ACOAD2m genes to '12570 or 14070'  
Medium chain, C6-C12  
Make ACOAD3m - ACOAD7m, and add genes '12570'  
#pero  
Change ACOA40OR to ACOAD1p
Change ACOA60OR to ACOAD2p
Change ACOA80OR to ACOAD3p
Change ACOA100OR to ACOAD4p
Change ACOA120OR to ACOAD5p
Change ACOA140OR to ACOAD6p
Change ACOA160OR to ACOAD7p
ACOA40OR - ACOA60OR '10408 or 12086'  
ACOA80OR - ACOA120OR '10408 or 12989'  
ACOA140OR - ACOA160OR '12989 or 16253'  
Add ACOAD8p, and genes '12989 or 16253'

In [15]:
model.reactions.get_by_id('ACOAD8m').gene_reaction_rule = '10012'
model.reactions.get_by_id('ACOAD9m').gene_reaction_rule = '14070'
model.reactions.get_by_id('ACOAD10m').gene_reaction_rule = '14070'
model.reactions.get_by_id('ACOAD1fm').id = 'ACOAD1m'
model.reactions.get_by_id('ACOAD1m').gene_reaction_rule = '12570 or 14070'
model.remove_reactions(['ACOAD1','ACOAD1f','ACOAD1fr'], remove_orphans=True)
r = model.reactions.get_by_id('ACOAD2f').copy()
model.reactions.get_by_id('ACOAD2f').id = 'ACOAD2m'
for m in r.metabolites:
    model.reactions.get_by_id('ACOAD2m').add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_m'): r.get_coefficient(m.id)})
model.reactions.get_by_id('ACOAD2m').lower_bound = 0.0
model.reactions.get_by_id('ACOAD2m').gene_reaction_rule = '12570 or 14070'
r = model.reactions.get_by_id('ACOAD2m').copy()
r.gene_reaction_rule = '12570'
r.id = 'ACOAD3m'
r.name = 'Acyl-CoA dehydrogenase (octanoyl-CoA)'
model.add_reactions([r])
model.reactions.get_by_id('ACOAD3m').add_metabolites({'hxcoa_m': 1.0, 'hx2coa_m': -1.0, 'occoa_m': -1.0, 'oc2coa_m': 1.0})
r = model.reactions.get_by_id('ACOAD2m').copy()
r.gene_reaction_rule = '12570'
r.id = 'ACOAD4m'
r.name = 'Acyl-CoA dehydrogenase (decanoyl-CoA)'
model.add_reactions([r])
model.reactions.get_by_id('ACOAD4m').add_metabolites({'hxcoa_m': 1.0, 'hx2coa_m': -1.0, 'dcacoa_m': -1.0, 'dc2coa_m': 1.0})
r = model.reactions.get_by_id('ACOAD2m').copy()
r.gene_reaction_rule = '12570'
r.id = 'ACOAD5m'
r.name = 'Acyl-CoA dehydrogenase (dodecanoyl-CoA)'
model.add_reactions([r])
model.reactions.get_by_id('ACOAD5m').add_metabolites({'hxcoa_m': 1.0, 'hx2coa_m': -1.0, 'ddcacoa_m': -1.0, 'dd2coa_m': 1.0})
r = model.reactions.get_by_id('ACOAD2m').copy()
r.gene_reaction_rule = '12570'
r.id = 'ACOAD6m'
r.name = 'Acyl-CoA dehydrogenase (tetradecanoyl-CoA)'
model.add_reactions([r])
model.reactions.get_by_id('ACOAD6m').add_metabolites({'hxcoa_m': 1.0, 'hx2coa_m': -1.0, 'tdcoa_m': -1.0, 'td2coa_m': 1.0})
r = model.reactions.get_by_id('ACOAD2m').copy()
r.gene_reaction_rule = '12570'
r.id = 'ACOAD7m'
r.name = 'Acyl-CoA dehydrogenase (palmitoyl-CoA)'
model.add_reactions([r])
model.reactions.get_by_id('ACOAD7m').add_metabolites({'hxcoa_m': 1.0, 'hx2coa_m': -1.0, 'pmtcoa_m': -1.0, 'hdd2coa_m': 1.0})
model.reactions.get_by_id('ACOA40OR').id = 'ACOAD1p'
model.reactions.get_by_id('ACOAD1p').name = 'Acyl-CoA dehydrogenase (butanoyl-CoA)'
model.reactions.get_by_id('ACOAD1p').gene_reaction_rule = '10408 or 12086'
model.reactions.get_by_id('ACOA60OR').id = 'ACOAD2p'
model.reactions.get_by_id('ACOAD2p').name = 'Acyl-CoA dehydrogenase (hexanoyl-CoA)'
model.reactions.get_by_id('ACOAD2p').gene_reaction_rule = '10408 or 12086'
model.reactions.get_by_id('ACOA80OR').id = 'ACOAD3p'
model.reactions.get_by_id('ACOAD3p').name = 'Acyl-CoA dehydrogenase (octanoyl-CoA)'
model.reactions.get_by_id('ACOAD3p').gene_reaction_rule = '10408 or 12989'
model.reactions.get_by_id('ACOA100OR').id = 'ACOAD4p'
model.reactions.get_by_id('ACOAD4p').name = 'Acyl-CoA dehydrogenase (decanoyl-CoA)'
model.reactions.get_by_id('ACOAD4p').gene_reaction_rule = '10408 or 12989'
model.reactions.get_by_id('ACOA120OR').id = 'ACOAD5p'
model.reactions.get_by_id('ACOAD5p').name = 'Acyl-CoA dehydrogenase (dodecanoyl-CoA)'
model.reactions.get_by_id('ACOAD5p').gene_reaction_rule = '10408 or 12989'
model.reactions.get_by_id('ACOA140OR').id = 'ACOAD6p'
model.reactions.get_by_id('ACOAD6p').name = 'Acyl-CoA dehydrogenase (tetradecanoyl-CoA)'
model.reactions.get_by_id('ACOAD6p').gene_reaction_rule = '12989 or 16253'
model.reactions.get_by_id('ACOA160OR').id = 'ACOAD7p'
model.reactions.get_by_id('ACOAD7p').name = 'Acyl-CoA dehydrogenase (palmitoyl-CoA)'
model.reactions.get_by_id('ACOAD7p').gene_reaction_rule = '12989 or 16253'
r = model.reactions.get_by_id('ACOAD7p').copy()
r.id = 'ACOAD8p'
r.name = 'Acyl-CoA dehydrogenase (stearoyl-CoA)'
model.add_reactions([r])
model.reactions.get_by_id('ACOAD8p').add_metabolites({'pmtcoa_x': 1.0, 'hdd2coa_x': -1.0, 'stcoa_x': -1.0, 'od2coa_x': 1.0})

In [16]:
for r in sorted(model.genes.get_by_id('10012').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('11989').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('14070').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12570').reactions, key=lambda x: x.id):
    if not r.id.startswith('FAO'):
        print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12564').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ACOAD8m fad_m + ivcoa_m --> 3mb2coa_m + fadh2_m 10012
MBCOAi fad_c + ivcoa_c --> 3mb2coa_c + fadh2_c 10012 or 12570

GLUTCOADHc fad_c + glutcoa_c + h_c --> b2coa_c + co2_c + fadh2_c 11989
GLUTCOADHm fad_m + glutcoa_m + h_m --> b2coa_m + co2_m + fadh2_m 11989

ACOAD10m 2mbcoa_m + fad_m --> 2mb2coa_m + fadh2_m 14070
ACOAD1m btcoa_m + fad_m --> b2coa_m + fadh2_m 12570 or 14070
ACOAD2m fad_m + hxcoa_m --> fadh2_m + hx2coa_m 12570 or 14070
ACOAD9m fad_m + ibcoa_m --> 2mp2coa_m + fadh2_m 14070
PPCOAOm fad_m + ppcoa_m --> fadh2_m + prpncoa_m 10408 or 12570 or 14070

ACOAD1m btcoa_m + fad_m --> b2coa_m + fadh2_m 12570 or 14070
ACOAD2m fad_m + hxcoa_m --> fadh2_m + hx2coa_m 12570 or 14070
ACOAD3m fad_m + occoa_m --> fadh2_m + oc2coa_m 12570
ACOAD4m dcacoa_m + fad_m --> dc2coa_m + fadh2_m 12570
ACOAD5m ddcacoa_m + fad_m --> dd2coa_m + fadh2_m 12570
ACOAD6m fad_m + tdcoa_m --> fadh2_m + td2coa_m 12570
ACOAD7m fad_m + pmtcoa_m --> fadh2_m + hdd2coa_m 12570
MBCOAi fad_c + ivcoa_c --> 3mb2coa_c + fa

Remove MBCOAi, GLUTCOADHc (orphan), GLUTCOADHm has correct gene  
Change PPCOAOm genes to '14070'

In [17]:
model.remove_reactions(['MBCOAi','GLUTCOADHc'], remove_orphans=True)
model.reactions.get_by_id('PPCOAOm').gene_reaction_rule = '14070'

In [18]:
for r in sorted(model.metabolites.get_by_id('ppcoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

CSNAT3x crn_x + ppcoa_x <=> coa_x + pcrn_x 14245
PEROXx 3.0 coa_x + 3.0 h2o_x + 3.0 nad_x + 2.0 o2_x + pristcoa_x --> accoa_x + dmnoncoa_x + 2.0 h2o2_x + 3.0 h_x + 3.0 nadh_x + 2.0 ppcoa_x (ACOX3 and EHHADH and 11362 and 13813) or (ACOX3 and EHHADH and 11362 and 9065) or (Acox3 and Ehhadh and 11362 and 13813) or (Acox3 and Ehhadh and 11362 and 9065)
SCP2x coa_x + dhocholoylcoa_x --> dgcholcoa_x + ppcoa_x 14055
SCP3x coa_x + dhcholestancoa_x + o2_x --> dcholcoa_x + h2o_x + ppcoa_x 14055
SCPx cholcoaone_x + coa_x --> cholcoa_x + ppcoa_x 14055


In [19]:
for r in sorted(model.metabolites.get_by_id('ppcoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('3mb2coa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('3mgcoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ACCOAL atp_c + coa_c + ppa_c --> adp_c + pi_c + ppcoa_c 14597
ACS2 atp_c + coa_c + ppa_c --> amp_c + ppcoa_c + ppi_c 14597 or 15276
PPCSCT ppcoa_c + succ_c --> ppa_c + succoa_c 13797
yli_R0594 coa_c + ppad_c <=> amp_c + 2.0 h_c + ppcoa_c 14597

MCCC 3mb2coa_c + atp_c + hco3_c --> 3mgcoa_c + adp_c + h_c + pi_c 12867
yli_R1593 3hivcoa_c <=> 3mb2coa_c + h2o_c 14805

MCCC 3mb2coa_c + atp_c + hco3_c --> 3mgcoa_c + adp_c + h_c + pi_c 12867


In [20]:
for r in sorted(model.metabolites.get_by_id('ppcoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('ivcoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('3mb2coa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('3mgcoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ACACT10m 2maacoa_m + coa_m <=> accoa_m + ppcoa_m 8678
ACCOALm atp_m + coa_m + ppa_m --> amp_m + ppcoa_m + ppi_m 14597
CSNAT2m coa_m + pcrn_m <=> crn_m + ppcoa_m 14245
FAOXC150m 6.0 coa_m + 6.0 fad_m + 6.0 h2o_m + 6.0 nad_m + ptdcacoa_m --> 6.0 accoa_m + 6.0 fadh2_m + 6.0 h_m + 6.0 nadh_m + ppcoa_m 12570
FAOXC170m 7.0 coa_m + 7.0 fad_m + 7.0 h2o_m + hpdcacoa_m + 7.0 nad_m --> 7.0 accoa_m + 7.0 fadh2_m + 7.0 h_m + 7.0 nadh_m + ppcoa_m 12570
MCITSm h2o_m + oaa_m + ppcoa_m --> 2mcit_m + coa_m + h_m 11331
MMSAD1m 2mop_m + coa_m + nad_m --> co2_m + nadh_m + ppcoa_m 8975
OBDHm 2obut_m + coa_m + nad_m --> co2_m + nadh_m + ppcoa_m (10040 and 11183 and 12566 and 15436) or (10040 and 11188 and 12566 and 15436)
PACPT coa_m + ppad_m <=> amp_m + h_m + ppcoa_m 14597
PPACOALm atp_m + coa_m + ppa_m --> amp_m + h_m + ppcoa_m + ppi_m 14597
PPCOACm atp_m + hco3_m + ppcoa_m --> adp_m + h_m + mmcoa__S_m + pi_m (PCCB and 13009) or (Pccb and 13009)
PPCOAOm fad_m + ppcoa_m --> fadh2_m + prpncoa_m 14070

ACOAD8

In [21]:
temp = ['14597','15276','8975','13797','13009']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
14597,"K01895: ACSS, acs; acetyl-CoA synthetase",,"ACS1,ACS2","ACSS1,ACSS2",ACS2,ACSS1,Not Essential,"cyto 13, plas 5, mito 3, pero 2, mito_nucl 2",SSE*
15276,"K01907: AACS, acsA; acetoacetyl-CoA synthetase",,,AACS,ACS1,AACS,Not Essential,"plas 9, pero 6, mito 5, cyto 5, cyto_mito 5",SKL*
8975,"K00140: mmsA, iolA, ALDH6A1; malonate-semialde...",,,ALDH6A1,UGA2,ALDH6,Not Essential,mito 27,THH*
13797,"K01067: E3.1.2.1, ACH1; acetyl-CoA hydrolase",,ACH1,,ACH1,,Not Essential,"mito 23, cyto 3",GWS*
13009,KOG1161: Protein involved in vacuolar polyphos...,,VTC4,PCCA,VTC4,,Not Essential,"plas 11, cyto 5.5, cyto_nucl 5.5, pero 5, nucl...",QGI*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
14597,0.134527,0.0997527,0.107444,-1.05713,0.0951078,-0.0881028,0.151616,0.954851,0.0158946,-0.116256,0.000665712
15276,-0.192767,0.221437,0.0764749,-0.176646,0.00381209,-0.117964,-0.0335189,-0.260812,-0.126874,-0.0575812,-0.262471
8975,-0.183461,0.337248,-0.0360008,-0.0852432,0.164365,0.133413,-0.0728905,-0.353268,-0.148485,-0.0951318,0.168732
13797,-0.0172407,-0.175817,-0.123985,-0.429914,-0.168931,-0.0772301,-0.0119267,0.333993,-0.0622429,-0.0438979,-0.158202
13009,0.0956749,-0.138998,-0.38607,0.0215119,-0.68409,0.176859,-1.08209,-0.72106,-1.62232,-1.54764,0.721124


14597	cyto 13, plas 5, mito 3, pero 2, mito_nucl 2	K01895: ACSS, acs; acetyl-CoA synthetase  
15276	plas 9, pero 6, mito 5, cyto 5, cyto_mito 5	K01907: AACS, acsA; acetoacetyl-CoA synthetase  
8975	mito 27	K00140: mmsA, iolA, ALDH6A1; malonate-semialdehyde dehydrogenase (acetylating) / methylmalonate-semialdehyde dehydrogenase  
13797	mito 23, cyto 3	K01067: E3.1.2.1, ACH1; acetyl-CoA hydrolase  
13009	plas 11, cyto 5.5, cyto_nucl 5.5, pero 5, nucl 4.5	KOG1161: Protein involved in vacuolar polyphosphate accumulation, contains SPX domain

12867	mito 27	K01968: E6.4.1.4A; 3-methylcrotonyl-CoA carboxylase alpha subunit  
15694	mito 27	K01969: E6.4.1.4B; 3-methylcrotonyl-CoA carboxylase beta subunit  
16128	mito 26	K05607: AUH; methylglutaconyl-CoA hydratase

14805	mito 26.5, cyto_mito 14	K07511: ECHS1; enoyl-CoA hydratase (mitochondria), both degradation and biosynthesis  
11362	cyto 15, mito 6, pero 4, mito_nucl 4	K14729: FOX2; multifunctional beta-oxidation protein (peroxisome)  
11907	extr 9, mito 8, cyto 5.5, cyto_nucl 4, pero 3	K12663: ECH1; Delta(3,5)-Delta(2,4)-dienoyl-CoA isomerase (peroxisome), no correct rxns in BiGG  

PPACOALm (incorrect stoic) = ACCOALm (incorrect stoic), ACCOAL (adp, cyto) = APAT + PACPT  
13009 not relevant remove PPCOACm  
ECH_3hivcoa is incorrect, 3hivcoa can be made from 2mb2coa by ech  
yli_R1593 is cyto version of ECH_3hivcoa  
MCCCrm is correct, remove MCCC and MCTC
Change ACS, ACS2 genes to '14597'

In [22]:
model.remove_reactions(['ACCOAL','ACCOALm','PPACOALm','APAT','PACPT','PPCOACm',
                        'ECH_3hivcoa','yli_R1593','MCCC','MCTC'], remove_orphans=True)
model.reactions.get_by_id('ACS').gene_reaction_rule = '14597'
model.reactions.get_by_id('ACS2').gene_reaction_rule = '14597'

In [23]:
for r in sorted(model.genes.get_by_id('14597').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15276').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8975').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13797').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ACS ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c 14597
ACS2 atp_c + coa_c + ppa_c --> amp_c + ppcoa_c + ppi_c 14597
ACSm ac_m + atp_m + coa_m --> accoa_m + amp_m + ppi_m 14597
ACSp ac_x + atp_x + coa_x --> accoa_x + amp_x + ppi_x 14597
yli_R0594 coa_c + ppad_c <=> amp_c + 2.0 h_c + ppcoa_c 14597
yli_R0596 atp_c + 2.0 h_c + yli_M00161_c <=> ppad_c + ppi_c 14597
yli_R1565 HC01672_c + coa_c --> accoa_c + amp_c 14597
yli_R1566 ac_c + atp_c <=> HC01672_c + ppi_c 14597

AACOAT acac_c + atp_c + coa_c <=> aacoa_c + amp_c + ppi_c 15276

MMSAD1m 2mop_m + coa_m + nad_m --> co2_m + nadh_m + ppcoa_m 8975
MMSAD3m coa_m + msa_m + nad_m --> accoa_m + co2_m + nadh_m 8975

ACOAH ac_c + coa_c + h_c --> accoa_c + h2o_c 13797
ACOAHim accoa_m + h2o_m --> ac_m + coa_m + h_m 13797
PPCSCT ppcoa_c + succ_c --> ppa_c + succoa_c 13797


ACS is cyto, mito, nucl in S. cer remove ACSp  
HC01672 is acetyl-AMP remove yli  
13797 is mito acetyl-coa hydrolase  
13048	cyto 20.5, cyto_mito 11, nucl 5	K01067: E3.1.2.1, ACH1; acetyl-CoA hydrolase -> ACOAH with reverse direction
Remove PPCSCT

In [24]:
model.remove_reactions(['ACSp','yli_R0594','yli_R0596','yli_R1565','yli_R1566'], remove_orphans=True)
model.reactions.get_by_id('ACOAH').gene_reaction_rule = '13048'
model.reactions.get_by_id('ACOAH').lower_bound = -1000.0
model.reactions.get_by_id('ACOAH').upper_bound = 0.0
model.remove_reactions(['PPCSCT'], remove_orphans=True)

In [25]:
for r in sorted(model.genes.get_by_id('12867').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15694').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('16128').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ACCOAC accoa_c + atp_c + hco3_c --> adp_c + h_c + malcoa_c + pi_c 8639 or (PP_0559 and PP_1607 and PP_1996 and 12867) or (b0185 and b2316 and b3255 and 12867)
ACCOAC_1 accoa_h + cbtnCCP_h + h_h <=> btnCCP_h + malcoa_h (CRv4_Au5_s12_g2759_t1 and CRv4_Au5_s17_g7349_t1) or (CRv4_Au5_s12_g2759_t1 and CRv4_Au5_s1_g1722_t1) or (CRv4_Au5_s12_g3449_t1 and CRv4_Au5_s17_g7349_t1) or (CRv4_Au5_s12_g3449_t1 and CRv4_Au5_s1_g1722_t1) or (CRv4_Au5_s17_g7349_t1 and 12867) or (CRv4_Au5_s1_g1722_t1 and 12867)
ACCOAhi accoa_h + atp_h + hco3_h --> adp_h + malcoa_h + pi_h (CRv4_Au5_s12_g2759_t1 and CRv4_Au5_s17_g7349_t1) or (CRv4_Au5_s12_g3449_t1 and CRv4_Au5_s17_g7349_t1) or (CRv4_Au5_s17_g7349_t1 and 12867)
BTNC atp_h + btnCCP_h + hco3_h --> adp_h + cbtnCCP_h + h_h + pi_h (CRv4_Au5_s17_g7349_t1 and 12867) or (CRv4_Au5_s1_g1722_t1 and 12867)
MCCCrm 3mb2coa_m + atp_m + hco3_m <=> 3mgcoa_m + adp_m + h_m + pi_m 12867 and 15694

MCCCrm 3mb2coa_m + atp_m + hco3_m <=> 3mgcoa_m + adp_m + h_m + pi_m 12867 and 15

Change ACCOAC genes to '8639'  
Remove ACCOAC_1, ACCOAhi, BTNC  
Remove 3HBCDm (ECOAH1m), HBCHLR (ECOAH1)  
Remove C2M26DCOAHLm, T2M26DCOAHLm, C2M26DCOAHLx, T2M26DCOAHLx (orphans)  
Remove HPHL (only in iRC, part of autotrophic co2 fixation)  
Change PRPNCOAHYDm genes to '14805'

In [26]:
model.reactions.get_by_id('ACCOAC').gene_reaction_rule = '8639'
model.remove_reactions(['ACCOAC_1','ACCOAhi','BTNC','3HBCDm','HBCHLR','C2M26DCOAHLm','T2M26DCOAHLm','C2M26DCOAHLx','T2M26DCOAHLx','HPHL'], remove_orphans=True)
model.reactions.get_by_id('PRPNCOAHYDm').gene_reaction_rule = '14805'

In [27]:
for r in sorted(model.genes.get_by_id('14805').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('11362').reactions, key=lambda x: x.id):
    if not r.id.startswith('FAO'):
        print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('11907').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

DHACOAH 23dhacoa_c + h2o_c <=> 3hadpcoa_c 14805
ECOAH1 3hbcoa_c <=> b2coa_c + h2o_c 14805 or 16128
ECOAH12 2mp2coa_c + h2o_c --> 3hibutcoa_c 14805
ECOAH12m 2mp2coa_m + h2o_m <=> 3hibutcoa_m 11907 or 14805 or (HADHA and HADHB) or (Hadha and Hadhb)
ECOAH1m 3hbcoa_m <=> b2coa_m + h2o_m 11907 or 14805 or 16128 or (HADHA and HADHB) or (Hadha and Hadhb)
ECOAH2 3hhcoa_c <=> h2o_c + hx2coa_c 14805
ECOAH3 3hocoa_c <=> h2o_c + oc2coa_c 14805
ECOAH4 3hdcoa_c <=> dc2coa_c + h2o_c 14805
ECOAH4m 3hdcoa_m <=> dc2coa_m + h2o_m 11907 or 14805
ECOAH5 3hddcoa_c <=> dd2coa_c + h2o_c 14805
ECOAH5m 3hddcoa_m <=> dd2coa_m + h2o_m 11907 or 14805
ECOAH6 3htdcoa_c <=> h2o_c + td2coa_c 14805
ECOAH6m 3htdcoa_m <=> h2o_m + td2coa_m 11907 or 14805
ECOAH7 3hhdcoa_c <=> h2o_c + hdd2coa_c 14805
ECOAH7m 3hhdcoa_m <=> h2o_m + hdd2coa_m 11907 or 14805
ECOAH9ir 2mb2coa_c + h2o_c --> 3hmbcoa_c 14805
ECOAH9m 2mb2coa_m + h2o_m <=> 3hmbcoa_m 11907 or 14805 or (HADHA and HADHB) or (Hadha and Hadhb)
HADPCOADH 3hadpcoa_c --> h2_

In [28]:
# Remove DHACOAH (part of phenylacetate degradation), HADPCOADH (wrong) cyto rxns 
# Remove HMR_3116 (C18 fatty acid mito degradation)
# Remove yli_R0191,yli_R0192,yli_R0243 (C26 fatty acid mito degradation)
model.remove_reactions(['DHACOAH','HADPCOADH','HMR_3116','yli_R0191','yli_R0192','yli_R0243'], remove_orphans=True)
# Remove C3STKR2r, ERG27 is missing
model.remove_reactions(['C3STKR2r'], remove_orphans=True)
# PEROXx is wrong, need 3 o2
model.remove_reactions(['PEROXx'])

In [29]:
temp = Annotation.index[Annotation['Sc288c Orthologs'].str.contains('ERG')]
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
8671,"K00222: TM7SF2, ERG24; Delta14-sterol reductase",A,ERG24,"LBR,TM7SF2",ERG24,LBR,Essential,"plas 13, mito 9, cyto 3",YVY*
8678,"K00626: E2.3.1.9, atoB; acetyl-CoA C-acetyltra...",,ERG10,ACAT1,ERG10,ACAT1,Not Essential,"mito 23.5, cyto_mito 14",QRL*
8702,"K00938: E2.7.4.2, mvaK2; phosphomevalonate kinase",,ERG8,,ERG8,,Essential,"cyto 16, cyto_mito 11.833, cyto_nucl 9.833, mi...",KKR*
8758,"K00869: E2.7.1.36, MVK, mvaK1; mevalonate kinase",,ERG12,MVK,ERG12,MVK,Essential,"cyto 13, extr 9, cyto_nucl 8.833, cyto_mito 7....",VCA*
8781,"K00227: SC5DL, ERG3; Delta7-sterol 5-desaturase",,ERG3,SC5D,ERG3,SC5D,Not Essential,plas 26,ARK*
8835,"K07748: E1.1.1.170, NSDHL, ERG26; sterol-4alph...",,ERG26,NSDHL,ERG26,NSDHL,Essential,"cyto 13, cyto_nucl 8.833, cyto_mito 8.833, per...",KKA*
9240,"K01852: LSS, ERG7; lanosterol synthase",,ERG7,LSS,ERG7,LSS,Essential,"cysk 22, cyto 5",EGW*
10113,K09829: ERG2; C-8 sterol isomerase,S,ERG2,SIGMAR1,ERG2,SIGMA,Essential,"plas 17, E.R. 5, mito 2, extr 2",GKI*
10981,K00223: ERG4; Delta24(24(1))-sterol reductase,,ERG4,,ERG4,LBR,Not Essential,"plas 18, E.R. 5, mito 3",YVF*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
13724,-0.0242052,-0.13888,-0.054681,0.0237792,-0.219687,0.208523,-0.0744516,-0.261135,-0.0276679,-0.136039,-0.093421


In [30]:
temp = Annotation.index[Annotation['Combined Annotations'].str.contains('hydroxysteroid')]
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
11599,KOG1430: C-3 sterol dehydrogenase/3-beta-hydro...,,,,,,Not Essential,cyto 25,VGG*
11765,K13370: HSD17B8; 17beta-estradiol 17-dehydroge...,,,BDH2,SPS19,DHRS4,Not Essential,"mito 19.5, cyto_mito 12, cyto 3.5, extr 2",TPT*
13356,KOG1014: 17 beta-hydroxysteroid dehydrogenase ...,,,,,,Not Essential,"cyto 9.5, cyto_nucl 6.5, mito 6, extr 6, nucl 2.5",EKF*
14267,K00038: E1.1.1.53; 3alpha(or 20beta)-hydroxyst...,,,,SPS19,DHRS4,Not Essential,"cyto 16.5, cyto_nucl 9, extr 6, mito 4",KWC*
15346,KOG1014: 17 beta-hydroxysteroid dehydrogenase ...,S,,,,,Not Essential,"mito 19, extr 5, cyto 2",EKW*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
11599,-0.149168,0.565516,0.0343418,0.138936,0.313571,0.0936143,0.0472852,-0.335484,-0.0458369,-0.0802779,-0.310492
11765,0.107044,0.135627,0.101293,0.0450894,0.0149791,-0.0316738,0.0112884,0.0701067,-0.118633,-0.0299555,0.387225
13356,0.0353567,0.0712028,-0.20468,-0.327876,0.127818,-0.0408538,-0.806669,-0.136497,0.0486512,-0.135179,0.261526
14267,0.207207,-0.248473,0.215494,0.063958,-0.173733,0.246239,0.0646736,0.0278355,-0.138097,-0.164489,0.459204
15346,-0.0721767,0.153242,-0.0470953,-0.000743865,-0.246982,-0.123396,0.0771705,-0.0813489,0.222447,-0.0457231,0.163785


In [31]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('ECOAH') and r.id.endswith('m'):
        print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('ECOAH') and (r.id.endswith('p') or r.id.endswith('x') or r.id.endswith('_1')):
        print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('ECOAH') and not (r.id.endswith('m') or r.id.endswith('p') or r.id.endswith('x') or r.id.endswith('_1')):
        print(r.id, r.reaction, r.gene_reaction_rule)

ECOAH12m 2mp2coa_m + h2o_m <=> 3hibutcoa_m 11907 or 14805 or (HADHA and HADHB) or (Hadha and Hadhb)
ECOAH1m 3hbcoa_m <=> b2coa_m + h2o_m 11907 or 14805 or 16128 or (HADHA and HADHB) or (Hadha and Hadhb)
ECOAH2m 3hhcoa_m <=> h2o_m + hx2coa_m 11907
ECOAH3m 3hocoa_m <=> h2o_m + oc2coa_m 11907
ECOAH4m 3hdcoa_m <=> dc2coa_m + h2o_m 11907 or 14805
ECOAH5m 3hddcoa_m <=> dd2coa_m + h2o_m 11907 or 14805
ECOAH6m 3htdcoa_m <=> h2o_m + td2coa_m 11907 or 14805
ECOAH7m 3hhdcoa_m <=> h2o_m + hdd2coa_m 11907 or 14805
ECOAH9m 2mb2coa_m + h2o_m <=> 3hmbcoa_m 11907 or 14805 or (HADHA and HADHB) or (Hadha and Hadhb)

ECOAH11p h2o_x + hxc2coa_x <=> 3hxccoa_x 11362
ECOAH1x 3hbcoa_x <=> b2coa_x + h2o_x 11362 or 11907
ECOAH2_1 h2o_x + hx2coa_x <=> 3hhcoa_x 11362
ECOAH3_1 h2o_x + oc2coa_x <=> 3hocoa_x 11362
ECOAH4p 3hdcoa_x <=> dc2coa_x + h2o_x 11362
ECOAH5p 3hddcoa_x <=> dd2coa_x + h2o_x 11362
ECOAH6p 3htdcoa_x <=> h2o_x + td2coa_x 11362
ECOAH7p 3hhdcoa_x <=> h2o_x + hdd2coa_x 11362
ECOAH8p 3hodcoa_x <=> h2o_

In [32]:
temp = ['11907','14805','16515','11362','9469']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
11907,"K12663: ECH1; Delta(3,5)-Delta(2,4)-dienoyl-Co...",,,ECH1,,ECH1,Not Essential,"extr 9, mito 8, cyto 5.5, cyto_nucl 4, pero 3",AKL*
14805,K07511: ECHS1; enoyl-CoA hydratase,,,ECHS1,EHD3,ECHS1,Not Essential,"mito 26.5, cyto_mito 14",TNS*
16515,HMMPfam:Enoyl-CoA hydratase/isomerase family:P...,,,,,ECHS1,Not Essential,"mito 10.5, cyto_mito 9.833, cyto 8, cyto_nucl ...",ARL*
11362,K14729: FOX2; multifunctional beta-oxidation p...,,FOX2,HSD17B4,FOX2,HSD17,Not Essential,"cyto 15, mito 6, pero 4, mito_nucl 4",LAQ*
9469,KOG1206: Peroxisomal multifunctional beta-oxid...,,,,FOX2,HSD17,Not Essential,cyto 25,SKL*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
11907,0.114424,-0.0555856,0.084374,0.0957274,0.0248728,0.0169171,-1.15316,0.0380234,-0.189194,-0.250849,-0.147514
14805,0.195211,0.271089,-0.627815,-1.43546,-0.649913,-0.937919,-2.90688,-4.21411,0.0902761,-0.0765189,0.461105
16515,-0.00111379,0.314535,0.231734,0.369911,0.142545,0.523174,-0.0657867,0.443237,-0.0131392,-0.0949175,0.317089
11362,0.0751211,0.0521463,0.00810646,-0.0907637,-0.150565,0.0238548,-1.49407,0.0551923,0.104864,-0.187183,-0.0642188
9469,-0.132763,0.0961697,-0.120022,-0.0748623,0.240087,-0.313024,0.0222837,0.0728534,0.0136585,-0.0957982,0.0982957


11907 perox Delta3,5-Delta2,4-dienoyl-CoA isomerase  
14805 ECHS1 mito enoyl-coA hydratase  
16515 Enoyl-CoA hydratase/isomerase family, no rxn, (paperblast to Mevalonyl-coenzyme A hydratase sidH in pero), protein upregulated in p-coumarate, couA?  
11362 FOX2 HSD17B4 perox multifunctional beta-oxidation protein  
9469 perox multifunctional beta-oxidation protein, but only last two domains, blast to peroxisomal dehydratase / enoyl-CoA hydratase 2, no rxn, upregulated in coumarate

Change ECOAH1x to ECOAH1p  
Change ECOAH2_1 to ECOAH2p  
Change ECOAH3_1 to ECOAH3p  

Change ECOAHxp genes to '11362'  
Change ECOAHxm genes to '14805'  
ECOAH8m is missing, add one?  
Remove ECOAHx cyto reactions  

In [33]:
model.reactions.get_by_id('ECOAH1x').id = 'ECOAH1p'
model.reactions.get_by_id('ECOAH2_1').id = 'ECOAH2p'
model.reactions.get_by_id('ECOAH3_1').id = 'ECOAH3p'
model.reactions.get_by_id('ECOAH1p').gene_reaction_rule = '11362'
model.reactions.get_by_id('ECOAH2p').gene_reaction_rule = '11362'
model.reactions.get_by_id('ECOAH3p').gene_reaction_rule = '11362'
model.reactions.get_by_id('ECOAH4p').gene_reaction_rule = '11362'
model.reactions.get_by_id('ECOAH5p').gene_reaction_rule = '11362'
model.reactions.get_by_id('ECOAH6p').gene_reaction_rule = '11362'
model.reactions.get_by_id('ECOAH7p').gene_reaction_rule = '11362'
model.reactions.get_by_id('ECOAH8p').gene_reaction_rule = '11362'
model.reactions.get_by_id('ECOAH11p').gene_reaction_rule = '11362'
model.reactions.get_by_id('ECOAH1m').gene_reaction_rule = '14805'
model.reactions.get_by_id('ECOAH2m').gene_reaction_rule = '14805'
model.reactions.get_by_id('ECOAH3m').gene_reaction_rule = '14805'
model.reactions.get_by_id('ECOAH4m').gene_reaction_rule = '14805'
model.reactions.get_by_id('ECOAH5m').gene_reaction_rule = '14805'
model.reactions.get_by_id('ECOAH6m').gene_reaction_rule = '14805'
model.reactions.get_by_id('ECOAH7m').gene_reaction_rule = '14805'
model.reactions.get_by_id('ECOAH9m').gene_reaction_rule = '14805'
model.reactions.get_by_id('ECOAH12m').gene_reaction_rule = '14805'
model.remove_reactions(['ECOAH1','ECOAH2','ECOAH3','ECOAH4','ECOAH5','ECOAH6','ECOAH7','ECOAH9ir','ECOAH12'], remove_orphans=True)

In [34]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('HACD') and r.id.endswith('m'):
        print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('HACD') and (r.id.endswith('p') or r.id.endswith('x') or r.id.endswith('_1')):
        print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('HACD') and not (r.id.endswith('m') or r.id.endswith('p') or r.id.endswith('x') or r.id.endswith('_1')):
        print(r.id, r.reaction, r.gene_reaction_rule)

HACD1m aacoa_m + h_m + nadh_m <=> 3hbcoa_m + nad_m 11203 or 15180 or 16284 or (HADHA and HADHB) or (Hadha and Hadhb)
HACD2m 3ohcoa_m + h_m + nadh_m <=> 3hhcoa_m + nad_m 11362
HACD3m 3oocoa_m + h_m + nadh_m <=> 3hocoa_m + nad_m 11362
HACD4m 3odcoa_m + h_m + nadh_m <=> 3hdcoa_m + nad_m 11362
HACD5m 3oddcoa_m + h_m + nadh_m <=> 3hddcoa_m + nad_m 11362
HACD6m 3otdcoa_m + h_m + nadh_m <=> 3htdcoa_m + nad_m 11362
HACD7m 3ohdcoa_m + h_m + nadh_m <=> 3hhdcoa_m + nad_m 11362
HACD9m 3hmbcoa_m + nad_m <=> 2maacoa_m + h_m + nadh_m 11362 or 16284

HACD10p 3hxccoa_x + nad_x <=> 3ohxccoa_x + h_x + nadh_x 11362
HACD1x aacoa_x + h_x + nadh_x <=> 3hbcoa_x + nad_x 11362
HACD2_1 3hhcoa_x + nad_x <=> 3ohcoa_x + h_x + nadh_x 11362
HACD3_1 3hocoa_x + nad_x <=> 3oocoa_x + h_x + nadh_x 11362
HACD4p 3odcoa_x + h_x + nadh_x <=> 3hdcoa_x + nad_x 11362
HACD5p 3oddcoa_x + h_x + nadh_x <=> 3hddcoa_x + nad_x 11362
HACD6p 3otdcoa_x + h_x + nadh_x <=> 3htdcoa_x + nad_x 11362
HACD7p 3ohdcoa_x + h_x + nadh_x <=> 3hhdcoa_

In [35]:
temp = ['11203','15180','16284','13606','11362']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
11203,K00022: HADH; 3-hydroxyacyl-CoA dehydrogenase,,,HADH,,HADH,Not Essential,"mito 20, cyto_mito 13.333, cyto 4.5, cyto_nucl...",YSS*
15180,"K00074: paaH, hbd, fadB, mmgB; 3-hydroxybutyry...",,,HADH,,HADH,Not Essential,"cyto 13, cyto_nucl 8, cysk 8, mito 4",YTAR
16284,K08683: HSD17B10; 3-hydroxyacyl-CoA dehydrogen...,S,,HSD17B10,,HSD17,Not Essential,"extr 11, mito 5, plas 5, E.R. 4",AKM*
13606,KOG1199: Short-chain alcohol dehydrogenase/3-h...,S,,,,HSD17,Not Essential,"cyto 11.5, cyto_mito 10, mito 7.5, extr 7",ASF*
11362,K14729: FOX2; multifunctional beta-oxidation p...,,FOX2,HSD17B4,FOX2,HSD17,Not Essential,"cyto 15, mito 6, pero 4, mito_nucl 4",LAQ*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
11203,-0.0203069,-0.0319005,0.142411,0.0669948,0.031557,0.0807583,-1.34826,-1.89792,0.232648,-0.109088,0.193759
15180,-0.0297025,0.104901,-0.197956,-0.248617,-0.285895,-0.000632661,-0.205114,-0.216211,-0.184384,0.068575,-0.0350937
16284,0.26627,0.405879,0.10098,0.288009,0.532942,0.271564,0.124402,0.270842,-0.788518,-0.311085,-0.82854
13606,0.279153,-0.0459621,0.327194,0.354471,0.719525,0.273068,-0.217612,0.0319699,0.37306,0.295145,0.0925538
11362,0.0751211,0.0521463,0.00810646,-0.0907637,-0.150565,0.0238548,-1.49407,0.0551923,0.104864,-0.187183,-0.0642188


11203 HADH 3-hydroxyacyl-CoA dehydrogenase 1.1.1.35 beta oxidation (mito, medium and short chain)  
15180 HADH paaH, hbd, fadB, mmgB; 3-hydroxybutyryl-CoA dehydrogenase 1.1.1.157 (S)-3-Hydroxybutanoyl-CoA + NADP+ <=> Acetoacetyl-CoA + NADPH + H+, also (3S)-3-Hydroxyadipyl-CoA + NAD+ <=> 3-Oxoadipyl-CoA + NADH + H+  
16284 HSD17B10 3-hydroxyacyl-CoA dehydrogenase / 3-hydroxy-2-methylbutyryl-CoA dehydrogenase 1.1.1.35 beta oxidation 1.1.1.178 (2S,3S)-3-Hydroxy-2-methylbutanoyl-CoA + NAD+ <=> 2-Methylacetoacetyl-CoA + NADH + H+  
13606 3-hydroxyacyl-CoA dehydrogenase / 3-hydroxy-2-methylbutyryl-CoA dehydrogenase 1.1.1.35 beta oxidation 1.1.1.178 (2S,3S)-3-Hydroxy-2-methylbutanoyl-CoA + NAD+ <=> 2-Methylacetoacetyl-CoA + NADH + H+, no rxn  
11362 FOX2 HSD17B4 pero multifunctional beta-oxidation protein  

Change HACD1x to HACD1p  
Change HACD2_1 to HACD2p  
Change HACD3_1 to HACD3p  
Change HACDxp genes to '11362'  
Change HACDxm genes to '11203'  
HACD9m is for 2-methylacetoacetyl-CoA, change genes to '13606 or 16284'  
Change HBCO_nadp genes to '15180', HADPCOADH3 is correct  
Remove HBCO_nadp_m 

In [36]:
model.reactions.get_by_id('HACD1x').id = 'HACD1p'
model.reactions.get_by_id('HACD2_1').id = 'HACD2p'
model.reactions.get_by_id('HACD3_1').id = 'HACD3p'
model.reactions.get_by_id('HACD1p').gene_reaction_rule = '11362'
model.reactions.get_by_id('HACD2p').gene_reaction_rule = '11362'
model.reactions.get_by_id('HACD3p').gene_reaction_rule = '11362'
model.reactions.get_by_id('HACD4p').gene_reaction_rule = '11362'
model.reactions.get_by_id('HACD5p').gene_reaction_rule = '11362'
model.reactions.get_by_id('HACD6p').gene_reaction_rule = '11362'
model.reactions.get_by_id('HACD7p').gene_reaction_rule = '11362'
model.reactions.get_by_id('HACD8p').gene_reaction_rule = '11362'
model.reactions.get_by_id('HACD10p').gene_reaction_rule = '11362'
model.reactions.get_by_id('HACD1m').gene_reaction_rule = '11203'
model.reactions.get_by_id('HACD2m').gene_reaction_rule = '11203'
model.reactions.get_by_id('HACD3m').gene_reaction_rule = '11203'
model.reactions.get_by_id('HACD4m').gene_reaction_rule = '11203'
model.reactions.get_by_id('HACD5m').gene_reaction_rule = '11203'
model.reactions.get_by_id('HACD6m').gene_reaction_rule = '11203'
model.reactions.get_by_id('HACD7m').gene_reaction_rule = '11203'
model.reactions.get_by_id('HACD9m').gene_reaction_rule = '13606 or 16284'
model.remove_reactions(['HACD1i','HACD2i','HACD3i','HACD4i','HACD5i','HACD6i','HACD7i'], remove_orphans=True)
model.reactions.get_by_id('HBCO_nadp').gene_reaction_rule = '15180'
model.remove_reactions(['HBCO_nadp_m'], remove_orphans=True)

In [37]:
for r in sorted(model.genes.get_by_id('11203').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15180').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('11362').reactions, key=lambda x: x.id):
    if not r.id.startswith('FAO'):
        print(r.id, r.reaction, r.gene_reaction_rule)

HACD1m aacoa_m + h_m + nadh_m <=> 3hbcoa_m + nad_m 11203
HACD2m 3ohcoa_m + h_m + nadh_m <=> 3hhcoa_m + nad_m 11203
HACD3m 3oocoa_m + h_m + nadh_m <=> 3hocoa_m + nad_m 11203
HACD4m 3odcoa_m + h_m + nadh_m <=> 3hdcoa_m + nad_m 11203
HACD5m 3oddcoa_m + h_m + nadh_m <=> 3hddcoa_m + nad_m 11203
HACD6m 3otdcoa_m + h_m + nadh_m <=> 3htdcoa_m + nad_m 11203
HACD7m 3ohdcoa_m + h_m + nadh_m <=> 3hhdcoa_m + nad_m 11203

HADPCOADH3 3hadpcoa_c + nad_c <=> h_c + nadh_c + oxadpcoa_c 15180
HBCO_nadp aacoa_c + h_c + nadph_c <=> 3hbcoa_c + nadp_c 15180

ECOAH11p h2o_x + hxc2coa_x <=> 3hxccoa_x 11362
ECOAH1p 3hbcoa_x <=> b2coa_x + h2o_x 11362
ECOAH2p h2o_x + hx2coa_x <=> 3hhcoa_x 11362
ECOAH3p h2o_x + oc2coa_x <=> 3hocoa_x 11362
ECOAH4p 3hdcoa_x <=> dc2coa_x + h2o_x 11362
ECOAH5p 3hddcoa_x <=> dd2coa_x + h2o_x 11362
ECOAH6p 3htdcoa_x <=> h2o_x + td2coa_x 11362
ECOAH7p 3hhdcoa_x <=> h2o_x + hdd2coa_x 11362
ECOAH8p 3hodcoa_x <=> h2o_x + od2coa_x 11362
HACD10p 3hxccoa_x + nad_x <=> 3ohxccoa_x + h_x + nadh_x 

mitochondrial beta oxidation genes  
ACADSB (branched and C4-C6) 14070  
ACADM (C4-C16) 12570  
ECHS1 14805  
HADH 11203  
ACAA2 8885  
FAOXCm - '12570 and 14805 and 11203 and 8885'  
FAOXCm (short) - '(12570 and 14805 and 11203 and 8885) or (14070 and 14805 and 11203 and 8885)'  

peroxisomal beta oxidation genes  
POX1/ACOX1 12742, 12752, 9700  
ACADS (C4-C6) 12086  
ACAD10 (C4-C12) 10408  
ACADL (C8-C22) 12989  
ACADVL (C14-C24) 16253  
FOX2 11362  
POT1/ACAA1 13813  
FAOXCx (o2) - '(12742 and 11362 and 13813) or (12752 and 11362 and 13813) or (9700 and 11362 and 13813)'

Unsaturated genes  
13228	SPS19 cyto 10, mito 7, cyto_nucl 7, pero 4, nucl 2, cysk 2	K13237: DECR2; peroxisomal 2,4-dienoyl-CoA reductase	SKL*, uses NADPH  
10293	ECI1 mito 8, cyto_nucl 7.5, extr 7, cyto 6, nucl 5	K13239: PECI; peroxisomal 3,2-trans-enoyl-CoA isomerase	HKI*  
11907	DCI1 extr 9, mito 8, cyto 5.5, cyto_nucl 4, pero 3	K12663: ECH1; Delta(3,5)-Delta(2,4)-dienoyl-CoA isomerase	AKL*, trans cis alk-3,5-dienoyl-CoA to trans trans alk-2,4-dienoyl-CoA  
11338	cyto 18.5, cyto_nucl 10, mito 4, cysk 4	K00219: fadH; 2,4-dienoyl-CoA reductase (NADPH2)	KEE*

Unsaturated, even number trans  
regular oxidation  
First route uses one less O2 compared to saturated  
or  
POX1 trans alk-4-enoyl-CoA to trans trans alk-2,4-dienoyl-CoA, uses O2  
SPS19 trans trans alk-2,4-dienoyl-CoA to trans alk-3-enoyl-CoA, uses NADPH  
ECH1 trans alk-3-enoyl-CoA to trans alk-2-enoyl-CoA  
Second route uses one additional NADPH compared to saturated

Unsaturated, even number cis  
POX1 cis alk-4-enoyl-CoA to trans cis alk-2,4-dienoyl-CoA, uses O2  
SPS19 trans cis alk-2,4-dienoyl-CoA to trans alk-3-enoyl-CoA, uses NADPH  
ECH1 trans alk-3-enoyl-CoA to trans alk-2-enoyl-CoA  
uses one additional nadph compared to saturated  

Unsaturated, odd number cis needs either  
ECI1 cis alk-3-enoyl-CoA to trans alk-2-enoyl-CoA  
First route uses one less O2 compared to saturated (FAO161p_odd)  
or  
POX1 cis alk-5-enoyl-CoA to trans cis alk-2,5-dienoyl-CoA, uses O2  
ECI1 trans cis alk-2,5-dienoyl-CoA to trans cis alk-3,5-dienoyl-CoA  
DCI1 trans cis alk-3,5-dienoyl-CoA to trans trans alk-2,4-dienoyl-CoA  
SPS19 trans trans alk-2,4-dienoyl-CoA to trans alk-3-enoyl-CoA, uses NADPH  
ECH1 trans alk-3-dienoyl-CoA to trans alk-2-dienoyl-CoA  
Second route uses one additional nadph compared to saturated (FAO161p_even)  

Unsaturated, odd number trans needs ECH1 (trans alk-3-enoyl-CoA to trans alk-2-enoyl-CoA)  
use one less O2 compared to saturated  

FAO181p_odd (9 cis) - 10293 + FAOXCx genes, one less O2  
FAO181p_even (9 cis) - 10293 and 11907 and 13228 + FAOXCx genes, one additional nadph  
FAO182p_odd (9 cis, 12 cis) - 10293 and 13228 + FAOXCx genes, one less O2 and one additional nadph  
FAO182p_even (9 cis, 12 cis) - 10293 and 11907 and 13228 + FAOXCx genes, two additional nadph  
FAO183p_odd (6 cis, 9 cis, 12 cis) - 10293 and 13228 + FAOXCx genes, one less O2 and two additional nadph  
FAO183p_even (6 cis, 9 cis, 12 cis) - 10293 and 11907 and 13228 + FAOXCx genes, three additional nadph  

Genes are peroxisomal -> remove mito unsaturated, odd number fatty acid degradation

In [38]:
temp = [r.id for r in sorted(model.reactions, key=lambda x: x.id) if r.id.startswith('FAOXC')]
temp_uniq = set(x.split('m')[0] if 'm' in x else x.split('x')[0] for x in temp)
for x in sorted(temp_uniq):
    print(x if x in model.reactions else '\t', end='\t')
    print(x+'m' if x+'m' in model.reactions else '\t', end='\t')
    print(x+'x' if x+'x' in model.reactions else '')

FAOXC11			
FAOXC140			
		FAOXC150m	
FAOXC160			
		FAOXC16080m	FAOXC16080x
		FAOXC16180m	
		FAOXC161802m	
		FAOXC170m	
FAOXC180			FAOXC180x
		FAOXC1811601m	
		FAOXC1811602m	
		FAOXC1811603m	
		FAOXC18280m	
		FAOXC182806m	
		FAOXC183803m	
		FAOXC183806m	FAOXC183806x
		FAOXC18480m	FAOXC18480x
		FAOXC200180m	FAOXC200180x
		FAOXC2031836m	
FAOXC204			
		FAOXC204184m	
		FAOXC2051843m	FAOXC2051843x
		FAOXC2242046m	FAOXC2242046x
		FAOXC2251836m	FAOXC2251836x
		FAOXC2252053m	FAOXC2252053x
FAOXC226			
		FAOXC226205m	FAOXC226205x
				FAOXC240200x
				FAOXC241181x
				FAOXC2442246x
				FAOXC2452253x
				FAOXC2452256x
				FAOXC246226x
				FAOXC260240x
FAOXC80			


In [39]:
# Mitochondrial FAOXC
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FAOXC') and not (r.id.endswith('m') or r.id.endswith('x')):
        print(r.id, r.reaction)
#        print("\'" + r.id + "\'", end=",")

FAOXC11 coa_m + dmnoncoa_m + h2o_m + nad_m --> accoa_m + dmhptcoa_m + h_m + nadh_m
FAOXC140 6.0 coa_m + 6.0 fad_m + 6.0 h2o_m + 6.0 nad_m + tdcoa_m --> 7.0 accoa_m + 6.0 fadh2_m + 6.0 h_m + 6.0 nadh_m
FAOXC160 7.0 coa_m + 7.0 fad_m + 7.0 h2o_m + 7.0 nad_m + pmtcoa_m --> 8.0 accoa_m + 7.0 fadh2_m + 7.0 h_m + 7.0 nadh_m
FAOXC180 coa_m + fad_m + h2o_m + nad_m + stcoa_m --> accoa_m + fadh2_m + h_m + nadh_m + pmtcoa_m
FAOXC204 arachdcoa_m + 9.0 coa_m + 5.0 fad_m + 9.0 h2o_m + 9.0 nad_m --> 10.0 accoa_m + 5.0 fadh2_m + 9.0 h_m + 9.0 nadh_m
FAOXC226 c226coa_m + 10.0 coa_m + 4.0 fad_m + 10.0 h2o_m + 10.0 nad_m --> 11.0 accoa_m + 4.0 fadh2_m + 10.0 h_m + 10.0 nadh_m
FAOXC80 3.0 coa_m + 3.0 fad_m + 3.0 h2o_m + 3.0 nad_m + occoa_m --> 4.0 accoa_m + 3.0 fadh2_m + 3.0 h_m + 3.0 nadh_m


FAOXC11 orphan  
FAOXC80, FAOXC140, FAOXC160 redundant with ACOAD/ECOAH/HACD/ACACT 
FAOXC180, FAOXC204, FAOXC226 long

In [40]:
model.remove_reactions(['FAOXC11','FAOXC80','FAOXC140','FAOXC160','FAOXC180','FAOXC204','FAOXC226'], remove_orphans=True)

In [41]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FAOXC') and r.id.endswith('m'):
        print(r.id, r.reaction, r.gene_reaction_rule)

FAOXC150m 6.0 coa_m + 6.0 fad_m + 6.0 h2o_m + 6.0 nad_m + ptdcacoa_m --> 6.0 accoa_m + 6.0 fadh2_m + 6.0 h_m + 6.0 nadh_m + ppcoa_m 12570
FAOXC16080m 4.0 coa_m + 4.0 fad_m + 4.0 h2o_m + 4.0 nad_m + pmtcoa_m --> 4.0 accoa_m + 4.0 fadh2_m + 4.0 h_m + 4.0 nadh_m + occoa_m 12570
FAOXC161802m 4.0 coa_m + 3.0 fad_m + 4.0 h2o_m + hdd2coa_m + 4.0 nad_m --> 4.0 accoa_m + 3.0 fadh2_m + 4.0 h_m + 4.0 nadh_m + occoa_m 12570
FAOXC16180m 4.0 coa_m + 3.0 fad_m + 4.0 h2o_m + hdcoa_m + 4.0 nad_m --> 4.0 accoa_m + 3.0 fadh2_m + 4.0 h_m + 4.0 nadh_m + occoa_m 12570
FAOXC170m 7.0 coa_m + 7.0 fad_m + 7.0 h2o_m + hpdcacoa_m + 7.0 nad_m --> 7.0 accoa_m + 7.0 fadh2_m + 7.0 h_m + 7.0 nadh_m + ppcoa_m 12570
FAOXC1811601m 5.0 coa_m + 4.0 fad_m + 5.0 h2o_m + 5.0 nad_m + od2coa_m --> 5.0 accoa_m + 4.0 fadh2_m + 5.0 h_m + 5.0 nadh_m + occoa_m 12570
FAOXC1811602m 5.0 coa_m + 4.0 fad_m + 5.0 h2o_m + 5.0 nad_m + vacccoa_m --> 5.0 accoa_m + 4.0 fadh2_m + 5.0 h_m + 5.0 nadh_m + occoa_m 12570
FAOXC1811603m 5.0 coa_m + 4.

FAOXC16080m - redundant with ACOAD/ECOAH/HACD/ACACT  
FAOXC16180m - hdcoa_m is from (Z)-hexadec-9-enoyl-CoA (n-C16:1, cis), odd number unsaturated  
FAOXC161802m - hdd2coa_m is trans-2-hexadecenoyl-CoA, even number unsaturated, but redundant  
FAOXC1811601m - od2coa is Trans-octadec-2-enoyl-CoA, even number unsaturated, long  
FAOXC1811602m - vacccoa is Trans-octadec-11-enoyl-CoA, odd number unsaturated, long  
FAOXC1811603m - odecoa is oleate, (Z)-octadec-9-enoyl-CoA (n-C18:1, cis), odd number unsaturated, long  
FAOXC182806m, FAOXC18280m, FAOXC183803m, FAOXC183806m, FAOXC18480m - long  
FAOXC200180m, FAOXC2031836m, FAOXC204184m, FAOXC2051843m - long  
FAOXC2242046m, FAOXC2251836m, FAOXC2252053m, FAOXC226205m - long  

Remove mito unsaturated odd number oxidation rxns, and long-chain (> C17) rxns  
Add odd number fatty acid mito oxidation from Recon3D and change genes to '12570 and 14805 and 11203 and 8885' 
Change FAOXC5C3x to FAOXC5C3m, and change genes to '(12570 and 14805 and 11203 and 8885) or (14070 and 14805 and 11203 and 8885)'

In [42]:
model.remove_reactions(['FAOXC150m','FAOXC16080m','FAOXC161802m','FAOXC16180m','FAOXC170m','FAOXC1811601m',
                        'FAOXC1811602m','FAOXC1811603m','FAOXC182806m','FAOXC18280m','FAOXC183803m','FAOXC183806m',
                        'FAOXC18480m','FAOXC200180m','FAOXC2031836m','FAOXC204184m','FAOXC2051843m','FAOXC2242046m',
                        'FAOXC2251836m','FAOXC2252053m','FAOXC226205m'], remove_orphans=True)
for x in ['FAOXC5C3x','FAOXC7C5m','FAOXC9C7m','FAOXC11C9m','FAOXC13C11m','FAOXC15C13m','FAOXC170150m']:
    model.add_reactions([hsa2.reactions.get_by_id(x).copy()])
    model.reactions.get_by_id(x).gene_reaction_rule = '12570 and 14805 and 11203 and 8885'
model.reactions.get_by_id('FAOXC5C3x').id = 'FAOXC5C3m'
model.reactions.get_by_id('FAOXC5C3m').gene_reaction_rule = '(12570 and 14805 and 11203 and 8885) or (14070 and 14805 and 11203 and 8885)'

In [43]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FAOXC') and 'm' in r.compartments:
        print(r.id, r.reaction, r.gene_reaction_rule)

FAOXC11C9m coa_m + fad_m + h2o_m + nad_m + undcoa_m --> accoa_m + fadh2_m + h_m + nadh_m + noncoa_m 12570 and 14805 and 11203 and 8885
FAOXC13C11m coa_m + fad_m + h2o_m + nad_m + tridcoa_m --> accoa_m + fadh2_m + h_m + nadh_m + undcoa_m 12570 and 14805 and 11203 and 8885
FAOXC15C13m coa_m + fad_m + h2o_m + nad_m + ptdcacoa_m --> accoa_m + fadh2_m + h_m + nadh_m + tridcoa_m 12570 and 14805 and 11203 and 8885
FAOXC170150m coa_m + fad_m + h2o_m + hpdcacoa_m + nad_m --> accoa_m + fadh2_m + h_m + nadh_m + ptdcacoa_m 12570 and 14805 and 11203 and 8885
FAOXC5C3m coa_m + fad_m + h2o_m + nad_m + pentcoa_m --> accoa_m + fadh2_m + h_m + nadh_m + ppcoa_m (12570 and 14805 and 11203 and 8885) or (14070 and 14805 and 11203 and 8885)
FAOXC7C5m coa_m + fad_m + h2o_m + hepcoa_m + nad_m --> accoa_m + fadh2_m + h_m + nadh_m + pentcoa_m 12570 and 14805 and 11203 and 8885
FAOXC9C7m coa_m + fad_m + h2o_m + nad_m + noncoa_m --> accoa_m + fadh2_m + h_m + hepcoa_m + nadh_m 12570 and 14805 and 11203 and 8885


In [44]:
# Now peroxisomal FAOXC
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FAO') and not r.id.startswith('FAOXC'):
        print(r.id, r.reaction, r.gene_reaction_rule)
#        print("\'" + r.id + "\'", end=",")

FAO141p_even 6.0 coa_x + 6.0 h2o_x + 6.0 nad_x + nadph_x + 6.0 o2_x + tdecoa_x --> 7.0 accoa_x + 6.0 h2o2_x + 5.0 h_x + 6.0 nadh_x + nadp_x (10293 and 11362 and 12742 and 13228 and 13813) or (10293 and 11362 and 12742 and 13228 and 9065) or (10293 and 11362 and 12752 and 13228 and 13813) or (10293 and 11362 and 12752 and 13228 and 9065) or (10293 and 11362 and 13228 and 13813 and 9700) or (10293 and 11362 and 13228 and 9065 and 9700)
FAO141p_odd 6.0 coa_x + 6.0 h2o_x + 6.0 nad_x + 5.0 o2_x + tdecoa_x --> 7.0 accoa_x + 5.0 h2o2_x + 6.0 h_x + 6.0 nadh_x (10293 and 11362 and 12742 and 13813) or (10293 and 11362 and 12742 and 9065) or (10293 and 11362 and 12752 and 13813) or (10293 and 11362 and 12752 and 9065) or (10293 and 11362 and 13813 and 9700) or (10293 and 11362 and 9065 and 9700)
FAO161p_even 7.0 coa_x + 7.0 h2o_x + hdcoa_x + 7.0 nad_x + nadph_x + 7.0 o2_x --> 8.0 accoa_x + 7.0 h2o2_x + 6.0 h_x + 7.0 nadh_x + nadp_x (10293 and 11362 and 12742 and 13228 and 13813) or (10293 and 113

14:1, 16:1, 18:1 are 9Z odd unsaturated, can use one less o2 (odd) or one additional nadph (even)  
18:2 is 9Z12Z, can be one less o2 and one additional nadph (oddeven), or two additional nadph (eveneven)  
18:3 is 6Z9Z12Z, can be one less o2 and two additional nadph (evenoddeven), or three additional nadph (eveneveneven)  
change FAO182p_eveneven to FAO182p_even and FAO182p_evenodd to FAO182p_odd, remove FAO182p_oddodd  
add FAO183p_even and FAO183p_odd

In [45]:
model.reactions.get_by_id('FAO141p_even').gene_reaction_rule = '(10293 and 11362 and 11907 and 12742 and 13228 and 13813) or (10293 and 11362 and 11907 and 12752 and 13228 and 13813) or (10293 and 11362 and 11907 and 13228 and 13813 and 9700)'
model.reactions.get_by_id('FAO161p_even').gene_reaction_rule = '(10293 and 11362 and 11907 and 12742 and 13228 and 13813) or (10293 and 11362 and 11907 and 12752 and 13228 and 13813) or (10293 and 11362 and 11907 and 13228 and 13813 and 9700)'
model.reactions.get_by_id('FAO181p_even').gene_reaction_rule = '(10293 and 11362 and 11907 and 12742 and 13228 and 13813) or (10293 and 11362 and 11907 and 12752 and 13228 and 13813) or (10293 and 11362 and 11907 and 13228 and 13813 and 9700)'
model.reactions.get_by_id('FAO182p_eveneven').id = 'FAO182p_even'
model.reactions.get_by_id('FAO182p_even').gene_reaction_rule = '(10293 and 11362 and 11907 and 12742 and 13228 and 13813) or (10293 and 11362 and 11907 and 12752 and 13228 and 13813) or (10293 and 11362 and 11907 and 13228 and 13813 and 9700)'

model.reactions.get_by_id('FAO141p_odd').gene_reaction_rule = '(10293 and 11362 and 12742 and 13813) or (10293 and 11362 and 12752 and 13813) or (10293 and 11362 and 13813 and 9700)'
model.reactions.get_by_id('FAO161p_odd').gene_reaction_rule = '(10293 and 11362 and 12742 and 13813) or (10293 and 11362 and 12752 and 13813) or (10293 and 11362 and 13813 and 9700)'
model.reactions.get_by_id('FAO181p_odd').gene_reaction_rule = '(10293 and 11362 and 12742 and 13813) or (10293 and 11362 and 12752 and 13813) or (10293 and 11362 and 13813 and 9700)'
model.reactions.get_by_id('FAO182p_evenodd').id = 'FAO182p_odd'
model.reactions.get_by_id('FAO182p_odd').gene_reaction_rule = '(10293 and 11362 and 12742 and 13228 and 13813) or (10293 and 11362 and 12752 and 13228 and 13813) or (10293 and 11362 and 13228 and 13813 and 9700)'

model.remove_reactions(['FAO182p_oddodd'], remove_orphans=True)
r1 = model.reactions.get_by_id('FAO182p_even').copy()
r1.id = 'FAO183p_even'
r2 = model.reactions.get_by_id('FAO182p_odd').copy()
r2.id = 'FAO183p_odd'
model.add_reactions([r1,r2])
r1.add_metabolites({'ocdycacoa_x': 1.0, 'lnlncgcoa_x': -1.0, 'nadph_x': -1.0, 'nadp_x': 1.0, 'h_x': -1.0})
r2.add_metabolites({'ocdycacoa_x': 1.0, 'lnlncgcoa_x': -1.0, 'nadph_x': -1.0, 'nadp_x': 1.0, 'h_x': -1.0})

In [46]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FAOXC') and r.id.endswith('x'):
        print(r.id, r.reaction, r.gene_reaction_rule)

FAOXC16080x 4.0 coa_x + 4.0 h2o_x + 4.0 nad_x + 4.0 o2_x + pmtcoa_x --> 4.0 accoa_x + 4.0 h2o2_x + 4.0 h_x + 4.0 nadh_x + occoa_x (EHHADH and 12742 and 13813) or (EHHADH and 12742 and 9065) or (EHHADH and 12752 and 13813) or (EHHADH and 12752 and 9065) or (EHHADH and 13813 and 9700) or (EHHADH and 9065 and 9700) or (Ehhadh and 12742 and 13813) or (Ehhadh and 12742 and 9065) or (Ehhadh and 12752 and 13813) or (Ehhadh and 12752 and 9065) or (Ehhadh and 13813 and 9700) or (Ehhadh and 9065 and 9700) or (11362 and 12742 and 13813) or (11362 and 12742 and 9065) or (11362 and 12752 and 13813) or (11362 and 12752 and 9065) or (11362 and 13813 and 9700) or (11362 and 9065 and 9700)
FAOXC180x coa_x + h2o_x + nad_x + o2_x + stcoa_x --> accoa_x + h2o2_x + h_x + nadh_x + pmtcoa_x (EHHADH and 12742 and 13813) or (EHHADH and 12742 and 9065) or (EHHADH and 12752 and 13813) or (EHHADH and 12752 and 9065) or (EHHADH and 13813 and 9700) or (EHHADH and 9065 and 9700) or (Ehhadh and 12742 and 13813) or (Eh

FAOXC16080x, FAOXC180x, FAO240p, FAOXC260240x are redundant  
ttccoa and lgnccoa / ttc and lgnc C24:0 are the same -> replace it with ttccoa / ttc

In [47]:
model.remove_reactions(['FAOXC16080x','FAOXC180x','FAO240p','FAOXC260240x'], remove_orphans=True)

In [48]:
for m in sorted(model.metabolites, key=lambda x: x.id):
    if m.id.startswith('ttc'):
        print(m.id)
for m in sorted(model.metabolites, key=lambda x: x.id):
    if m.id.startswith('lgnc'):
        print(m.id)

ttc_c
ttc_ggdp_c
ttc_m
ttc_x
ttccoa_r
ttccoa_x
lgnc_c
lgnc_e
lgnccoa_c
lgnccoa_m
lgnccoa_x
lgnccrn_c
lgnccrn_m


In [49]:
for r in sorted(model.metabolites.get_by_id('ttc_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('ttc_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('ttc_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

FA240tp ttc_c --> ttc_x 9912
FAS240_L 9.0 h_c + 3.0 malcoa_c + 6.0 nadph_c + ocdca_c --> 3.0 co2_c + 3.0 coa_c + 3.0 h2o_c + 6.0 nadp_c + ttc_c (16241 and 16655) or (10677 and 16241 and 16695)
FAS260 3.0 h_c + malcoa_c + 2.0 nadph_c + ttc_c --> co2_c + coa_c + h2o_c + hexc_c + 2.0 nadp_c 16241 and 16655
yli_R0137 3.0 h_c + malcoa_c + 2.0 nadph_c + ttc_c --> co2_c + coa_c + h2o_c + 2.0 nadp_c + yli_M05672_c 10677 and 16241 and 16695
yli_R0150 atp_c + coa_c + ttc_c <=> amp_c + ppi_c + yli_M04095_c 11167 or 12538 or 12555 or 15748

yli_R0228 h2o_m + yli_M04095_m --> coa_m + h_m + ttc_m 16048
yli_R0242 atp_m + coa_m + ttc_m <=> amp_m + ppi_m + yli_M04095_m 11167 or 12538 or 12555 or 15748

FA240tp ttc_c --> ttc_x 9912
FACOAL240p atp_x + coa_x + ttc_x <=> amp_x + ppi_x + ttccoa_x 9912
PTE12x h2o_x + ttccoa_x --> coa_x + h_x + ttc_x 16048


In [50]:
for r in sorted(model.metabolites.get_by_id('lgnc_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('lgnc_e').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

FACOAL240 atp_c + coa_c + lgnc_c <=> amp_c + lgnccoa_c + ppi_c 12538 or 12555
FATP8t lgnc_c + na1_c <=> lgnc_e + na1_e 9912

FATP8t lgnc_c + na1_c <=> lgnc_e + na1_e 9912


yli_R0137 and FAS260 are the same  
Remove yli_R0137 and change FAS260 genes to '(16241 and 16655) or (10677 and 16241 and 16695)'  

yli_R0150 and FACOAL240 are the same (cyto)  
Remove yli_R0150 and change FACOAL240 genes to '11167 or 12538 or 12555 or 15748'

Replace lgnc with ttc

In [51]:
model.remove_reactions(['yli_R0137','yli_R0150'], remove_orphans=True)
model.reactions.get_by_id('FAS260').gene_reaction_rule = '(16241 and 16655) or (10677 and 16241 and 16695)'
for r in model.metabolites.get_by_id('lgnc_c').reactions:
    r.add_metabolites({'lgnc_c': -r.get_coefficient('lgnc_c'), 'ttc_c': r.get_coefficient('lgnc_c')})
m = model.metabolites.get_by_id('ttc_c').copy()
m.id = 'ttc_e'
m.compartment = 'e'
model.add_metabolites([m])
for r in model.metabolites.get_by_id('lgnc_e').reactions:
    r.add_metabolites({'lgnc_e': -r.get_coefficient('lgnc_e'), 'ttc_e': r.get_coefficient('lgnc_e')})    

In [52]:
for r in sorted(model.metabolites.get_by_id('ttccoa_r').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('ttccoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

CERS124er sphgn_r + ttccoa_r --> cer1_24_r + coa_r + h_r 11391 or 15168
CERS224er psphings_r + ttccoa_r --> cer2_24_r + coa_r + h_r 11391 or 15168

ACACT9p 3ohxccoa_x + coa_x --> accoa_x + ttccoa_x 13813
FACOAL240p atp_x + coa_x + ttc_x <=> amp_x + ppi_x + ttccoa_x 9912
PTE12x h2o_x + ttccoa_x --> coa_x + h_x + ttc_x 16048


In [53]:
for r in sorted(model.metabolites.get_by_id('lgnccoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
for r in sorted(model.metabolites.get_by_id('lgnccoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
for r in sorted(model.metabolites.get_by_id('lgnccoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('lgnccrn_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
for r in sorted(model.metabolites.get_by_id('lgnccrn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

FACOAL240 atp_c + coa_c + ttc_c <=> amp_c + lgnccoa_c + ppi_c 12538 or 12555
LGNCCPT1 crn_c + lgnccoa_c --> coa_c + lgnccrn_c 9315
LGNCCPT2 coa_m + lgnccrn_m --> crn_m + lgnccoa_m 13580
FAOXC240200x 2.0 coa_x + 2.0 h2o_x + lgnccoa_x + 2.0 nad_x + 2.0 o2_x --> 2.0 accoa_x + arachcoa_x + 2.0 h2o2_x + 2.0 h_x + 2.0 nadh_x (EHHADH and 12742 and 13813) or (EHHADH and 12742 and 9065) or (EHHADH and 12752 and 13813) or (EHHADH and 12752 and 9065) or (EHHADH and 13813 and 9700) or (EHHADH and 9065 and 9700) or (Ehhadh and 12742 and 13813) or (Ehhadh and 12742 and 9065) or (Ehhadh and 12752 and 13813) or (Ehhadh and 12752 and 9065) or (Ehhadh and 13813 and 9700) or (Ehhadh and 9065 and 9700) or (11362 and 12742 and 13813) or (11362 and 12742 and 9065) or (11362 and 12752 and 13813) or (11362 and 12752 and 9065) or (11362 and 13813 and 9700) or (11362 and 9065 and 9700)

LGNCCPT1 crn_c + lgnccoa_c --> coa_c + lgnccrn_c 9315
LGNCCRNt lgnccrn_c --> lgnccrn_m 9331
LGNCCPT2 coa_m + lgnccrn_m --> crn

ttccoa not in mitochondria in Recon3D (C24), remove crn transport reactions  
replace lgnccoa with ttccoa

In [54]:
model.remove_reactions(['LGNCCPT1','LGNCCPT2','LGNCCRNt'], remove_orphans=True)
m = model.metabolites.get_by_id('ttccoa_x').copy()
m.id = 'ttccoa_c'
m.compartment = 'c'
model.add_metabolites([m])
for x in ['lgnccoa_c','lgnccoa_x']:
    for r in model.metabolites.get_by_id(x).reactions:
        r.add_metabolites({x: -r.get_coefficient(x), x.replace('lgnc','ttc'): r.get_coefficient(x)})

In [55]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FAOXC') and 'x' in r.compartments:
        print(r.id, r.reaction, r.gene_reaction_rule, sep='\n')
        print()

FAOXC183806x
5.0 coa_x + 5.0 h2o_x + lnlncgcoa_x + 5.0 nad_x + 2.0 o2_x --> 5.0 accoa_x + 2.0 h2o2_x + 5.0 h_x + 5.0 nadh_x + occoa_x
(EHHADH and 12742 and 13813) or (EHHADH and 12742 and 9065) or (EHHADH and 12752 and 13813) or (EHHADH and 12752 and 9065) or (EHHADH and 13813 and 9700) or (EHHADH and 9065 and 9700) or (Ehhadh and 12742 and 13813) or (Ehhadh and 12742 and 9065) or (Ehhadh and 12752 and 13813) or (Ehhadh and 12752 and 9065) or (Ehhadh and 13813 and 9700) or (Ehhadh and 9065 and 9700) or (11362 and 12742 and 13813) or (11362 and 12742 and 9065) or (11362 and 12752 and 13813) or (11362 and 12752 and 9065) or (11362 and 13813 and 9700) or (11362 and 9065 and 9700)

FAOXC18480x
5.0 coa_x + 5.0 h2o_x + 5.0 nad_x + o2_x + strdnccoa_x --> 5.0 accoa_x + h2o2_x + 5.0 h_x + 5.0 nadh_x + occoa_x
(EHHADH and 12742 and 13813) or (EHHADH and 12742 and 9065) or (EHHADH and 12752 and 13813) or (EHHADH and 12752 and 9065) or (EHHADH and 13813 and 9700) or (EHHADH and 9065 and 9700) or (

These unsaturaed reactions are currently all blocked, need to add transport reactions (no gene in hsa)  
FAOXC2251836x (4Z,7Z,10Z,13Z,16Z)-docosapentaenoyl-CoA to (6Z,9Z,12Z)-octadecatrienoyl-CoA, wrong need o2  
FAOXC183806x Gamma-Linolenoyl-CoA, (6Z,9Z,12Z)-octadecatrienoyl-CoA to occoa, wrong need nadph  
FAOXC2252053x (4Z,7Z,10Z,13Z,16Z)-docosapentaenoyl-CoA to (5Z,8Z,11Z,14Z,17Z)-icosapentaenoyl-CoA, wrong need o2  
FAOXC2051843x (5Z,8Z,11Z,14Z,17Z)-icosapentaenoyl-CoA to (6Z,9Z,12Z,15Z)-octadecatetraenoyl-CoA, wrong need o2, nadph  
FAOXC18480x (6Z,9Z,12Z,15Z)-octadecatetraenoyl-CoA to occoa, wrong cannot make occoa  
FAOXC2442246x (9Z,12Z,15Z,18Z)-tetracosatetraenoyl-CoA to (7Z,10Z,13Z,16Z)-docosatetraenoyl CoA, OK normal  
FAOXC2242046x (7Z,10Z,13Z,16Z)-docosatetraenoyl-CoA to (5Z,8Z,11Z,14Z)-eicosatetraenoyl-CoA, OK normal  
FAOXC246226x (6Z,9Z,12Z,15Z,18Z,21Z)-tetracosahexaenoyl-CoA to (4Z,7Z,10Z,13Z,16Z,19Z)-docosahexaenoyl CoA, OK normal  
FAOXC226205x (4Z,7Z,10Z,13Z,16Z,19Z)-docosahexaenoyl-CoA to (5Z,8Z,11Z,14Z,17Z)-icosapentaenoyl-CoA, wrong need o2, nadph  
FAOXC241181x (15Z)-tetracosenoyl-CoA to (9Z)-octadecenoyl-CoA, OK normal  
FAOXC2452253x (9Z,12Z,15Z,18Z,21Z)-tetracosapentaenoyl-CoA to (7Z,10Z,13Z,16Z,19Z)-Docosapentaenoyl-CoA, OK normal  
FAOXC2452256x (6Z,9Z,12Z,15Z,18Z)-tetracosapentaenoyl-CoA to (4Z,7Z,10Z,13Z,16Z)-docosapentaenoyl-CoA, OK normal  

In [56]:
model.remove_reactions(['FAOXC2251836x','FAOXC183806x','FAOXC2252053x','FAOXC2051843x','FAOXC18480x','FAOXC226205x'], remove_orphans=True)

In [57]:
# peroxisomal beta oxidation lumped reactions
model.reactions.get_by_id('FAOXC200180x').gene_reaction_rule = '(12742 and 11362 and 13813) or (12752 and 11362 and 13813) or (9700 and 11362 and 13813)'
model.reactions.get_by_id('FAOXC2242046x').gene_reaction_rule = '(12742 and 11362 and 13813) or (12752 and 11362 and 13813) or (9700 and 11362 and 13813)'
model.reactions.get_by_id('FAOXC240200x').gene_reaction_rule = '(12742 and 11362 and 13813) or (12752 and 11362 and 13813) or (9700 and 11362 and 13813)'
model.reactions.get_by_id('FAOXC241181x').gene_reaction_rule = '(12742 and 11362 and 13813) or (12752 and 11362 and 13813) or (9700 and 11362 and 13813)'
model.reactions.get_by_id('FAOXC2442246x').gene_reaction_rule = '(12742 and 11362 and 13813) or (12752 and 11362 and 13813) or (9700 and 11362 and 13813)'
model.reactions.get_by_id('FAOXC2452253x').gene_reaction_rule = '(12742 and 11362 and 13813) or (12752 and 11362 and 13813) or (9700 and 11362 and 13813)'
model.reactions.get_by_id('FAOXC2452256x').gene_reaction_rule = '(12742 and 11362 and 13813) or (12752 and 11362 and 13813) or (9700 and 11362 and 13813)'
model.reactions.get_by_id('FAOXC246226x').gene_reaction_rule = '(12742 and 11362 and 13813) or (12752 and 11362 and 13813) or (9700 and 11362 and 13813)'

In [58]:
temp = ['16048','8456','14585']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
16048,"K11992: ACOT8, PTE; acyl-CoA thioesterase 8",,TES1,ACOT8,,ACOT8,Not Essential,"nucl 8.5, cyto_nucl 7, pero 7, mito 5, cyto 4.5",AAL*
8456,K17361: ACOT9; acyl-coenzyme A thioesterase 9,,,ACOT9,,ACOT9,Not Essential,"nucl 13.5, cyto_nucl 11.5, cyto 8.5, mito 3",QAP*
14585,K17362: ACOT13; acyl-coenzyme A thioesterase 13,,,ACOT13,,ACOT1,Not Essential,"cysk 9, mito 8, cyto 5.5, cyto_nucl 5, nucl 3.5",PSE*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
16048,-0.0941699,-0.0375465,0.0121643,-0.147875,-0.0126005,-0.0406894,0.172548,0.0362263,-0.245912,-0.040811,0.228448
8456,-0.0934737,-0.207925,-0.306311,-0.212615,-0.296622,-0.0315301,0.0612052,0.0553147,-0.326957,-0.394361,-0.0124277
14585,-0.518157,0.276727,-0.224457,0.0451634,-0.339184,0.298084,-0.011245,-0.0680009,-0.110504,-0.149529,0.172852


In [59]:
# 16048 ACOT8, PTE; acyl-CoA thioesterase 8, ends with AAL* -> pero
# 8456 ACOT9; acyl-coenzyme A thioesterase 9 -> cyto
# 14585 ACOT13; acyl-coenzyme A thioesterase 13, no rxns -> mito
for r in sorted(model.genes.get_by_id('16048').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8456').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

FACOAE100 dcacoa_c + h2o_c --> coa_c + dca_c + h_c 16048
FACOAE120 ddcacoa_c + h2o_c --> coa_c + ddca_c + h_c 16048
FACOAE140 h2o_c + tdcoa_c --> coa_c + h_c + ttdca_c 16048
FACOAE141 h2o_c + tdecoa_c --> coa_c + h_c + ttdcea_c 16048
FACOAE160 h2o_c + pmtcoa_c --> coa_c + h_c + hdca_c 16048 or 8456
FACOAE161 h2o_c + hdcoa_c --> coa_c + h_c + hdcea_c 16048
FACOAE180 h2o_c + stcoa_c --> coa_c + h_c + ocdca_c 16048 or 8456
FACOAE181 h2o_c + odecoa_c --> coa_c + h_c + ocdcea_c 16048
FACOAE60 h2o_c + hxcoa_c --> coa_c + h_c + hxa_c 16048
FACOAE80 h2o_c + occoa_c --> coa_c + h_c + octa_c 16048
PTE10x dcacoa_x + h2o_x --> coa_x + dca_x + h_x 16048
PTE11x ddcacoa_x + h2o_x --> coa_x + ddca_x + h_x 16048
PTE12x h2o_x + ttccoa_x --> coa_x + h_x + ttc_x 16048
PTE2x h2o_x + pmtcoa_x --> coa_x + h_x + hdca_x 16048
PTE7x h2o_x + tdcoa_x --> coa_x + h_x + ttdca_x 16048
PTE8x h2o_x + stcoa_x --> coa_x + h_x + ocdca_x 16048
PTE9x h2o_x + occoa_x --> coa_x + h_x + octa_x 16048
RE0344M h2o_m + stcoa_m --

In [60]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FACOAE'):
        print(r.id, r.reaction, r.gene_reaction_rule)

FACOAE100 dcacoa_c + h2o_c --> coa_c + dca_c + h_c 16048
FACOAE120 ddcacoa_c + h2o_c --> coa_c + ddca_c + h_c 16048
FACOAE140 h2o_c + tdcoa_c --> coa_c + h_c + ttdca_c 16048
FACOAE141 h2o_c + tdecoa_c --> coa_c + h_c + ttdcea_c 16048
FACOAE160 h2o_c + pmtcoa_c --> coa_c + h_c + hdca_c 16048 or 8456
FACOAE161 h2o_c + hdcoa_c --> coa_c + h_c + hdcea_c 16048
FACOAE180 h2o_c + stcoa_c --> coa_c + h_c + ocdca_c 16048 or 8456
FACOAE181 h2o_c + odecoa_c --> coa_c + h_c + ocdcea_c 16048
FACOAE1819Z h2o_c + ocdce9coa_c --> coa_c + h_c + ocdce9a_c 8456
FACOAE1829Z12Z h2o_c + lnlccoa_c --> coa_c + h_c + lnlc_c 8456
FACOAE1836Z9Z12Z h2o_c + lnlncgcoa_c --> coa_c + h_c + lnlncg_c 8456
FACOAE1839Z12Z15Z h2o_c + lnlncacoa_c --> coa_c + h_c + lnlnca_c 8456
FACOAE60 h2o_c + hxcoa_c --> coa_c + h_c + hxa_c 16048
FACOAE80 h2o_c + occoa_c --> coa_c + h_c + octa_c 16048


In [61]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FACOAE'):
        r.gene_reaction_rule = '8456'

In [62]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('PTE') and r.id.endswith('x'):
        print(r.id, r.reaction, r.gene_reaction_rule)

PTE10x dcacoa_x + h2o_x --> coa_x + dca_x + h_x 16048
PTE11x ddcacoa_x + h2o_x --> coa_x + ddca_x + h_x 16048
PTE12x h2o_x + ttccoa_x --> coa_x + h_x + ttc_x 16048
PTE2x h2o_x + pmtcoa_x --> coa_x + h_x + hdca_x 16048
PTE7x h2o_x + tdcoa_x --> coa_x + h_x + ttdca_x 16048
PTE8x h2o_x + stcoa_x --> coa_x + h_x + ocdca_x 16048
PTE9x h2o_x + occoa_x --> coa_x + h_x + octa_x 16048


In [63]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if 'h2o_m' in [m.id for m in r.reactants] and 'coa_m' in [m.id for m in r.products] and len(r.reactants) == 2:
        print(r.id, r.reaction, r.gene_reaction_rule)

3HBCOAHLm 3hibutcoa_m + h2o_m --> 3hmp_m + coa_m + h_m 15218
ACOAHim accoa_m + h2o_m --> ac_m + coa_m + h_m 13797
RE0344M h2o_m + stcoa_m --> coa_m + h_m + ocdca_m 16048
yli_R0226 dccoa_m + h2o_m --> coa_m + h_m + yli_M01250_m 16048
yli_R0227 ddcacoa_m + h2o_m --> coa_m + h_m + yli_M01723_m 16048
yli_R0228 h2o_m + yli_M04095_m --> coa_m + h_m + ttc_m 16048
yli_R0229 h2o_m + pmtcoa_m --> coa_m + h_m + hdca_m 16048
yli_R0230 h2o_m + tdcoa_m --> coa_m + h_m + yli_M03073_m 16048
yli_R0232 h2o_m + occoa_m --> coa_m + h_m + octa_m 16048


In [64]:
# 15218 K05605: HIBCH; 3-hydroxyisobutyryl-CoA hydrolase, mito
model.reactions.get_by_id('RE0344M').id = 'PTE8m' # C18
model.reactions.get_by_id('yli_R0226').id = 'PTE10m' # C10
model.reactions.get_by_id('yli_R0227').id = 'PTE11m' # C12
model.reactions.get_by_id('yli_R0228').id = 'PTE12m' # C24
model.reactions.get_by_id('yli_R0229').id = 'PTE2m' # C16
model.reactions.get_by_id('yli_R0230').id = 'PTE7m' # C14
model.reactions.get_by_id('yli_R0232').id = 'PTE9m' # C8

In [65]:
# C18 and C24 mito reactions -> long chain, remove
for r in model.metabolites.get_by_id('stcoa_m').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('stcrn_m').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('stcrn_c').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('ocdca_m').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('yli_M04095_m').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('ttc_m').reactions:
    print(r.id, r.reaction)

PTE8m h2o_m + stcoa_m --> coa_m + h_m + ocdca_m
yli_R0196 o2_m + stcoa_m --> h2o2_m + od2coa_m
yli_R0239 atp_m + coa_m + ocdca_m <=> amp_m + ppi_m + stcoa_m
C180CPT2 coa_m + stcrn_m --> crn_m + stcoa_m

C180CRNt stcrn_c --> stcrn_m
C180CPT2 coa_m + stcrn_m --> crn_m + stcoa_m

C180CRNt stcrn_c --> stcrn_m
C180CPT1 crn_c + stcoa_c --> coa_c + stcrn_c

yli_R0239 atp_m + coa_m + ocdca_m <=> amp_m + ppi_m + stcoa_m
yli_R1473 h2o_m + 0.005 pchol_cho_m --> 0.5 g3pc_m + h_m + 0.27 hdca_m + 0.05 ocdca_m + 0.02 yli_M01250_m + 0.06 yli_M01723_m + 0.1 yli_M03073_m + 0.17 yli_M03299_m + 0.24 yli_M04622_m + 0.09 yli_M05526_m
PTE8m h2o_m + stcoa_m --> coa_m + h_m + ocdca_m

PTE12m h2o_m + yli_M04095_m --> coa_m + h_m + ttc_m
yli_R0242 atp_m + coa_m + ttc_m <=> amp_m + ppi_m + yli_M04095_m

PTE12m h2o_m + yli_M04095_m --> coa_m + h_m + ttc_m
yli_R0242 atp_m + coa_m + ttc_m <=> amp_m + ppi_m + yli_M04095_m


In [66]:
model.remove_reactions(['PTE8m','C180CPT2','yli_R0239','yli_R0196','C180CRNt','C180CPT1','yli_R1473',
                        'yli_R0242','PTE12m'], remove_orphans=True)

In [67]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('PTE') and r.id.endswith('m'):
        print(r.id, r.reaction, r.gene_reaction_rule)

PTE10m dccoa_m + h2o_m --> coa_m + h_m + yli_M01250_m 16048
PTE11m ddcacoa_m + h2o_m --> coa_m + h_m + yli_M01723_m 16048
PTE2m h2o_m + pmtcoa_m --> coa_m + h_m + hdca_m 16048
PTE7m h2o_m + tdcoa_m --> coa_m + h_m + yli_M03073_m 16048
PTE9m h2o_m + occoa_m --> coa_m + h_m + octa_m 16048


In [68]:
for k, v in {'yli_M01250': 'dca', 'yli_M01723': 'ddca', 'yli_M03073': 'ttdca'}.items():
    model.metabolites.get_by_id(k+'_m').id = v+'_m'

In [69]:
for k, v in {'yli_M01250': 'dca', 'yli_M01723': 'ddca', 'yli_M03073': 'ttdca'}.items():
    for m in model.metabolites:
        if k in m.id:
            print(m.id)
            for r in sorted(m.reactions, key=lambda x: x.id):
                print(r.id, r.reaction)
            temp = v+'_'+m.id.rsplit('_',1)[1]
            print(temp)
            if temp in model.metabolites:
                for r in sorted(model.metabolites.get_by_id(temp).reactions, key=lambda x: x.id):
                    print(r.id, r.reaction)
            print()

yli_M01250_c
yli_R0127 3.0 h_c + malcoa_c + 2.0 nadph_c + octa_c --> co2_c + coa_c + h2o_c + 2.0 nadp_c + yli_M01250_c
yli_R0128 3.0 h_c + malcoa_c + 2.0 nadph_c + yli_M01250_c --> co2_c + coa_c + h2o_c + 2.0 nadp_c + yli_M01723_c
yli_R0141 atp_c + coa_c + yli_M01250_c <=> amp_c + dccoa_c + ppi_c
yli_R0312 h2o_c + 0.01 tag_cho_c --> 0.01 dag_hs_c + h_c + 0.27 hdca_c + 0.05 ocdca_c + 0.02 yli_M01250_c + 0.06 yli_M01723_c + 0.1 yli_M03073_c + 0.17 yli_M03299_c + 0.24 yli_M04622_c + 0.09 yli_M05526_c
yli_R0895 h2o_c + 0.005 pchol_cho_c --> 0.5 g3pc_c + h_c + 0.27 hdca_c + 0.05 ocdca_c + 0.02 yli_M01250_c + 0.06 yli_M01723_c + 0.1 yli_M03073_c + 0.17 yli_M03299_c + 0.24 yli_M04622_c + 0.09 yli_M05526_c
dca_c
AACPS8 ACP_c + atp_c + dca_c --> amp_c + dcaACP_c + ppi_c
FACOAE100 dcacoa_c + h2o_c --> coa_c + dca_c + h_c
FAS100 3.0 h_c + malcoa_c + 2.0 nadph_c + octa_c --> co2_c + coa_c + dca_c + h2o_c + 2.0 nadp_c
FAS120 dca_c + 3.0 h_c + malcoa_c + 2.0 nadph_c --> co2_c + coa_c + ddca_c + h2o_

In [70]:
# Remove redundant yli reactions and irrelevant reactions
model.remove_reactions(['yli_R0127','yli_R0128','yli_R0312','yli_R0895','yli_R1396','yli_R0130',
                        'yli_R0138','yli_R0173','yli_R0132','yli_R0139','yli_R0143','yli_R0174'], remove_orphans=True)
# FACOAL100 and FACOAL120
model.reactions.get_by_id('yli_R0141').id = 'FACOAL100'
model.reactions.get_by_id('FACOAL100').add_metabolites({'yli_M01250_c': 1.0, 'dca_c': -1.0,
                                                        'dccoa_c': -1.0, 'dcacoa_c': 1.0})
model.reactions.get_by_id('yli_R0142').id = 'FACOAL120'
model.reactions.get_by_id('FACOAL120').add_metabolites({'yli_M01723_c': 1.0, 'ddca_c': -1.0})

In [71]:
for r in model.metabolites.get_by_id('dccoa_c').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('dccoa_m').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('dccoa_r').reactions:
    print(r.id, r.reaction)

yli_R0335 0.02 dccoa_c + 0.06 ddcacoa_c + dhap_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c + 0.17 yli_M04594_c + 0.09 yli_M04625_c + 0.24 yli_M04626_c --> 0.01 adhap_hs_c + coa_c
yli_R0307 0.02 dccoa_c + 0.06 ddcacoa_c + glyc3p_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c + 0.17 yli_M04594_c + 0.09 yli_M04625_c + 0.24 yli_M04626_c --> 0.01 1ag3p_SC_c + coa_c
yli_R0129 dccoa_c + 3.0 h_c + malcoa_c + 2.0 nadph_c --> co2_c + coa_c + ddcacoa_c + h2o_c + 2.0 nadp_c
yli_R0308 0.01 1ag3p_SC_c + 0.02 dccoa_c + 0.06 ddcacoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c + 0.17 yli_M04594_c + 0.09 yli_M04625_c + 0.24 yli_M04626_c --> coa_c + 0.01 pa_EC_c
yli_R0126 3.0 h_c + malcoa_c + 2.0 nadph_c + occoa_c --> co2_c + coa_c + dccoa_c + h2o_c + 2.0 nadp_c
yli_R0319 0.02 dccoa_c + 0.06 ddcacoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c + 0.01 yli_M02277_c + 0.17 yli_M04594_c + 0.09 yli_M04625_c + 0.24 yli_M04626_c --> coa_c + 0.01 pchol_cho_c

yli_R0233 atp_m + coa_m + dca_m <=> amp_m + d

In [72]:
for r in model.metabolites.get_by_id('dcacoa_c').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('dcacoa_m').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('dcacoa_x').reactions:
    print(r.id, r.reaction)

GAT2_SC 0.02 dcacoa_c + 0.06 ddcacoa_c + dhap_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c --> 0.01 1agly3p_SC_c + coa_c
LPCAT_SC 0.01 1agpc_SC_c + 0.02 dcacoa_c + 0.06 ddcacoa_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c --> coa_c + 0.01 pc_SC_c
FACOAE100 dcacoa_c + h2o_c --> coa_c + dca_c + h_c
FAS120COA dcacoa_c + 3.0 h_c + malcoa_c + 2.0 nadph_c --> co2_c + coa_c + ddcacoa_c + h2o_c + 2.0 nadp_c
AGAT_SC 0.01 1ag3p_SC_c + 0.02 dcacoa_c + 0.06 ddcacoa_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c --> coa_c + 0.01 pa_SC_c
FAS100COA 3.0 h_c + malcoa_c + 2.0 nadph_c + occoa_c --> co2_c + coa_c + dcacoa_c + h2o_c + 2.0 nadp_c
FACOAL100 atp_c + coa_c + dca_c <=> amp_c + dcacoa_c + ppi_c
TRIGS_SC 0.01 12dgr_SC_c + 0.02 dcacoa_c + 0.06 ddcacoa_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c

In [73]:
# replace dccoa with dcacoa
model.reactions.get_by_id('PTE10m').add_metabolites({'dccoa_m': 1.0, 'dcacoa_m': -1.0})

In [74]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('PTE') and r.id.endswith('m'):
        r.gene_reaction_rule = '14585'

Acyl-CoA synthetase  
11167	cyto 11, cyto_nucl 8, mito 7, nucl 3, plas 3	K01897: ACSL, fadD; long-chain acyl-CoA synthetase	VYP*  
15745	cyto_nucl 11.333, nucl 11, cyto 7.5, cyto_mito 6.332, extr 4, mito 3.5	KOG1256: Long-chain acyl-CoA synthetases (AMP-forming)	VDS*  
15746	cyto 13, mito 5, cysk 4, mito_nucl 4	KOG1180: Acyl-CoA synthetase	EPY*  
15748	cyto 23, mito 3	K01897: ACSL, fadD; long-chain acyl-CoA synthetase	VYP*  
15745 and 15746 look like splitted genes -> 899170 SigP cyto 15.5, cyto_mito 9, nucl 7, pero 2, mito 1.5, cysk 1 VDS* 

12555	cyto 15.5, cyto_nucl 11.5, pero 6, nucl 4.5	K01897: ACSL, fadD; long-chain acyl-CoA synthetase	AKL*  
15900	cyto 13, pero 11, cyto_nucl 8.5	K01896: ACSM; medium-chain acyl-CoA synthetase 	AKL*  
9912	extr 12, plas 8, pero 4, mito 1, cyto 1, E.R. 1, cyto_mito 1	K08745: SLC27A1_4, FATP1, FATP4; solute carrier family 27 (fatty acid transporter), member 1/4, Very long-chain acyl-CoA synthetase/fatty acid transporter
	VRL*  

12538	mito 18.5, cyto_mito 12.5, cyto 5.5	KOG1256: Long-chain acyl-CoA synthetases (AMP-forming)	GEQ*  
12272	mito 27	KOG1176: Acyl-CoA synthetase, blast to acetate/butyrate-CoA ligase, paperblast to 3-(methylthio)propionyl-CoA ligase	MSQ*  

15306	extr 20, cyto 4, mito 3	SUPERFAMILY::SSF56801, blast to acetyl-CoA synthetase-like protein	WLQ*  

12555 has fitness defect in oleic acid, and less defect in ricinoleic acid  
15746 has fitness defect in ricinoleic acid, but not in oleic acid  

In [75]:
temp = ['11167','15745','15746','15748','12555','12538','15900','9912','12272','15306']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
11167,"K01897: ACSL, fadD; long-chain acyl-CoA synthe...",,"FAA3,FAA4,FAA1","ACSL3,ACSL4",FAA1,ACSL4,Not Essential,"cyto 11, cyto_nucl 8, mito 7, nucl 3, plas 3",VYP*
15745,KOG1256: Long-chain acyl-CoA synthetases (AMP-...,,,,,,Not Essential,"cyto_nucl 11.333, nucl 11, cyto 7.5, cyto_mito...",VDS*
15746,KOG1180: Acyl-CoA synthetase,,,,FAA1,ACSL1,Not Essential,"cyto 13, mito 5, cysk 4, mito_nucl 4",EPY*
15748,"K01897: ACSL, fadD; long-chain acyl-CoA synthe...",,"FAA3,FAA4,FAA1","ACSL3,ACSL4",FAA1,ACSL4,Not Essential,"cyto 23, mito 3",VYP*
12555,"K01897: ACSL, fadD; long-chain acyl-CoA synthe...",,FAA2,"ACSL1,ACSL5,ACSL6,CTB-127M13.1",FAA2,ACSL1,Not Essential,"cyto 15.5, cyto_nucl 11.5, pero 6, nucl 4.5",AKL*
12538,KOG1256: Long-chain acyl-CoA synthetases (AMP-...,,FAA2,"ACSL1,ACSL5,ACSL6,CTB-127M13.1",FAA2,ACSL5,Not Essential,"mito 18.5, cyto_mito 12.5, cyto 5.5",GEQ*
15900,K01896: ACSM; medium-chain acyl-CoA synthetase,,,"ACSM1,ACSM2A,ACSM2B,ACSM3,ACSM4,ACSM5",ACS2,ACSM3,Not Essential,"cyto 13, pero 11, cyto_nucl 8.5",AKL*
9912,"K08745: SLC27A1_4, FATP1, FATP4; solute carrie...",S,FAT1,"SLC27A1,SLC27A4,SLC27A2,SLC27A5,SLC27A6",FAT1,SLC27,Not Essential,"extr 12, plas 8, pero 4, mito 1, cyto 1, E.R. ...",VRL*
12272,KOG1176: Acyl-CoA synthetase,,,,,,Not Essential,mito 27,MSQ*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
11167,0.0946003,-0.123058,-0.258405,-0.0485982,-0.0949716,0.12414,0.437731,0.018182,-0.147964,-0.0276757,-0.200199
15745,-0.1045,-0.0341638,0.065697,-0.0468312,0.0131684,-0.155592,-0.0930866,-0.0553649,-0.322587,-0.129447,0.125032
15746,-0.292855,-0.044816,-0.192011,-0.333648,-0.131629,0.0496685,0.183731,-1.1449,0.233525,0.530271,1.0953
15748,-0.455473,0.172527,-0.163183,-0.136613,-0.855098,-0.410175,-0.256403,-0.113934,-1.02932,-0.810487,0.0346502
12555,-0.311448,-0.162485,-0.131826,-0.274268,-0.854248,-0.027038,-1.52861,-0.659443,-0.300839,-0.371389,-0.434224
12538,-0.0228742,-0.00284459,0.0444012,0.0245939,0.0476792,-0.0249695,0.0981869,0.240673,0.0573442,0.0435764,0.0668027
15900,-0.402251,0.0381164,-0.120976,-0.323788,-0.290967,-0.292245,0.0775032,0.423076,0.367482,-0.304034,-0.394707
9912,0.0677425,-0.270103,-0.302936,-0.051684,-0.404229,0.0682701,-0.0630695,-0.272301,-0.340334,-0.0743227,-0.0288945
12272,-0.197686,-0.196943,-0.00208274,0.0746683,0.025135,0.0984501,-0.335589,-0.358104,-0.0593179,-0.0656978,0.0740662


In [76]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FACOAL'):
        print(r.id, r.reaction, r.gene_reaction_rule)

FACOAL100 atp_c + coa_c + dca_c <=> amp_c + dcacoa_c + ppi_c 11167 or 12538 or 12555 or 15748
FACOAL100p atp_x + coa_x + dca_x <=> amp_x + dcacoa_x + ppi_x 11167 or 12538 or 12555 or 15748
FACOAL100t2pp atp_c + coa_c + dca_p + h_p --> amp_c + dcacoa_c + h_c + ppi_c 11833 or 13700 or 14802 or 15900
FACOAL120 atp_c + coa_c + ddca_c <=> amp_c + ddcacoa_c + ppi_c 11167 or 12538 or 12555 or 15748
FACOAL120p atp_x + coa_x + ddca_x <=> amp_x + ddcacoa_x + ppi_x 11167 or 12538 or 12555 or 15748
FACOAL120t2pp atp_c + coa_c + ddca_p + h_p --> amp_c + ddcacoa_c + h_c + ppi_c 11833 or 13700 or 14802 or 15900
FACOAL140 atp_c + coa_c + ttdca_c <=> amp_c + ppi_c + tdcoa_c 11167 or 12538 or 12555 or 15748
FACOAL140p atp_x + coa_x + ttdca_x <=> amp_x + ppi_x + tdcoa_x 11167 or 12538 or 12555 or 15748
FACOAL140t2pp atp_c + coa_c + h_p + ttdca_p --> amp_c + h_c + ppi_c + tdcoa_c 11833 or 13700 or 14802 or 15900
FACOAL141 atp_c + coa_c + ttdcea_c <=> amp_c + ppi_c + tdecoa_c 11167 or 12538 or 12555 or 157

In [77]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FACOAL') and 'c' in r.compartments:
        print(r.id, r.reaction, r.gene_reaction_rule)

FACOAL100 atp_c + coa_c + dca_c <=> amp_c + dcacoa_c + ppi_c 11167 or 12538 or 12555 or 15748
FACOAL100t2pp atp_c + coa_c + dca_p + h_p --> amp_c + dcacoa_c + h_c + ppi_c 11833 or 13700 or 14802 or 15900
FACOAL120 atp_c + coa_c + ddca_c <=> amp_c + ddcacoa_c + ppi_c 11167 or 12538 or 12555 or 15748
FACOAL120t2pp atp_c + coa_c + ddca_p + h_p --> amp_c + ddcacoa_c + h_c + ppi_c 11833 or 13700 or 14802 or 15900
FACOAL140 atp_c + coa_c + ttdca_c <=> amp_c + ppi_c + tdcoa_c 11167 or 12538 or 12555 or 15748
FACOAL140t2pp atp_c + coa_c + h_p + ttdca_p --> amp_c + h_c + ppi_c + tdcoa_c 11833 or 13700 or 14802 or 15900
FACOAL141 atp_c + coa_c + ttdcea_c <=> amp_c + ppi_c + tdecoa_c 11167 or 12538 or 12555 or 15748
FACOAL141t2pp atp_c + coa_c + h_p + ttdcea_p --> amp_c + h_c + ppi_c + tdecoa_c 11833 or 13700 or 14802
FACOAL150 atp_c + coa_c + ptdca_c <=> amp_c + ppi_c + ptdcacoa_c 11167 or 12538 or 12555 or 15748
FACOAL160 atp_c + coa_c + hdca_c <=> amp_c + pmtcoa_c + ppi_c 11167 or 12538 or 125

In [78]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FACOAL') and r.id.endswith('t2pp'):
        print(r.id, r.reaction, r.gene_reaction_rule)

FACOAL100t2pp atp_c + coa_c + dca_p + h_p --> amp_c + dcacoa_c + h_c + ppi_c 11833 or 13700 or 14802 or 15900
FACOAL120t2pp atp_c + coa_c + ddca_p + h_p --> amp_c + ddcacoa_c + h_c + ppi_c 11833 or 13700 or 14802 or 15900
FACOAL140t2pp atp_c + coa_c + h_p + ttdca_p --> amp_c + h_c + ppi_c + tdcoa_c 11833 or 13700 or 14802 or 15900
FACOAL141t2pp atp_c + coa_c + h_p + ttdcea_p --> amp_c + h_c + ppi_c + tdecoa_c 11833 or 13700 or 14802
FACOAL160t2pp atp_c + coa_c + h_p + hdca_p --> amp_c + h_c + pmtcoa_c + ppi_c 11833 or 13700 or 14802 or 15900
FACOAL161t2pp atp_c + coa_c + h_p + hdcea_p --> amp_c + h_c + hdcoa_c + ppi_c 11833 or 13700 or 14802
FACOAL180t2pp atp_c + coa_c + h_p + ocdca_p --> amp_c + h_c + ppi_c + stcoa_c 11833 or 13700 or 14802
FACOAL181t2pp atp_c + coa_c + h_p + ocdcea_p --> amp_c + h_c + odecoa_c + ppi_c 11833 or 13700 or 14802
FACOAL60t2pp atp_c + coa_c + h_p + hxa_p --> amp_c + h_c + hxcoa_c + ppi_c 11833 or 13700 or 14802 or 15900
FACOAL80t2pp atp_c + coa_c + h_p + o

In [79]:
temp = [r.id for r in model.reactions if r.id.startswith('FACOAL') and r.id.endswith('t2pp')]
model.remove_reactions(temp, remove_orphans=True)

In [80]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FACOAL'):
        r.lower_bound = 0.0

FACOAL180_1 ocdccoa_c is stcoa_c -> replace ocdccoa_c with stcoa, and remove FACOAL180_1  
FACOAL18111Z ocdcecoa_c is 11Z and same as vacccoa_c, but coming from ocdcea_c 9Z  
 -> replace ocdcecoa_c with vacccoa_c, and remove FACOAL18111Z,  
    replace ocdcea_c with vacc_c in 11Z reactions  
FACOAL181_1 ocdce9a_c is the same as ocdcea_c, and ocdce9coa_c is the same as odecoa_c  
 -> replace ocdce9a_c with ocdcea_c and ocdce9coa_c with odecoa_c, and remove FACOAL181_1, FACOAE1819Z  
    COA1819ZD9DS is transferring electrons from O2 to focytb5_c  
FACOAL204_copy1 -> FACOAL204 and remove FACOAL204_copy2  
FACOAL226_copy1 -> FACOAL226  

In [81]:
for r in model.metabolites.get_by_id('ocdccoa_c').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('ocdcecoa_c').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('vacccoa_c').reactions:
    print(r.id, r.reaction)

ACOADAGAT1819Z18111Z180 12dgr1819Z18111Z_c + ocdccoa_c --> coa_c + tag1819Z18111Z180_c
ACOADAGAT1601819Z180 12dgr1601819Z_c + ocdccoa_c --> coa_c + tag1601819Z180_c
ACOADAGAT18111Z1819Z180 12dgr18111Z1819Z_c + ocdccoa_c --> coa_c + tag18111Z1819Z180_c
FACOAL180_1 atp_c + coa_c + ocdca_c --> amp_c + ocdccoa_c + ppi_c
ACOADAGAT18111Z18111Z180 12dgr18111Z18111Z_c + ocdccoa_c --> coa_c + tag18111Z18111Z180_c
ACOADAGAT16018111Z180 12dgr16018111Z_c + ocdccoa_c --> coa_c + tag16018111Z180_c
ACOADAGAT1801819Z180 12dgr1801819Z_c + ocdccoa_c --> coa_c + tag1801819Z180_c
ACOADAGAT1819Z1819Z180 12dgr1819Z1819Z_c + ocdccoa_c --> coa_c + tag1819Z1819Z180_c

ACOADAGAT18111Z18111Z18111Z 12dgr18111Z18111Z_c + ocdcecoa_c --> coa_c + tag18111Z18111Z18111Z_c
ACOADAGAT1601819Z18111Z 12dgr1601819Z_c + ocdcecoa_c --> coa_c + tag1601819Z18111Z_c
AGPATCOA1819Z18111Z 1odec9eg3p_c + ocdcecoa_c --> coa_c + pa1819Z18111Z_c
ACOADAGAT18111Z1819Z18111Z 12dgr18111Z1819Z_c + ocdcecoa_c --> coa_c + tag18111Z1819Z18111Z_

In [82]:
for r in model.metabolites.get_by_id('ocdce9a_c').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('ocdcea_c').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('ocdce9coa_c').reactions:
    print(r.id, r.reaction)
print()
for r in model.metabolites.get_by_id('odecoa_c').reactions:
    print(r.id, r.reaction)

PLPSA21801819Z h2o_c + pe1801819Z_c --> 1agpe180_c + h_c + ocdce9a_c
FACOAE1819Z h2o_c + ocdce9coa_c --> coa_c + h_c + ocdce9a_c
MAGAH1819Z h2o_c + mag1819Z_c --> glyc_c + h_c + ocdce9a_c
FACOAL181_1 atp_c + coa_c + ocdce9a_c --> amp_c + ocdce9coa_c + ppi_c
PLPSA21819Z1819Z h2o_c + pe1819Z1819Z_c --> 1agpe1819Z_c + h_c + ocdce9a_c
PLPSA218111Z1819Z h2o_c + pe18111Z1819Z_c --> 1agpe18111Z_c + h_c + ocdce9a_c
LPLPS1AGPE1819Z 1agpe1819Z_c + h2o_c --> g3pe_c + h_c + ocdce9a_c

FA181ACPH h2o_c + octeACP_c <=> ACP_c + h_c + ocdcea_c
FATP2t na1_c + ocdcea_c <=> na1_e + ocdcea_e
FAS181 4.0 h_c + hdca_c + malcoa_c + 3.0 nadph_c + o2_c --> co2_c + coa_c + 3.0 h2o_c + 3.0 nadp_c + ocdcea_c
FECOSTESTH_SC 0.01 fecostest_SC_c + h2o_c --> 0.01 fecost_c + h_c + 0.02 hdca_c + 0.655 hdcea_c + 0.01 hexc_c + 0.03 ocdca_c + 0.27 ocdcea_c + 0.015 ttdca_c
TAGL_SC h2o_c + 0.01 triglyc_SC_c --> 0.01 12dgr_SC_c + 0.02 dca_c + 0.06 ddca_c + h_c + 0.27 hdca_c + 0.17 hdcea_c + 0.05 ocdca_c + 0.24 ocdcea_c + 0.09 o

In [83]:
for r in model.metabolites.get_by_id('ocdccoa_c').reactions:
    r.add_metabolites({'ocdccoa_c': -r.get_coefficient('ocdccoa_c'), 'stcoa_c': r.get_coefficient('ocdccoa_c')})
model.remove_reactions(['FACOAL180_1'], remove_orphans=True)
for r in model.metabolites.get_by_id('ocdcecoa_c').reactions:
    r.add_metabolites({'ocdcecoa_c': -r.get_coefficient('ocdcecoa_c'), 'vacccoa_c': r.get_coefficient('ocdcecoa_c')})
model.remove_reactions(['FACOAL18111Z'], remove_orphans=True)
for r in model.metabolites.get_by_id('ocdcea_c').reactions:
    if '18111Z' in r.id:
        r.add_metabolites({'ocdcea_c': -r.get_coefficient('ocdcea_c'), 'vacc_c': r.get_coefficient('ocdcea_c')})
for r in model.metabolites.get_by_id('ocdce9a_c').reactions:
    r.add_metabolites({'ocdce9a_c': -r.get_coefficient('ocdce9a_c'), 'ocdcea_c': r.get_coefficient('ocdce9a_c')})
for r in model.metabolites.get_by_id('ocdce9coa_c').reactions:
    r.add_metabolites({'ocdce9coa_c': -r.get_coefficient('ocdce9coa_c'), 'odecoa_c': r.get_coefficient('ocdce9coa_c')})
model.remove_reactions(['FACOAL181_1','FACOAE1819Z'], remove_orphans=True)
model.reactions.get_by_id('FACOAL204_copy1').id = 'FACOAL204'
model.reactions.get_by_id('FACOAL226_copy1').id = 'FACOAL226'
model.remove_reactions(['FACOAL204_copy2'], remove_orphans=True)

In [84]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FACOAL') and 'c' in r.compartments:
        r.gene_reaction_rule = '11167 or 15746 or 15748'

In [85]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if (not r.id.startswith('FACOAL')) and 'atp_c' in [m.id for m in r.reactants] and 'coa_c' in [m.id for m in r.reactants]:
        print(r.id, r.reaction, r.gene_reaction_rule)

AACOAT acac_c + atp_c + coa_c <=> aacoa_c + amp_c + ppi_c 15276
ACITL atp_c + cit_c + coa_c --> accoa_c + adp_c + oaa_c + pi_c CRv4_Au5_s5_g12182_t1 and 9726
ACS ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c 14597
ACS2 atp_c + coa_c + ppa_c --> amp_c + ppcoa_c + ppi_c 14597
CACOAL atp_c + ca_c + coa_c --> amp_c + cacoa_c + h_c + ppi_c 11167 or 12538 or 12555 or 15748
CRNCAL2 atp_c + coa_c + crn_c --> adp_c + crncoa_c + pi_c 11833 or 13700
CRNDCAL2 atp_c + coa_c + crn__D_c --> adp_c + crnDcoa_c + pi_c 11833 or 13700
CTBTCAL2 atp_c + coa_c + ctbt_c --> adp_c + ctbtcoa_c + pi_c 11833 or 13700
PACOAL atp_c + coa_c + pa_c --> amp_c + pacoa_c + ppi_c 11167 or 12538 or 12555 or 15748
SUCOAS atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c 15967 and 16144
yli_R0144 atp_c + coa_c + yli_M04636_c <=> amp_c + ppi_c + yli_M04635_c 11167 or 12538 or 12555 or 15748
yli_R0146 atp_c + coa_c + yli_M03299_c <=> amp_c + ppi_c + yli_M04594_c 11167 or 12538 or 12555 or 15748
yli_R0148 atp_c + coa_c + y

In [86]:
# Fix ACITL GPR and remove 5Z fatty acid reactions and redundant yli reactions
model.reactions.get_by_id('ACITL').gene_reaction_rule = '9726'
# Pinolenic acid and Coniferonic acid only in iRC model
model.remove_reactions(['PACOAL','CACOAL'], remove_orphans=True)
model.remove_reactions(['yli_R0144','yli_R0146','yli_R0148','yli_R0149','yli_R0151'], remove_orphans=True)

Other Acyl-CoA synthetase  
14597	cyto 13, plas 5, mito 3, pero 2, mito_nucl 2	K01895: ACSS, acs; acetyl-CoA synthetase	SSE* -> OK  
15276	plas 9, pero 6, mito 5, cyto 5, cyto_mito 5	K01907: AACS, acsA; acetoacetyl-CoA synthetase	SKL* -> OK  
12122	nucl 11, cyto_nucl 10, mito 9, cyto 7	K01641: E2.3.3.10; hydroxymethylglutaryl-CoA synthase	HVA* -> Fix  

In [87]:
temp = ['14597','15276','12122']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
14597,"K01895: ACSS, acs; acetyl-CoA synthetase",,"ACS1,ACS2","ACSS1,ACSS2",ACS2,ACSS1,Not Essential,"cyto 13, plas 5, mito 3, pero 2, mito_nucl 2",SSE*
15276,"K01907: AACS, acsA; acetoacetyl-CoA synthetase",,,AACS,ACS1,AACS,Not Essential,"plas 9, pero 6, mito 5, cyto 5, cyto_mito 5",SKL*
12122,K01641: E2.3.3.10; hydroxymethylglutaryl-CoA s...,,ERG13,"HMGCS1,HMGCS2",ERG13,HMGCS,Essential,"nucl 11, cyto_nucl 10, mito 9, cyto 7",HVA*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
14597,0.134527,0.0997527,0.107444,-1.05713,0.0951078,-0.0881028,0.151616,0.954851,0.0158946,-0.116256,0.000665712
15276,-0.192767,0.221437,0.0764749,-0.176646,0.00381209,-0.117964,-0.0335189,-0.260812,-0.126874,-0.0575812,-0.262471


In [88]:
for r in sorted(model.genes.get_by_id('14597').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15276').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12122').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ACS ac_c + atp_c + coa_c --> accoa_c + amp_c + ppi_c 14597
ACS2 atp_c + coa_c + ppa_c --> amp_c + ppcoa_c + ppi_c 14597
ACSm ac_m + atp_m + coa_m --> accoa_m + amp_m + ppi_m 14597

AACOAT acac_c + atp_c + coa_c <=> aacoa_c + amp_c + ppi_c 15276

HMGCOAS coa_c + h_c + hmgcoa_c <=> aacoa_c + accoa_c + h2o_c 12122
HMGCOASm coa_m + h_m + hmgcoa_m <=> aacoa_m + accoa_m + h2o_m 12122
MHGS aacoa_c + accoa_c + h2o_c --> coa_c + hmgcoa_c 12122
yli_R0771 aacoa_c + accoa_c + h2o_c --> coa_c + h_c + yli_M00348_c 12122
yli_R1457 aacoa_m + accoa_m + h2o_m --> coa_m + h_m + yli_M00348_m 12122


In [89]:
for r in sorted(model.metabolites.get_by_id('hmgcoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('yli_M00348_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('hmgcoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('yli_M00348_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('hmgcoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('hmgcoa_r').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

HMGCOAR coa_c + mev__R_c + 2.0 nadp_c <=> 2.0 h_c + hmgcoa_c + 2.0 nadph_c 9574
HMGCOAS coa_c + h_c + hmgcoa_c <=> aacoa_c + accoa_c + h2o_c 12122
HMGL hmgcoa_c --> acac_c + accoa_c 15693
MHGS aacoa_c + accoa_c + h2o_c --> coa_c + hmgcoa_c 12122

yli_R0765 2.0 h_c + 2.0 nadph_c + yli_M00348_c --> coa_c + mev__R_c + 2.0 nadp_c 9574
yli_R0771 aacoa_c + accoa_c + h2o_c --> coa_c + h_c + yli_M00348_c 12122
yli_R0773 yli_M00348_c --> acac_c + accoa_c 15693

HMGCOASm coa_m + h_m + hmgcoa_m <=> aacoa_m + accoa_m + h2o_m 12122
HMGL_1 hmgcoa_m --> acac_m + accoa_m + h_m 15693
HMGLm hmgcoa_m --> acac_m + accoa_m 15693
MGCHrm 3mgcoa_m + h2o_m <=> hmgcoa_m 16128

yli_R1457 aacoa_m + accoa_m + h2o_m --> coa_m + h_m + yli_M00348_m 12122
yli_R1458 yli_M00348_m --> acac_m + accoa_m 15693

HMGCOARx 2.0 h_x + hmgcoa_x + 2.0 nadph_x --> coa_x + mev__R_x + 2.0 nadp_x 9574
HMGLx hmgcoa_x --> acac_x + accoa_x 15693

HMGCOARr 2.0 h_r + hmgcoa_r + 2.0 nadph_r --> coa_r + mev__R_r + 2.0 nadp_r 9574


In [90]:
model.remove_reactions(['HMGL_1','yli_R0765','yli_R0771','yli_R0773','yli_R1457','yli_R1458'], remove_orphans=True)

In [91]:
temp = ['11833','13700','14802','16129','16635']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
11833,KOG1176: Acyl-CoA synthetase,,,,PCS60,ACSF2,Not Essential,"cyto 16, cyto_nucl 11.5, nucl 5, pero 5",AKL*
13700,K01904: 4CL; 4-coumarate--CoA ligase,,,,PCS60,ACSM3,Not Essential,"cyto 16.5, cyto_nucl 12.5, nucl 5.5, mito 3",LPS*
14802,K01904: 4CL; 4-coumarate--CoA ligase,,,,PCS60,ACSF2,Not Essential,"cyto 17.5, cyto_nucl 9.5, plas 5, pero 2",SKL*
16129,KOG1176: Acyl-CoA synthetase,,,,PCS60,ACSF2,Not Essential,"nucl 6.5, cyto_nucl 6.5, plas 6, cyto 5.5, mit...",AKL*
16635,"K01897: ACSL, fadD; long-chain acyl-CoA synthe...",,,,PCS60,ACSF2,Not Essential,"mito 18, cyto 5.5, cyto_nucl 3.5, pero 3",AKL*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
11833,0.218588,-0.139988,0.12494,-0.131598,-0.0323055,0.0393466,0.176377,0.128813,0.120682,-0.0829543,-0.133672
13700,-0.107884,-0.155441,-0.25423,-0.0388004,0.149475,-0.0544939,-0.0665896,-0.16933,-0.25386,-0.32858,-0.324718
14802,0.318433,0.250797,0.241559,0.171794,-0.0119312,0.176682,-0.1177,-0.0917166,0.131314,-0.0334729,-0.125359
16129,-0.223651,-0.0133421,0.159221,-0.114595,-0.0296944,-0.117031,0.0551636,0.0633251,0.415751,0.445395,-0.195331
16635,-0.386094,0.0748675,-0.24649,-0.872329,-1.76568,-3.60067,-0.198188,-0.0374127,0.311272,0.402185,0.87277


11833	Acyl-CoA synthetase, blast to 4-coumarate--CoA ligase	AKL*  
13700	K01904: 4CL; 4-coumarate--CoA ligase	LPS*  
14802	K01904: 4CL; 4-coumarate--CoA ligase	SKL*, no reaction  
16129	KOG1176: Acyl-CoA synthetase, blast to 4-coumarate--CoA ligase / succinylbenzoate--CoA ligase	AKL*  
16635	K01897: ACSL, fadD; long-chain acyl-CoA synthetase, blast to 4-coumarate--CoA ligase	AKL*  
*16635 has fitness defect and is upregulated in coumarate and ferulate, while 11183, 13700, 14802 do/are not

In [92]:
for r in sorted(model.genes.get_by_id('11833').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

CRNCAL2 atp_c + coa_c + crn_c --> adp_c + crncoa_c + pi_c 11833 or 13700
CRNDCAL2 atp_c + coa_c + crn__D_c --> adp_c + crnDcoa_c + pi_c 11833 or 13700
CTBTCAL2 atp_c + coa_c + ctbt_c --> adp_c + ctbtcoa_c + pi_c 11833 or 13700


In [93]:
print(hsa2.reactions.get_by_id('4CMCOAS'))
print(hsa2.reactions.get_by_id('COUCOAFm'))

4CMCOAS: T4hcinnm_c + atp_c + coa_c --> amp_c + coucoa_c + ppi_c
COUCOAFm: T4hcinnm_m + atp_m + coa_m --> amp_m + coucoa_m + ppi_m


In [94]:
model.remove_reactions(['CRNCAL2','CRNDCAL2','CTBTCAL2'], remove_orphans=True)
r1 = hsa2.reactions.get_by_id('4CMCOAS').copy() # 4-Coumarate-coA ligase
r1.gene_reaction_rule = '11833 or 13700 or 14802 or 16129'
r2 = hsa2.reactions.get_by_id('COUCOAFm').copy() # P coumaroyl CoA formation
r2.gene_reaction_rule = '16635'
model.add_reactions([r1,r2])

15900	cyto 13, pero 11, cyto_nucl 8.5	K01896: ACSM; medium-chain acyl-CoA synthetase 	AKL*  
12555	cyto 15.5, cyto_nucl 11.5, pero 6, nucl 4.5	K01897: ACSL, fadD; long-chain acyl-CoA synthetase	AKL*  
9912	extr 12, plas 8, pero 4, mito 1, cyto 1, E.R. 1, cyto_mito 1	K08745: SLC27A1_4, FATP1, FATP4; solute carrier family 27 (fatty acid transporter), member 1/4, Very long-chain acyl-CoA synthetase/fatty acid transporter
	VRL* 

In [95]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FACOAL') and 'x' in r.compartments:
        print(r.id, r.reaction, r.gene_reaction_rule)

FACOAL100p atp_x + coa_x + dca_x --> amp_x + dcacoa_x + ppi_x 11167 or 12538 or 12555 or 15748
FACOAL120p atp_x + coa_x + ddca_x --> amp_x + ddcacoa_x + ppi_x 11167 or 12538 or 12555 or 15748
FACOAL140p atp_x + coa_x + ttdca_x --> amp_x + ppi_x + tdcoa_x 11167 or 12538 or 12555 or 15748
FACOAL141p atp_x + coa_x + ttdcea_x --> amp_x + ppi_x + tdecoa_x 11167 or 12538 or 12555 or 15748
FACOAL160p atp_x + coa_x + hdca_x --> amp_x + pmtcoa_x + ppi_x 11167 or 12538 or 12555 or 15748
FACOAL161p atp_x + coa_x + hdcea_x --> amp_x + hdcoa_x + ppi_x 11167 or 12538 or 12555 or 15748
FACOAL240p atp_x + coa_x + ttc_x --> amp_x + ppi_x + ttccoa_x 9912
FACOAL260p atp_x + coa_x + hexc_x --> amp_x + hexccoa_x + ppi_x 9912
FACOAL80p atp_x + coa_x + octa_x --> amp_x + occoa_x + ppi_x 11167 or 12538 or 12555 or 15748


In [96]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FACOAL') and 'x' in r.compartments and (not r.id.startswith('FACOAL2')):
        r.gene_reaction_rule = '12555 or 15900'

In [97]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if (not r.id.startswith('FACOAL')) and 'atp_x' in [m.id for m in r.reactants] and 'coa_x' in [m.id for m in r.reactants]:
        print(r.id, r.reaction, r.gene_reaction_rule)

VLCS2p atp_x + coa_x + dhcholestanate_x --> amp_x + dhcholestancoa_x + ppi_x 9912
VLCSp atp_x + coa_x + thcholstoic_x --> amp_x + cholcoar_x + ppi_x 9912


In [98]:
for r in sorted(model.genes.get_by_id('9912').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

FA240tp ttc_c --> ttc_x 9912
FA260tp hexc_c --> hexc_x 9912
FACOAL240p atp_x + coa_x + ttc_x --> amp_x + ppi_x + ttccoa_x 9912
FACOAL260p atp_x + coa_x + hexc_x --> amp_x + hexccoa_x + ppi_x 9912
FATP1t hdca_c + na1_c <=> hdca_e + na1_e 9912
FATP2t na1_c + ocdcea_c <=> na1_e + ocdcea_e 9912
FATP3t na1_c + ocdca_c <=> na1_e + ocdca_e 9912
FATP4t arach_c + na1_c <=> arach_e + na1_e 9912
FATP5t arachd_c + na1_c <=> arachd_e + na1_e 9912
FATP6t adrn_c + na1_c <=> adrn_e + na1_e 9912
FATP7t crvnc_c + na1_c <=> crvnc_e + na1_e 9912
FATP8t na1_c + ttc_c <=> na1_e + ttc_e 9912
FATP9t na1_c + nrvnc_c <=> na1_e + nrvnc_e 9912
VLCS2p atp_x + coa_x + dhcholestanate_x --> amp_x + dhcholestancoa_x + ppi_x 9912
VLCS2r atp_r + coa_r + dhcholestanate_r --> amp_r + dhcholestancoa_r + ppi_r 9912
VLCSp atp_x + coa_x + thcholstoic_x --> amp_x + cholcoar_x + ppi_x 9912
VLCSr atp_r + coa_r + thcholstoic_r --> amp_r + cholcoar_r + ppi_r 9912


12538	mito 18.5, cyto_mito 12.5, cyto 5.5	KOG1256: Long-chain acyl-CoA synthetases (AMP-forming)	GEQ*  
12272	mito 27	KOG1176: Acyl-CoA synthetase, blast to acetate/butyrate-CoA ligase, paperblast to 3-(methylthio)propionyl-CoA ligase	MSQ*

12538 expression does not match the gene model and the gene model is < 50% coverage blast

In [99]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FACOAL') and 'm' in r.compartments:
        print(r.id, r.reaction, r.gene_reaction_rule)

FACOAL40im atp_m + but_m + coa_m --> amp_m + btcoa_m + ppi_m 15900
FACOAL_160_m atp_m + coa_m + hdca_m --> amp_m + pmtcoa_m + ppi_m 11167 or 12538 or 12555 or 15748


In [100]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if (not r.id.startswith('FACOAL')) and 'atp_m' in [m.id for m in r.reactants] and 'coa_m' in [m.id for m in r.reactants]:
        print(r.id, r.reaction, r.gene_reaction_rule)

ACSm ac_m + atp_m + coa_m --> accoa_m + amp_m + ppi_m 14597
COUCOAFm T4hcinnm_m + atp_m + coa_m --> amp_m + coucoa_m + ppi_m 16635
ITCOALm atp_m + coa_m + itacon_m <=> adp_m + itaccoa_m + pi_m 15967 and 16144
MECOALm atp_m + coa_m + mescon_m <=> adp_m + mescoa_m + pi_m 15967 and 16144
SUCLm atp_m + coa_m + h_m + succ_m <=> adp_m + pi_m + succoa_m 15967 and 16144
SUCOASm atp_m + coa_m + succ_m <=> adp_m + pi_m + succoa_m 15967 and 16144
yli_R0233 atp_m + coa_m + dca_m <=> amp_m + dccoa_m + ppi_m 11167 or 12538 or 12555 or 15748
yli_R0234 atp_m + coa_m + ddca_m <=> amp_m + ddcacoa_m + ppi_m 11167 or 12538 or 12555 or 15748
yli_R0235 atp_m + coa_m + ttdca_m <=> amp_m + ppi_m + tdcoa_m 11167 or 12538 or 12555 or 15748
yli_R0236 atp_m + coa_m + yli_M04636_m <=> amp_m + ppi_m + yli_M04635_m 11167 or 12538 or 12555 or 15748
yli_R0238 atp_m + coa_m + yli_M03299_m <=> amp_m + ppi_m + yli_M04594_m 11167 or 12538 or 12555 or 15748
yli_R0240 atp_m + coa_m + yli_M04622_m <=> amp_m + ppi_m + yli_M04

In [101]:
model.reactions.get_by_id('ACSm').gene_reaction_rule = '12272 or 14597'
model.reactions.get_by_id('FACOAL40im').gene_reaction_rule = '12272'

In [102]:
model.remove_reactions(['FACOAL_160_m','yli_R0233','yli_R0234','yli_R0235','yli_R0236','yli_R0238',
                        'yli_R0240','yli_R0241','yli_R0244'], remove_orphans=True)

15967	mito 18.5, cyto_mito 12.5, cyto 5.5	K01900: LSC2; succinyl-CoA synthetase beta subunit	AAQ*  
16144	mito 24, pero 2	K01899: LSC1; succinyl-CoA synthetase alpha subunit	GLA*

In [103]:
for r in sorted(model.genes.get_by_id('15967').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('16144').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ITCOALm atp_m + coa_m + itacon_m <=> adp_m + itaccoa_m + pi_m 15967 and 16144
MECOALm atp_m + coa_m + mescon_m <=> adp_m + mescoa_m + pi_m 15967 and 16144
SUCLm atp_m + coa_m + h_m + succ_m <=> adp_m + pi_m + succoa_m 15967 and 16144
SUCOAS atp_c + coa_c + succ_c <=> adp_c + pi_c + succoa_c 15967 and 16144
SUCOAS1m coa_m + gtp_m + succ_m <=> gdp_m + pi_m + succoa_m (SUCLG2 and 16144) or (Suclg2 and 16144) or (15967 and 16144)
SUCOASm atp_m + coa_m + succ_m <=> adp_m + pi_m + succoa_m 15967 and 16144

ITCOAL1m coa_m + gtp_m + itacon_m <=> gdp_m + itaccoa_m + pi_m (SUCLG2 and 16144) or (Suclg2 and 16144)
ITCOALm atp_m + coa_m + itacon_m <=> adp_m + itaccoa_m + pi_m 15967 and 16144
MECOALm atp_m + coa_m + mescon_m <=> adp_m + mescoa_m + pi_m 15967 and 16144
MECOAS1m coa_m + gtp_m + mescon_m <=> gdp_m + mescoa_m + pi_m (SUCLG2 and 16144) or (Suclg2 and 16144)
SUCLm atp_m + coa_m + h_m + succ_m <=> adp_m + pi_m + succoa_m 15967 and 16144
SUCOAS atp_c + coa_c + succ_c <=> adp_c + pi_c + succ

In [104]:
# Remove irrelevant and wrong reactions
model.remove_reactions(['ITCOALm','MECOALm','SUCLm','SUCOAS','SUCOAS1m','ITCOAL1m','MECOAS1m'], remove_orphans=True)

In [105]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('DESAT'):
        print(r.id, r.reaction, r.gene_reaction_rule)

DESAT16 h_c + nadph_c + o2_c + pmtcoa_c --> 2.0 h2o_c + hdcoa_c + nadp_c 9730
DESAT16_2 h_c + nadh_c + o2_c + pmtcoa_c --> 2.0 h2o_c + hdcoa_c + nad_c 9730
DESAT18 h_c + nadph_c + o2_c + stcoa_c --> 2.0 h2o_c + nadp_c + odecoa_c 9730
DESAT18_10 h_c + lnlncacoa_c + nadh_c + o2_c --> 2.0 h2o_c + nad_c + strdnccoa_c 9578
DESAT18_3 h_c + nadh_c + o2_c + stcoa_c --> 2.0 h2o_c + nad_c + odecoa_c 9730
DESAT18_4 h_c + nadh_c + o2_c + stcoa_c --> 2.0 h2o_c + nad_c + vacccoa_c 9730
DESAT18_5 h_c + nadh_c + o2_c + stcoa_c --> 2.0 h2o_c + nad_c + od2coa_c 9730
DESAT18_6 h_c + nadh_c + o2_c + odecoa_c --> 2.0 h2o_c + lneldccoa_c + nad_c 9578
DESAT18_7 h_c + nadh_c + o2_c + vacccoa_c --> 2.0 h2o_c + lneldccoa_c + nad_c 9578
DESAT18_8 h_c + nadh_c + o2_c + od2coa_c --> 2.0 h2o_c + lneldccoa_c + nad_c 9578
DESAT18_9 h_c + lnlccoa_c + nadh_c + o2_c --> 2.0 h2o_c + lnlncgcoa_c + nad_c 9578
DESAT20_1 dlnlcgcoa_c + h_c + nadh_c + o2_c --> arachdcoa_c + 2.0 h2o_c + nad_c 9578
DESAT20_2 eicostetcoa_c + h_c 

9730	mito 8, plas 8, cyto 7.5, cyto_nucl 5, nucl 1.5	K00507: SCD, desC; stearoyl-CoA desaturase (Delta-9 desaturase), has cytochrome b5-like domain	VKA*  
8845	plas 12, mito 9, cyto 2, pero 2, cyto_pero 2	K10256: FAD2; omega-6 fatty acid desaturase / acyl-lipid omega-6 desaturase (Delta-12 desaturase), no cytochrome b5-like domain, introduces a cis double bond at position 12 of fatty-acyl-CoAs that contain a cis double bond at position 9, Requires cytochrome b5 as the electron donor, introduces a cis double bond in fatty acids attached to lipid molecules at a location 6 carbons away from the methyl end	EGL*  
9578	plas 22, mito 3	K13076: SLD; Delta 6-fatty acid desaturase/delta-8 sphingolipid desaturase, has cytochrome b5-like domain, introduces a double bond at Delta 6 of fatty acids, introduces a trans double bond at the 8-position of sphingoid bases in sphingolipids	LHH*  

Fatty acid desaturase  
9730 - saturated -> 9Z  
8845 - 9Z -> 9Z, 12Z  
9578 - 9Z,12Z -> 6Z,9Z,12Z

In [106]:
for r in sorted(model.genes.get_by_id('9730').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8845').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9578').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

DESAT16 h_c + nadph_c + o2_c + pmtcoa_c --> 2.0 h2o_c + hdcoa_c + nadp_c 9730
DESAT16_2 h_c + nadh_c + o2_c + pmtcoa_c --> 2.0 h2o_c + hdcoa_c + nad_c 9730
DESAT18 h_c + nadph_c + o2_c + stcoa_c --> 2.0 h2o_c + nadp_c + odecoa_c 9730
DESAT18_3 h_c + nadh_c + o2_c + stcoa_c --> 2.0 h2o_c + nad_c + odecoa_c 9730
DESAT18_4 h_c + nadh_c + o2_c + stcoa_c --> 2.0 h2o_c + nad_c + vacccoa_c 9730
DESAT18_5 h_c + nadh_c + o2_c + stcoa_c --> 2.0 h2o_c + nad_c + od2coa_c 9730
DGDGD7DS1819Z1617Z dgdg1819Z160_h + fdxox_h + 4.0 nadph_h + o2_h --> dgdg1819Z1617Z_h + fdxrd_h + 2.0 h2o_h + 4.0 nadp_h (CRv4_Au5_s13_g4871_t1 and CRv4_Au5_s16_g6229_t1) or (CRv4_Au5_s13_g4871_t1 and CRv4_Au5_s17_g7064_t1) or (CRv4_Au5_s13_g4871_t1 and CRv4_Au5_s3_g10824_t1) or (CRv4_Au5_s13_g4871_t1 and CRv4_Au5_s6_g13523_t1) or (CRv4_Au5_s13_g4871_t1 and CRv4_Au5_s7_g14133_t1) or (CRv4_Au5_s16_g6229_t1 and 9730) or (CRv4_Au5_s17_g7064_t1 and 9730) or (CRv4_Au5_s3_g10824_t1 and 9730) or (CRv4_Au5_s6_g13523_t1 and 9730) or (

In [107]:
# Remove h compartment reactions, redundant reactions, sulfoquinovosyl-DAGs, and DAG-O4-NNN-trimetyl homoserine
# Remove delta 5 desaturase
model.remove_reactions(['DGDGD7DS1819Z1617Z','DGDGD7DS1829Z12Z1617Z','FAH120','FAH140','FAH160','FAH1819Z',
                        'FAH1829Z12Z','MGDGD7DS1819Z1617Z','MGDGD7DS1829Z12Z1617Z','yli_R0154','yli_R0155',
                        'ASQDCADS1829Z12Z160','ASQDCADS1839Z12Z15Z160','ASQDPADS1829Z12Z160','ASQDPADS1839Z12Z15Z160',
                        'DGTSD5DS1601835Z9Z12Z','DGTSD5DS18111Z1835Z9Z12Z','DGTSD5DS1819Z1835Z9Z12Z',
                        'DGTSD5DS1829Z12Z1835Z9Z12Z','DGTSDS1601829Z12Z','DGTSDS1601845Z9Z12Z15Z',
                        'DGTSDS18111Z1829Z12Z','DGTSDS18111Z1845Z9Z12Z15Z','DGTSDS1819Z1829Z12Z',
                        'DGTSDS1819Z1845Z9Z12Z15Z','DGTSDS1829Z12Z18111Z','DGTSDS1829Z12Z1819Z',
                        'DGTSDS1829Z12Z1829Z12Z1','DGTSDS1829Z12Z1829Z12Z2','DGTSDS1829Z12Z1835Z9Z12Z',
                        'DGTSDS1829Z12Z1845Z9Z12Z15Z1','DGTSDS1829Z12Z1845Z9Z12Z15Z2','DGTSDS1839Z12Z15Z18111Z',
                        'DGTSDS1839Z12Z15Z1819Z','DGTSDS1839Z12Z15Z1835Z9Z12Z','DGTSDS1839Z12Z15Z1845Z9Z12Z15Z1',
                        'DGTSDS1839Z12Z15Z1845Z9Z12Z15Z2',
                               ], remove_orphans=True)
# Remove 5Z fatty acid reactions
temp = [r.id for r in sorted(model.reactions, key=lambda x: x.id) if '5Z9Z' in r.id]
model.remove_reactions(temp, remove_orphans=True)

In [108]:
for r in sorted(model.genes.get_by_id('9730').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8845').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9578').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

DESAT16 h_c + nadph_c + o2_c + pmtcoa_c --> 2.0 h2o_c + hdcoa_c + nadp_c 9730
DESAT16_2 h_c + nadh_c + o2_c + pmtcoa_c --> 2.0 h2o_c + hdcoa_c + nad_c 9730
DESAT18 h_c + nadph_c + o2_c + stcoa_c --> 2.0 h2o_c + nadp_c + odecoa_c 9730
DESAT18_3 h_c + nadh_c + o2_c + stcoa_c --> 2.0 h2o_c + nad_c + odecoa_c 9730
DESAT18_4 h_c + nadh_c + o2_c + stcoa_c --> 2.0 h2o_c + nad_c + vacccoa_c 9730
DESAT18_5 h_c + nadh_c + o2_c + stcoa_c --> 2.0 h2o_c + nad_c + od2coa_c 9730

PEDS1801829Z12Z 2.0 nadh_c + o2_c + pe1801819Z_c --> 2.0 h2o_c + 2.0 nad_c + pe1801829Z12Z_c 8845
PEDS18111Z1829Z12Z 2.0 nadh_c + o2_c + pe18111Z1819Z_c --> 2.0 h2o_c + 2.0 nad_c + pe18111Z1829Z12Z_c 8845
PEDS1819Z1829Z12Z 2.0 nadh_c + o2_c + pe1819Z1819Z_c --> 2.0 h2o_c + 2.0 nad_c + pe1819Z1829Z12Z_c 8845

DESAT18_10 h_c + lnlncacoa_c + nadh_c + o2_c --> 2.0 h2o_c + nad_c + strdnccoa_c 9578
DESAT18_6 h_c + nadh_c + o2_c + odecoa_c --> 2.0 h2o_c + lneldccoa_c + nad_c 9578
DESAT18_7 h_c + nadh_c + o2_c + vacccoa_c --> 2.0 h2

In [109]:
# Keep delta-9 NADH reactions for 9730
model.reactions.get_by_id('DESAT16_2').id = 'DESAT1619Z'
model.reactions.get_by_id('DESAT18_3').id = 'DESAT1819Z'
model.remove_reactions(['DESAT16','DESAT18','DESAT18_4','DESAT18_5'], remove_orphans=True)
# Keep omega-6/delta-12 reactions for 8845
# 8845 needs cytchrome b5 and reductase 10123 and 16097
# C18:1 9Z -> C18:2 9Z12Z are missing -> add, how about C16:1 9Z -> C16:2 9Z12Z?
# Remove 5Z unsaturated fatty acids - no gene 
# C18:2 is named ocdcya/ocdycacoa in iMM904, but lnlc/lnlccoa in Recon1/3D 
# replace ocdcya/ocdycacoa with lnlc/lnlccoa 
r = sce.reactions.get_by_id('DESAT18_2').copy()
r.id = 'DESAT1829Z12Z'
model.add_reactions([r])
model.reactions.get_by_id('DESAT1829Z12Z').add_metabolites({'nadph_c': -r.get_coefficient('nadph_c'),
                                                            'nadh_c': r.get_coefficient('nadph_c'),
                                                            'nadp_c': -r.get_coefficient('nadp_c'),
                                                            'nad_c': r.get_coefficient('nadp_c')})
model.reactions.get_by_id('DESAT1829Z12Z').gene_reaction_rule = '10123 and 16097 and 8845'
# 8845 is known to desaturate PCs, but not PEs
model.remove_reactions(['PEDS1801829Z12Z','PEDS18111Z1829Z12Z','PEDS1819Z1829Z12Z'], remove_orphans=True)
# Keep delta-6 fatty acid and delta-8 sphingolipid reactions for 9578
# lneldc_c is all trans C18:2, but reaction says 6Z,9Z -> remove lneldc reactions 
model.reactions.get_by_id('DESAT18_9').id = 'DESAT1836Z9Z12Z'
model.remove_reactions(['DESAT18_6','DESAT18_7','DESAT18_8'], remove_orphans=True)
# DESAT18_10, DESAT24_1 are 6Z okay but irrelevant, DESAT20_1, DESAT20_2 is acyl-lipid 5 desaturase for arachidonate 
model.remove_reactions(['DESAT18_10','DESAT20_1','DESAT20_2','DESAT24_1'], remove_orphans=True)
# Desaturase localized to ER, change cyto to ER
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('DESAT'):
        r.id = r.id+'er'
        for m in r.metabolites:
            if not m.id.replace('_c','_r') in model.metabolites:
                m2 = m.copy()
                m2.id = m.id.replace('_c','_r')
                m2.compartment = 'r'
                model.add_metabolites([m2])
            r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})

In [110]:
for r in sorted(model.metabolites.get_by_id('ocdcya_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
print()
for r in sorted(model.metabolites.get_by_id('ocdcya_e').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
print()
for r in sorted(model.metabolites.get_by_id('lnlc_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)

FA182ACPH h2o_c + ocdcyaACP_c <=> ACP_c + h_c + ocdcya_c
FACOAL182 atp_c + coa_c + ocdcya_c --> amp_c + ocdycacoa_c + ppi_c
PLBPC_SC h2o_c + 0.005 pc_SC_c --> 0.02 dca_c + 0.06 ddca_c + 0.5 g3pc_c + h_c + 0.27 hdca_c + 0.17 hdcea_c + 0.05 ocdca_c + 0.24 ocdcea_c + 0.09 ocdcya_c + 0.1 ttdca_c
TAGL_SC h2o_c + 0.01 triglyc_SC_c --> 0.01 12dgr_SC_c + 0.02 dca_c + 0.06 ddca_c + h_c + 0.27 hdca_c + 0.17 hdcea_c + 0.05 ocdca_c + 0.24 ocdcea_c + 0.09 ocdcya_c + 0.1 ttdca_c

PLBP1I_SCe h2o_e + 0.005 ptd1ino_SC_e --> 0.02 dca_e + 0.06 ddca_e + 0.5 g3pi_e + h_e + 0.27 hdca_e + 0.17 hdcea_e + 0.05 ocdca_e + 0.24 ocdcea_e + 0.09 ocdcya_e + 0.1 ttdca_e
PLBPC_SCe h2o_e + 0.005 pc_SC_e --> 0.02 dca_e + 0.06 ddca_e + 0.5 g3pc_e + h_e + 0.27 hdca_e + 0.17 hdcea_e + 0.05 ocdca_e + 0.24 ocdcea_e + 0.09 ocdcya_e + 0.1 ttdca_e

FACOAE1829Z12Z h2o_c + lnlccoa_c --> coa_c + h_c + lnlc_c
FACOAL1821 atp_c + coa_c + lnlc_c --> amp_c + lnlccoa_c + ppi_c
LPLPS1AGPE1829Z12Z 1agpe1829Z12Z_c + h2o_c --> g3pe_c + h_c 

In [111]:
# replace lnlc_c with ocdcya_c and remove redundant reactions
for r in model.metabolites.get_by_id('lnlc_c').reactions:
    r.add_metabolites({'lnlc_c': -r.get_coefficient('lnlc_c'), 'ocdcya_c': r.get_coefficient('lnlc_c')})
model.remove_reactions(['FACOAL1821'], remove_orphans=True)

In [112]:
for r in sorted(model.metabolites.get_by_id('ocdycacoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('ocdycacoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('ocdycacoa_r').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
print()
for r in sorted(model.metabolites.get_by_id('lnlccoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('lnlccoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('lnlccoa_r').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('lnlccrn_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('lnlccrn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)

AGAT_SC 0.01 1ag3p_SC_c + 0.02 dcacoa_c + 0.06 ddcacoa_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c --> coa_c + 0.01 pa_SC_c
FA182COAabcp atp_x + h2o_x + ocdycacoa_c --> adp_x + h_x + ocdycacoa_x + pi_x
FACOAL182 atp_c + coa_c + ocdcya_c --> amp_c + ocdycacoa_c + ppi_c
GAT1_SC 0.02 dcacoa_c + 0.06 ddcacoa_c + glyc3p_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c --> 0.01 1ag3p_SC_c + coa_c
GAT2_SC 0.02 dcacoa_c + 0.06 ddcacoa_c + dhap_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c --> 0.01 1agly3p_SC_c + coa_c
LPCAT_SC 0.01 1agpc_SC_c + 0.02 dcacoa_c + 0.06 ddcacoa_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c --> coa_c + 0.01 pc_SC_c
TRIGS_SC 0.01 12dgr_SC_c + 0.02 dcacoa_c + 0.06 ddcacoa_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c -

In [113]:
# replace lnlccoa_c with ocdycacoa_c and remove mito reactions C18 long
for r in model.metabolites.get_by_id('lnlccoa_c').reactions:
    r.add_metabolites({'lnlccoa_c': -r.get_coefficient('lnlccoa_c'), 'ocdycacoa_c': r.get_coefficient('lnlccoa_c')})
for r in model.metabolites.get_by_id('lnlccoa_r').reactions:
    r.add_metabolites({'lnlccoa_r': -r.get_coefficient('lnlccoa_r'), 'ocdycacoa_r': r.get_coefficient('lnlccoa_r')})
model.remove_reactions(['LNLCCPT1','LNLCCPT2','LNLCCRNt'], remove_orphans=True)

In [114]:
# lneldc_c is all trans C18:2, but reaction says 6Z,9Z -> remove lneldc reactions 
for r in sorted(model.metabolites.get_by_id('lneldc_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('lneldccoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('lneldccoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('lneldccrn_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('lneldccrn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)

FACOAL1822 atp_c + coa_c + lneldc_c --> amp_c + lneldccoa_c + ppi_c
FACOAL1822 atp_c + coa_c + lneldc_c --> amp_c + lneldccoa_c + ppi_c
LNELDCCPT1 crn_c + lneldccoa_c --> coa_c + lneldccrn_c
LNELDCCPT2 coa_m + lneldccrn_m --> crn_m + lneldccoa_m
LNELDCCPT1 crn_c + lneldccoa_c --> coa_c + lneldccrn_c
LNELDCCRNt lneldccrn_c --> lneldccrn_m
LNELDCCPT2 coa_m + lneldccrn_m --> crn_m + lneldccoa_m
LNELDCCRNt lneldccrn_c --> lneldccrn_m


In [115]:
model.remove_reactions(['FACOAL1822','LNELDCCPT1','LNELDCCPT2','LNELDCCRNt'], remove_orphans=True)

DESAT18_6 h_c + nadh_c + o2_c + odecoa_c --> 2.0 h2o_c + lneldccoa_c + nad_c 9578  
DESAT18_7 h_c + nadh_c + o2_c + vacccoa_c --> 2.0 h2o_c + lneldccoa_c + nad_c 9578  
DESAT18_8 h_c + nadh_c + o2_c + od2coa_c --> 2.0 h2o_c + lneldccoa_c + nad_c 9578  

DESAT18_10 h_c + lnlncacoa_c + nadh_c + o2_c --> 2.0 h2o_c + nad_c + strdnccoa_c 9578  
DESAT20_1 dlnlcgcoa_c + h_c + nadh_c + o2_c --> arachdcoa_c + 2.0 h2o_c + nad_c 9578  
DESAT20_2 eicostetcoa_c + h_c + nadh_c + o2_c --> 2.0 h2o_c + nad_c + tmndnccoa_c 9578  
DESAT24_1 h_c + nadh_c + o2_c + tetpent3coa_c --> 2.0 h2o_c + nad_c + tethex3coa_c 9578

In [116]:
for r in sorted(model.metabolites.get_by_id('vacccoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('vacccoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('vacccrn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
print()
for r in sorted(model.reactions, key=lambda x: x.id):
    if '18111Z' in r.id:
        print(r.id, r.reaction)

ACOADAGAT16018111Z18111Z 12dgr16018111Z_c + vacccoa_c --> coa_c + tag16018111Z18111Z_c
ACOADAGAT1601819Z18111Z 12dgr1601819Z_c + vacccoa_c --> coa_c + tag1601819Z18111Z_c
ACOADAGAT1801819Z18111Z 12dgr1801819Z_c + vacccoa_c --> coa_c + tag1801819Z18111Z_c
ACOADAGAT18111Z18111Z18111Z 12dgr18111Z18111Z_c + vacccoa_c --> coa_c + tag18111Z18111Z18111Z_c
ACOADAGAT18111Z1819Z18111Z 12dgr18111Z1819Z_c + vacccoa_c --> coa_c + tag18111Z1819Z18111Z_c
ACOADAGAT1819Z18111Z18111Z 12dgr1819Z18111Z_c + vacccoa_c --> coa_c + tag1819Z18111Z18111Z_c
ACOADAGAT1819Z1819Z18111Z 12dgr1819Z1819Z_c + vacccoa_c --> coa_c + tag1819Z1819Z18111Z_c
AGPATCOA16018111Z 1hdecg3p_c + vacccoa_c --> coa_c + pa16018111Z_c
AGPATCOA18111Z18111Z 1odec11eg3p_c + vacccoa_c --> coa_c + pa18111Z18111Z_c
AGPATCOA1819Z18111Z 1odec9eg3p_c + vacccoa_c --> coa_c + pa1819Z18111Z_c
FACOAL1812 atp_c + coa_c + vacc_c --> amp_c + ppi_c + vacccoa_c
VACCCPT1 crn_c + vacccoa_c --> coa_c + vacccrn_c
VACCCPT2 coa_m + vacccrn_m --> crn_m + vaccc

In [117]:
# Remove vacccoa - 18:1 11Z produced by bacteria and plants - no enzyme in Rhodo
model.remove_reactions(['FACOAL1812','VACCCPT1','VACCCPT2','VACCCRNt'], remove_orphans=True)
# Remove 18111Z fatty acid reactions
temp = [r.id for r in sorted(model.reactions, key=lambda x: x.id) if '18111Z' in r.id]
model.remove_reactions(temp, remove_orphans=True)

In [118]:
for r in sorted(model.metabolites.get_by_id('lnlnca_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('lnlncacoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('lnlncacoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('lnlncacrn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
print()
for r in sorted(model.metabolites.get_by_id('strdnc_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('strdnccoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('strdnccoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('strdnccrn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
print()
for r in sorted(model.metabolites.get_by_id('dlnlcg_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('dlnlcgcoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('dlnlcgcoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('dlnlcgcrn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
print()
for r in sorted(model.metabolites.get_by_id('arachd_e').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('arachd_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('arachd_r').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('arachdcoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('arachdcoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('arachdcoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('arachdcrn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
print()
for r in sorted(model.metabolites.get_by_id('adrn_e').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('adrn_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('adrncoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('adrncoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('adrncrn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)

FACOAE1839Z12Z15Z h2o_c + lnlncacoa_c --> coa_c + h_c + lnlnca_c
FACOAL1832 atp_c + coa_c + lnlnca_c --> amp_c + lnlncacoa_c + ppi_c
FACOAE1839Z12Z15Z h2o_c + lnlncacoa_c --> coa_c + h_c + lnlnca_c
FACOAL1832 atp_c + coa_c + lnlnca_c --> amp_c + lnlncacoa_c + ppi_c
LNLNCACPT1 crn_c + lnlncacoa_c --> coa_c + lnlncacrn_c
LNLNCACPT2 coa_m + lnlncacrn_m --> crn_m + lnlncacoa_m
LNLNCACPT2 coa_m + lnlncacrn_m --> crn_m + lnlncacoa_m
LNLNCACRNt lnlncacrn_c --> lnlncacrn_m

FACOAL184 atp_c + coa_c + strdnc_c --> amp_c + ppi_c + strdnccoa_c
FACOAL184 atp_c + coa_c + strdnc_c --> amp_c + ppi_c + strdnccoa_c
STRDNCCPT1 crn_c + strdnccoa_c --> coa_c + strdnccrn_c
STRDNCCPT2 coa_m + strdnccrn_m --> crn_m + strdnccoa_m
STRDNCCPT2 coa_m + strdnccrn_m --> crn_m + strdnccoa_m
STRDNCCRNt strdnccrn_c --> strdnccrn_m

FACOAL203 atp_c + coa_c + dlnlcg_c --> amp_c + dlnlcgcoa_c + ppi_c
DLNLCGCPT1 crn_c + dlnlcgcoa_c --> coa_c + dlnlcgcrn_c
FACOAL203 atp_c + coa_c + dlnlcg_c --> amp_c + dlnlcgcoa_c + ppi_c
D

In [119]:
model.remove_reactions(['FACOAE1839Z12Z15Z','FACOAL1832','LNLNCACPT1','LNLNCACPT2','LNLNCACRNt',
                        'FACOAL184','STRDNCCPT1','STRDNCCPT2','STRDNCCRNt',
                        'FACOAL203','DLNLCGCPT1','DLNLCGCPT2','DLNLCGCRNt',
                        'FATP5t','FACOAL204','P4504B1r','C204CPT1','C204CPT2',
                        'FAOXC2242046x','FATP6t','FACOAL224','ADRNCPT1','ADRNCPT2','ADRNCRNt'], remove_orphans=True)

In [120]:
for r in sorted(model.metabolites.get_by_id('eicostet_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('eicostetcoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('eicostetcoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('eicostetcrn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
print()
for r in sorted(model.metabolites.get_by_id('tmndnc_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('tmndnccoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('tmndnccoa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('tmndnccrn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
print()
for r in sorted(model.metabolites.get_by_id('tetpent3_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('tetpent3coa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('tetpent3coa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('tetpent3crn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
print()
for r in sorted(model.metabolites.get_by_id('tetpent6_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('tetpent6coa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('tetpent6coa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('tetpent6crn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
print()
for r in sorted(model.metabolites.get_by_id('tethex3_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('tethex3coa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('tethex3coa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
print()
for r in sorted(model.metabolites.get_by_id('crvnc_e').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('crvnc_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('c226coa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('c226coa_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('c226coa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)
for r in sorted(model.metabolites.get_by_id('c226crn_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction)

FACOAL2042 atp_c + coa_c + eicostet_c --> amp_c + eicostetcoa_c + ppi_c
EICOSTETCPT1 crn_c + eicostetcoa_c --> coa_c + eicostetcrn_c
FACOAL2042 atp_c + coa_c + eicostet_c --> amp_c + eicostetcoa_c + ppi_c
EICOSTETCPT2 coa_m + eicostetcrn_m --> crn_m + eicostetcoa_m
EICOSTETCPT2 coa_m + eicostetcrn_m --> crn_m + eicostetcoa_m
EICOSTETCRNt eicostetcrn_c --> eicostetcrn_m

FACOAL205 atp_c + coa_c + tmndnc_c --> amp_c + ppi_c + tmndnccoa_c
FACOAL205 atp_c + coa_c + tmndnc_c --> amp_c + ppi_c + tmndnccoa_c
TMNDNCCPT1 crn_c + tmndnccoa_c --> coa_c + tmndnccrn_c
TMNDNCCPT2 coa_m + tmndnccrn_m --> crn_m + tmndnccoa_m
TMNDNCCPT2 coa_m + tmndnccrn_m --> crn_m + tmndnccoa_m
TMNDNCCRNt tmndnccrn_c --> tmndnccrn_m

FACOAL245_2 atp_c + coa_c + tetpent3_c --> amp_c + ppi_c + tetpent3coa_c
FACOAL245_2 atp_c + coa_c + tetpent3_c --> amp_c + ppi_c + tetpent3coa_c
TETPENT3CPT1 crn_c + tetpent3coa_c --> coa_c + tetpent3crn_c
TETPENT3CPT2 coa_m + tetpent3crn_m --> crn_m + tetpent3coa_m
TETPENT3CPT2 coa_m +

In [121]:
model.remove_reactions(['FACOAL2042','EICOSTETCPT1','EICOSTETCPT2','EICOSTETCRNt',
                        'FACOAL205','TMNDNCCPT1','TMNDNCCPT2','TMNDNCCRNt',
                        'FACOAL245_2','TETPENT3CPT1','TETPENT3CPT2','TETPENT3CRNt',
                        'FACOAL245_1','TETPENT6CPT1','TETPENT6CPT2','TETPENT6CRNt',
                        'FACOAL246_1','FAOXC246226x','FATP7t','FACOAL226','C226CPT1','C226CPT2'], remove_orphans=True)

In [122]:
for r in sorted(model.genes.get_by_id('9730').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8845').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9578').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

DESAT1619Zer h_r + nadh_r + o2_r + pmtcoa_r --> 2.0 h2o_r + hdcoa_r + nad_r 9730
DESAT1819Zer h_r + nadh_r + o2_r + stcoa_r --> 2.0 h2o_r + nad_r + odecoa_r 9730

DESAT1829Z12Zer h_r + nadh_r + o2_r + odecoa_r --> 2.0 h2o_r + nad_r + ocdycacoa_r 10123 and 16097 and 8845

DESAT1836Z9Z12Zer h_r + nadh_r + o2_r + ocdycacoa_r --> 2.0 h2o_r + lnlncgcoa_r + nad_r 9578


9629	plas 21, mito 2, E.R. 2	K04712: DEGS; sphingolipid Delta-4 desaturase, no cytochrome b5-like domain, generates a trans double bond at position 4 of sphinganine bases in sphingolipids, requires an external cytochrome b5	KDE*  
9578	plas 22, mito 3	K13076: SLD; Delta 6-fatty acid desaturase/delta-8 sphingolipid desaturase, has cytochrome b5-like domain, introduces a double bond at Delta 6 of fatty acids, introduces a trans double bond at the 8-position of sphingoid bases in sphingolipids	LHH*  
10197	cyto 11.5, cyto_nucl 7.5, mito 4, extr 4, E.R. 3, nucl 2.5	K20238: E2.1.1.317; sphingolipid C9-methyltransferase	LVV*  
15324	plas 17, extr 6, vacu 2	K00720: UGCG; ceramide glucosyltransferase	EPR*  

Sphingolipid  
9629 dihydroceramide -> 4E-sphingosine ceramide  
9578 4E-sphingosine ceramide -> 4E,8E-sphingadienine ceramide  
10197 4E,8E-sphingadienine ceramide -> 9-methyl-4E,8E-sphingadienine ceramide  
15324 9-methyl-4E,8E-sphingadienine ceramide -> glucosylceramide
https://www.ncbi.nlm.nih.gov/pubmed/20019081

Ceramide transport from ER to golgi via vesicle (COPII) or CERT-like Nvj2 (14324)
https://www.ncbi.nlm.nih.gov/pubmed/11733544
https://www.ncbi.nlm.nih.gov/pubmed/28011845

In [123]:
for r in sorted(model.genes.get_by_id('9629').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9578').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('10197').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15324').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

DHCRD1 dhcrm_hs_c + nadp_c --> crm_hs_c + h_c + nadph_c 9629
DHCRD2 dhcrm_hs_c + fad_c --> crm_hs_c + fadh2_c 9629
yli_R0706 0.01 pail_cho_c + yli_M05741_c --> 0.01 dag_hs_c + 0.01 yli_M04599_c 9629
yli_R0707 0.01 pail_cho_c + yli_M05742_c --> 0.01 dag_hs_c + 0.01 yli_M04600_c 9629
yli_R0708 0.01 pail_cho_c + yli_M05748_c --> 0.01 dag_hs_c + 0.01 yli_M04601_c 9629
yli_R0709 0.01 pail_cho_c + yli_M05749_c --> 0.01 dag_hs_c + 0.01 yli_M04602_c 9629
yli_R0710 0.01 pail_cho_c + yli_M05724_c --> 0.01 dag_hs_c + 0.01 yli_M04603_c 9629
yli_R0711 0.01 pail_cho_c + yli_M05725_c --> 0.01 dag_hs_c + 0.01 yli_M04604_c 9629

DESAT1836Z9Z12Zer h_r + nadh_r + o2_r + ocdycacoa_r --> 2.0 h2o_r + lnlncgcoa_r + nad_r 9578

CFAS160E 2.0 amet_c + pe161_c --> 2.0 ahcys_c + cpe160_c + 2.0 h_c 10197
CFAS160G 2.0 amet_c + pg161_c --> 2.0 ahcys_c + cpg160_c + 2.0 h_c 10197
CFAS180E 2.0 amet_c + pe181_c --> 2.0 ahcys_c + cpe180_c + 2.0 h_c 10197
CFAS180G 2.0 amet_c + pg181_c --> 2.0 ahcys_c + cpg180_c + 2.0 h_c 

In [124]:
# Remove irrelevant, human general, bacteria/plant specific reactions
# CFAS cyclopropane FA synthase are specific to bacteria/plant
model.remove_reactions(['DHCRD1','DHCRD2','yli_R0706','yli_R0707','yli_R0708','yli_R0709','yli_R0710',
                        'yli_R0711','CFAS160E','CFAS160G','CFAS180E','CFAS180G','UGCG'], remove_orphans=True)
# Need to add reactions for sphingolipid desaturases

In [125]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if 'CERS' in r.id:
        print(r.id, r.reaction, r.compartments, r.subsystem, r.gene_reaction_rule)

CERS124er sphgn_r + ttccoa_r --> cer1_24_r + coa_r + h_r {'r'} S_Sphingolipid_Metabolism 11391 or 15168
CERS126er hexccoa_r + sphgn_r --> cer1_26_r + coa_r + h_r {'r'} S_Sphingolipid_Metabolism 11391 or 15168
CERS224er psphings_r + ttccoa_r --> cer2_24_r + coa_r + h_r {'r'} S_Sphingolipid_Metabolism 11391 or 15168
CERS226er hexccoa_r + psphings_r --> cer2_26_r + coa_r + h_r {'r'} S_Sphingolipid_Metabolism 11391 or 15168
CERS2p24er cer1_24_r + h_r + nadph_r + o2_r --> cer2p_24_r + h2o_r + nadp_r {'r'} S_Sphingolipid_Metabolism 9664
CERS2p26er cer1_26_r + h_r + nadph_r + o2_r --> cer2p_26_r + h2o_r + nadp_r {'r'} S_Sphingolipid_Metabolism 9664
CERS324er cer2_24_r + h_r + nadph_r + o2_r --> cer3_24_r + h2o_r + nadp_r {'r'} S_Sphingolipid_Metabolism 9664
CERS326er cer2_26_r + h_r + nadph_r + o2_r --> cer3_26_r + h2o_r + nadp_r {'r'} S_Sphingolipid_Metabolism 9664


In [126]:
for r in model.genes.get_by_id('15685').reactions:
    print(r)
model.metabolites.get_by_id('34dhbz_p')

ACLSm: h_m + 2.0 pyr_m --> alac__S_m + co2_m
ACHBSm: 2obut_m + h_m + pyr_m --> 2ahbut_m + co2_m


Metabolite identifier,34dhbz_p
Name,"3,4-Dihydroxybenzoate"
Memory address,0x0102f725978
Formula,C7H5O4
Compartment,p
In 1 reaction(s),3_4DHBZt2


In [127]:
# check nadh/nadph since sphingolipid reactions use cytochrome b5
# may need to replace nadph with nadh in all reactions

# CERS118er sphgn_r + stcoa_r --> cer1_18_r + coa_r + h_r
# CERS2p18er cer1_18_r + h_r + nadph_r + o2_r --> cer2p_18_r + h2o_r + nadp_r
r1 = model.reactions.get_by_id('CERS124er').copy()
r2 = model.reactions.get_by_id('CERS2p24er').copy()
r1.id = 'CERS118er'
r2.id = 'CERS2p18er'
r1.name = 'Ceramide 1 synthase  18C   endoplasmic reticular'
r2.name = 'Ceramide 2p synthase  18C'
model.add_reactions([r1,r2])
m1 = model.metabolites.get_by_id('cer1_24_r').copy()
m2 = model.metabolites.get_by_id('cer2p_24_r').copy()
m1.id = 'cer1_18_r'
m2.id = 'cer2p_18_r'
m1.name = 'Ceramide 1 (Sphinganine:n-C18:0, Cer(d18:0/18:0))'
m2.name = 'Ceramide 2p (Sphinganine:n-C18:0OH, Cer(d18:0/18:0(2OH)))'
m1.formula = 'C36H73NO3'
m2.formula = 'C36H73NO4'
model.add_metabolites([m1,m2])
model.reactions.get_by_id('CERS118er').add_metabolites({'ttccoa_r': 1.0, 'stcoa_r': -1.0,
                                                        'cer1_24_r': -1.0, 'cer1_18_r': 1.0})
model.reactions.get_by_id('CERS2p18er').add_metabolites({'cer1_24_r': 1.0, 'cer1_18_r': -1.0,
                                                         'cer2p_24_r': -1.0, 'cer2p_18_r': 1.0})
# CERD418er cer2p_18_r + h_r + nadph_r + o2_r --> cer4_18_r + 2 h2o_r + nadp_r
# CERD518er cer4_18_r + h_r + nadph_r + o2_r --> cer5_18_r + 2 h2o_r + nadp_r
r1 = model.reactions.get_by_id('CERS2p18er').copy()
r2 = model.reactions.get_by_id('CERS2p18er').copy()
r1.id = 'CERD418er'
r2.id = 'CERD518er'
r1.name = 'Sphingolipid 4-desaturase  18C'
r2.name = 'Sphingolipid 8-desaturase  18C'
r1.gene_reaction_rule = '10123 and 16097 and 9629'
r2.gene_reaction_rule = '9578'
model.add_reactions([r1,r2])
m1 = model.metabolites.get_by_id('cer2p_18_r').copy()
m2 = model.metabolites.get_by_id('cer2p_18_r').copy()
m1.id = 'cer4_18_r'
m2.id = 'cer5_18_r'
m1.name = 'Ceramide 4 (Sphingosine:n-C18:0OH, Cer(d18:1(4E)/18:0(2OH)))'
m2.name = 'Ceramide 5 (Sphinga-4,8-dienine:n-C18:0OH, Cer(d18:2(4E,8E)/18:0(2OH)))'
m1.formula = 'C36H71NO4'
m2.formula = 'C36H69NO4'
model.add_metabolites([m1,m2])
model.reactions.get_by_id('CERD418er').add_metabolites({'cer1_18_r': 1.0, 'cer2p_18_r': -2.0,
                                                        'cer4_18_r': 1.0, 'h2o_r': 1.0})
model.reactions.get_by_id('CERD518er').add_metabolites({'cer1_18_r': 1.0, 'cer2p_18_r': -1.0,
                                                        'cer4_18_r': -1.0, 'cer5_18_r': 1.0, 'h2o_r': 1.0})
# CERMT618er cer5_18_r + amet_r --> cer6_18_r + ahcys_r + h_r
# CER618tg cer6_18_r --> cer6_18_g
# CERGT718g cer6_18_g + udpg_g --> glccer_18_g + h_g + udp_g
r1 = cobra.Reaction('CERMT618er')
r2 = cobra.Reaction('CER618trg')
r3 = cobra.Reaction('CERGT18g')
r1.name = 'Sphingolipid C9-methyltransferase  18C'
r2.name = 'Ceramide transport from ER to golgi'
r3.name = 'Ceramide glucosyltransferase  18C'
r1.subsystem = 'S_Sphingolipid_Metabolism'
r2.subsystem = 'S_Sphingolipid_Metabolism'
r3.subsystem = 'S_Sphingolipid_Metabolism'
r1.gene_reaction_rule = '10197'
r2.gene_reaction_rule = '14324 or COPII'
r3.gene_reaction_rule = '15324'
model.add_reactions([r1,r2,r3])
m1 = model.metabolites.get_by_id('cer5_18_r').copy()
m2 = model.metabolites.get_by_id('cer5_18_r').copy()
m3 = model.metabolites.get_by_id('cer5_18_r').copy()
m1.id = 'cer6_18_r'
m2.id = 'cer6_18_g'
m3.id = 'glccer_18_g'
m2.compartment = 'g'
m3.compartment = 'g'
m1.name = 'Ceramide 6 (9-Methyl-sphinga-4,8-dienine:n-C18:0OH, Cer(d18:2(4E,8E)(9Me)/18:0(2OH)))'
m2.name = 'Ceramide 6 (9-Methyl-sphinga-4,8-dienine:n-C18:0OH, Cer(d18:2(4E,8E)(9Me)/18:0(2OH)))'
m3.name = 'Glucosylceramide (Glucosyl 9-methyl-sphinga-4,8-dienine:n-C18:0OH, GlcCer(d18:2(4E,8E)(9Me)/18:0(2OH)))'
m1.formula = 'C37H71NO4'
m2.formula = 'C37H71NO4'
m3.formula = 'C43H81NO9'
model.add_metabolites([m1,m2,m3])
model.reactions.get_by_id('CERMT618er').add_metabolites({'cer5_18_r': -1.0, 'amet_r': -1.0,
                                                         'cer6_18_r': 1.0, 'ahcys_r': 1.0, 'h_r': 1.0})
model.reactions.get_by_id('CER618trg').add_metabolites({'cer6_18_r': -1.0, 'cer6_18_g': 1.0})
model.reactions.get_by_id('CERGT18g').add_metabolites({'cer6_18_g': -1.0, 'udpg_g': -1.0,
                                                        'glccer_18_g': 1.0, 'h_g': 1.0, 'udp_g': 1.0})

In [128]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if 'CER' in r.id:
        print(r.id, r.reaction, r.compartments, r.gene_reaction_rule)

CER618trg cer6_18_r --> cer6_18_g {'g', 'r'} 14324 or COPII
CERD418er cer2p_18_r + h_r + nadph_r + o2_r --> cer4_18_r + 2.0 h2o_r + nadp_r {'r'} 10123 and 16097 and 9629
CERD518er cer4_18_r + h_r + nadph_r + o2_r --> cer5_18_r + 2.0 h2o_r + nadp_r {'r'} 9578
CERGT18g cer6_18_g + udpg_g --> glccer_18_g + h_g + udp_g {'g'} 15324
CERH124er cer1_24_r + h_r + nadph_r + o2_r --> cer2_24_r + h2o_r + nadp_r {'r'} 15314
CERH126er cer1_26_r + h_r + nadph_r + o2_r --> cer2_26_r + h2o_r + nadp_r {'r'} 15314
CERMT618er amet_r + cer5_18_r --> ahcys_r + cer6_18_r + h_r {'r'} 10197
CERS118er sphgn_r + stcoa_r --> cer1_18_r + coa_r + h_r {'r'} 11391 or 15168
CERS124er sphgn_r + ttccoa_r --> cer1_24_r + coa_r + h_r {'r'} 11391 or 15168
CERS126er hexccoa_r + sphgn_r --> cer1_26_r + coa_r + h_r {'r'} 11391 or 15168
CERS224er psphings_r + ttccoa_r --> cer2_24_r + coa_r + h_r {'r'} 11391 or 15168
CERS226er hexccoa_r + psphings_r --> cer2_26_r + coa_r + h_r {'r'} 11391 or 15168
CERS2p18er cer1_18_r + h_r + n

In [129]:
for m in sorted(model.metabolites, key=lambda x: x.id):
    if m.id.startswith('cer'):
        print(m.id, m.name, m.formula, m.formula_weight)
print()
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('CER'):
        print(r.id, r.name, r.gene_reaction_rule)

cer1_18_r Ceramide 1 (Sphinganine:n-C18:0, Cer(d18:0/18:0)) C36H73NO3 567.96972
cer1_24_c Ceramide 1  Sphinganinen C240  C42H85NO3 C42H85NO3 652.1292
cer1_24_r Ceramide 1  Sphinganinen C240  C42H85NO3 C42H85NO3 652.1292
cer1_26_c Ceramide 1  Sphinganinen C260  C44H89NO3 C44H89NO3 680.1823600000001
cer1_26_r Ceramide 1  Sphinganinen C260  C44H89NO3 C44H89NO3 680.1823600000001
cer2_24_c Ceramide 2  Phytosphingosinen C240  C42H85NO4 C42H85NO4 668.1286
cer2_24_r Ceramide 2  Phytosphingosinen C240  C42H85NO4 C42H85NO4 668.1286
cer2_26_c Ceramide 2  Phytosphingosinen C260  C44H89NO4 C44H89NO4 696.1817600000002
cer2_26_r Ceramide 2  Phytosphingosinen C260  C44H89NO4 C44H89NO4 696.1817600000002
cer2p_18_r Ceramide 2p (Sphinganine:n-C18:0OH, Cer(d18:0/18:0(2OH))) C36H73NO4 583.9691200000001
cer2p_24_r Ceramide 2p (Sphinganine:n-C24:0OH) C42H85NO4 668.1286
cer2p_26_r Ceramide 2p (Sphinganine:n-C26:0OH) C44H89NO4 696.1817600000002
cer3_24_c Ceramide 3  Phytosphingosinen C240OH  C42H85NO5 C42H85NO

In [130]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if '124' in r.id:
        print(r.id, r.reaction, r.gene_reaction_rule)

CERH124er cer1_24_r + h_r + nadph_r + o2_r --> cer2_24_r + h2o_r + nadp_r 15314
CERS124er sphgn_r + ttccoa_r --> cer1_24_r + coa_r + h_r 11391 or 15168
IPC124PLC_SC h2o_c + 0.01 ipc124_SC_c --> cer1_24_c + h_c + mi1p__D_c 15857
IPCS124_SC cer1_24_c + 0.01 ptd1ino_SC_c --> 0.01 12dgr_SC_c + 0.01 ipc124_SC_c 12927
MIP2C124PLC_SC h2o_c + 0.01 mip2c124_SC_c --> cer1_24_c + h_c + man2mi1p__D_c 15857
MIP2CS124_SC 0.01 mipc124_SC_c + 0.01 ptd1ino_SC_c --> 0.01 12dgr_SC_c + 0.01 mip2c124_SC_c 13172
MIPC124PLC_SC h2o_c + 0.01 mipc124_SC_c --> cer1_24_c + h_c + manmi1p__D_c 15857
MIPCS124_SC gdpmann_c + 0.01 ipc124_SC_c --> gdp_c + h_c + 0.01 mipc124_SC_c YBR036C and 16453


Inositol phosphorylceramide synthase needs two subunits, Aur1 and Kei1 (12927 and 8747)
12927	plas 22, mito 2, vacu 2	Aur1, HMMPfam:PAP2 superfamily:PF01569,SMART:Acid phosphatase homologues:SM00014,SUPERFAMILY::SSF48317	RRD*  
8747	mito 7, extr 5, nucl 4, cyto_mito 4, plas 3, pero 3	Kei1, HMMPfam:Inositolphosphorylceramide synthase subunit Kei1:PF08552	TPR*  
16453	extr 13, plas 12	CSH1/SUR1,  HMMPfam:Glycosyltransferase sugar-binding region containing DXD motif:PF04488,SUPERFAMILY::SSF53448	ESP*  
13172	extr 10, mito 7, plas 4, cyto_mito 4	Ipt1, HMMPfam:PAP2 superfamily:PF01569	SLA*  
15857	plas 14, mito 4, extr 4, cyto 3	Isc1, K12351: SMPD2; sphingomyelin phosphodiesterase 2	WTR*  

Change IPCSxxx_SC genes to '12927 and 8747'  
Cannot find YBR036C (CSG2 regulatory subunit), change MIPCSxxx_SC genes to '16453'  
Change compartment from cyto to golgi  

Change IPCxxx_SC / MIPCxxx_SC to xxxg_RT

In [131]:
for r in sorted(model.genes.get_by_id('12927').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('16453').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13172').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15857').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

IPCS124_SC cer1_24_c + 0.01 ptd1ino_SC_c --> 0.01 12dgr_SC_c + 0.01 ipc124_SC_c 12927
IPCS126_SC cer1_26_c + 0.01 ptd1ino_SC_c --> 0.01 12dgr_SC_c + 0.01 ipc126_SC_c 12927
IPCS224_SC cer2_24_c + 0.01 ptd1ino_SC_c --> 0.01 12dgr_SC_c + 0.01 ipc224_SC_c 12927
IPCS226_SC cer2_26_c + 0.01 ptd1ino_SC_c --> 0.01 12dgr_SC_c + 0.01 ipc226_SC_c 12927
IPCS324_SC cer3_24_c + 0.01 ptd1ino_SC_c --> 0.01 12dgr_SC_c + 0.01 ipc324_SC_c 12927
IPCS326_SC cer3_26_c + 0.01 ptd1ino_SC_c --> 0.01 12dgr_SC_c + 0.01 ipc326_SC_c 12927
yli_R0718 0.01 pail_cho_c + 0.01 yli_M04615_c --> 0.01 dag_hs_c + 0.01 yli_M04609_c 12927
yli_R0719 0.01 pail_cho_c + 0.01 yli_M04616_c --> 0.01 dag_hs_c + 0.01 yli_M04610_c 12927
yli_R0720 0.01 pail_cho_c + 0.01 yli_M04617_c --> 0.01 dag_hs_c + 0.01 yli_M04611_c 12927
yli_R0721 0.01 pail_cho_c + 0.01 yli_M04618_c --> 0.01 dag_hs_c + 0.01 yli_M04612_c 12927
yli_R0722 0.01 pail_cho_c + 0.01 yli_M04619_c --> 0.01 dag_hs_c + 0.01 yli_M04613_c 12927
yli_R0723 0.01 pail_cho_c + 0.01 y

In [132]:
# Remove redundant yli reactions
model.remove_reactions(['yli_R0718','yli_R0719','yli_R0720','yli_R0721','yli_R0722','yli_R0723'], remove_orphans=True)

In [133]:
for g in ['12927','16453','13172','15857']:
    for r in model.genes.get_by_id(g).reactions:
        r.id = r.id.replace('_SC','g_RT')
        if g == '12927':
            r.gene_reaction_rule = '12927 and 8747'
        if g == '16453':
            r.gene_reaction_rule = '16453'
        for m in r.metabolites:
            if not m.id.replace('_c','_g') in model.metabolites:
                m2 = m.copy()
                m2.id = m.id.replace('_c','_g')
                m2.compartment = 'g'
                model.add_metabolites([m2])
            r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_g'): r.get_coefficient(m.id)})

In [134]:
r = model.reactions.get_by_id('CER618trg').copy()
r.id = 'CER124trg'
model.add_reactions([r])
r.add_metabolites({'cer6_18_r': 1.0, 'cer6_18_g': -1.0, 'cer1_24_r': -1.0, 'cer1_24_g': 1.0})
r = model.reactions.get_by_id('CER618trg').copy()
r.id = 'CER126trg'
model.add_reactions([r])
r.add_metabolites({'cer6_18_r': 1.0, 'cer6_18_g': -1.0, 'cer1_26_r': -1.0, 'cer1_26_g': 1.0})
r = model.reactions.get_by_id('CER618trg').copy()
r.id = 'CER224trg'
model.add_reactions([r])
r.add_metabolites({'cer6_18_r': 1.0, 'cer6_18_g': -1.0, 'cer2_24_r': -1.0, 'cer2_24_g': 1.0})
r = model.reactions.get_by_id('CER618trg').copy()
r.id = 'CER226trg'
model.add_reactions([r])
r.add_metabolites({'cer6_18_r': 1.0, 'cer6_18_g': -1.0, 'cer2_26_r': -1.0, 'cer2_26_g': 1.0})
r = model.reactions.get_by_id('CER618trg').copy()
r.id = 'CER324trg'
model.add_reactions([r])
r.add_metabolites({'cer6_18_r': 1.0, 'cer6_18_g': -1.0, 'cer3_24_r': -1.0, 'cer3_24_g': 1.0})
r = model.reactions.get_by_id('CER618trg').copy()
r.id = 'CER326trg'
model.add_reactions([r])
r.add_metabolites({'cer6_18_r': 1.0, 'cer6_18_g': -1.0, 'cer3_26_r': -1.0, 'cer3_26_g': 1.0})

In [135]:
for r in sorted(model.genes.get_by_id('12927').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('16453').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13172').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15857').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

IPCS124g_RT cer1_24_g + 0.01 ptd1ino_SC_g --> 0.01 12dgr_SC_g + 0.01 ipc124_SC_g 12927 and 8747
IPCS126g_RT cer1_26_g + 0.01 ptd1ino_SC_g --> 0.01 12dgr_SC_g + 0.01 ipc126_SC_g 12927 and 8747
IPCS224g_RT cer2_24_g + 0.01 ptd1ino_SC_g --> 0.01 12dgr_SC_g + 0.01 ipc224_SC_g 12927 and 8747
IPCS226g_RT cer2_26_g + 0.01 ptd1ino_SC_g --> 0.01 12dgr_SC_g + 0.01 ipc226_SC_g 12927 and 8747
IPCS324g_RT cer3_24_g + 0.01 ptd1ino_SC_g --> 0.01 12dgr_SC_g + 0.01 ipc324_SC_g 12927 and 8747
IPCS326g_RT cer3_26_g + 0.01 ptd1ino_SC_g --> 0.01 12dgr_SC_g + 0.01 ipc326_SC_g 12927 and 8747

MIPCS124g_RT gdpmann_g + 0.01 ipc124_SC_g --> gdp_g + h_g + 0.01 mipc124_SC_g 16453
MIPCS126g_RT gdpmann_g + 0.01 ipc126_SC_g --> gdp_g + h_g + 0.01 mipc126_SC_g 16453
MIPCS224g_RT gdpmann_g + 0.01 ipc224_SC_g --> gdp_g + h_g + 0.01 mipc224_SC_g 16453
MIPCS226g_RT gdpmann_g + 0.01 ipc226_SC_g --> gdp_g + h_g + 0.01 mipc226_SC_g 16453
MIPCS324g_RT gdpmann_g + 0.01 ipc324_SC_g --> gdp_g + h_g + 0.01 mipc324_SC_g 16453
MIP

cytchrome b5 reductase and cyt b5 10123 and 16097  
10123	cyto 14.5, cyto_nucl 9.5, mito 5, nucl 3.5, extr 3	IRC21, 22% cov of CYB5R4, K00326: E1.6.2.2; cytochrome-b5 reductase	VPE*  
13206	extr 8, cyto 7, E.R. 3, nucl 2, mito 2, plas 2, golg 2, mito_nucl 2	CBR1, K00326: E1.6.2.2; cytochrome-b5 reductase	FCF*  
12471	extr 11, mito 9, cyto 4, nucl 1, pero 1, E.R. 1	MCR1, K00326: E1.6.2.2; cytochrome-b5 reductase	VKL*, no rxn  
15098	mito 24, cyto 3	MCR1, K00326: E1.6.2.2; cytochrome-b5 reductase	FRF*  

16097	cyto 23.5, cyto_nucl 13	CYB5, KOG0537: Cytochrome b5	VAD  
15652	cyto 11, cyto_nucl 10.333, cyto_mito 9.333, nucl 8.5, mito 6.5	KOG0537: Cytochrome b5	GSL*, no rxn  
15808	mito 19, pero 4, cyto 2.5, cyto_nucl 2.5	20% cov of LDH, KOG0537: Cytochrome b5	SKL*, no rxn  

16097 wrong gene model / different isoform -> 960651, ~100% cov of cyt b5  
15652 low cov ~43% of cyt b5  
15808 wrong gene model / different isoform -> 742088, still low cov ~60% of cyt b5  

For ER/golgi, (12471 or 13206) and 16097

In [136]:
for r in sorted(model.genes.get_by_id('10123').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13206').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15098').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

CERD418er cer2p_18_r + h_r + nadph_r + o2_r --> cer4_18_r + 2.0 h2o_r + nadp_r 10123 and 16097 and 9629
COA1819ZD9DS 2.0 focytb5_c + 2.0 h_c + o2_c + stcoa_c --> 2.0 ficytb5_c + 2.0 h2o_c + odecoa_c 10123 or 16097
CY_focytb5_c ficytb5_c <=> focytb5_c 10123 or 16097
DESAT1829Z12Zer h_r + nadh_r + o2_r + odecoa_r --> 2.0 h2o_r + nad_r + ocdycacoa_r 10123 and 16097 and 8845

C22STDSx ergtrol_c + h_c + nadh_c + o2_c --> ergtetrol_c + 2.0 h2o_c + nad_c (11716 and 13206 and 16097) or (11716 and 15098 and 16097)
LNS14DMx 2.0 h_c + lanost_c + 3.0 nadh_c + 3.0 o2_c --> 44mctr_c + for_c + 4.0 h2o_c + 3.0 nad_c (12843 and 13206 and 16097) or (12843 and 15098 and 16097)
SQLErx h_r + nadh_r + o2_r + sql_r --> Ssq23epx_r + h2o_r + nad_r (13206 and 13729 and 16097) or (13729 and 15098 and 16097)

C22STDSx ergtrol_c + h_c + nadh_c + o2_c --> ergtetrol_c + 2.0 h2o_c + nad_c (11716 and 13206 and 16097) or (11716 and 15098 and 16097)
LNS14DMx 2.0 h_c + lanost_c + 3.0 nadh_c + 3.0 o2_c --> 44mctr_c + for_

In [137]:
model.reactions.get_by_id('DESAT1829Z12Zer').gene_reaction_rule = '(12471 and 16097 and 8845) or (13206 and 16097 and 8845)'
model.reactions.get_by_id('CERD418er').gene_reaction_rule = '(12471 and 16097 and 9629) or (13206 and 16097 and 9629)'
model.reactions.get_by_id('C22STDSx').gene_reaction_rule = '(11716 and 12471 and 16097) or (11716 and 13206 and 16097)'
model.reactions.get_by_id('LNS14DMx').gene_reaction_rule = '(12843 and 12471 and 16097) or (12843 and 13206 and 16097)'
model.reactions.get_by_id('SQLErx').gene_reaction_rule = '(13729 and 12471 and 16097) or (13729 and 13206 and 16097)'

In [138]:
for x in ['C22STDSx','LNS14DMx']:
    r = model.reactions.get_by_id(x)
    r.id = r.id.replace('x','rx')
    for m in r.metabolites:
        if not m.id.replace('_c','_r') in model.metabolites:
            m2 = m.copy()
            m2.id = m.id.replace('_c','_r')
            m2.compartment = 'r'
            model.add_metabolites([m2])
        r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})

In [139]:
for r in sorted(model.metabolites.get_by_id('ficytb5_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

COA1819ZD9DS 2.0 focytb5_c + 2.0 h_c + o2_c + stcoa_c --> 2.0 ficytb5_c + 2.0 h2o_c + odecoa_c 10123 or 16097
CY_focytb5_c ficytb5_c <=> focytb5_c 10123 or 16097


In [140]:
model.remove_reactions(['COA1819ZD9DS','CY_focytb5_c'], remove_orphans=True)

non-x reactions use NADPH, needs 13286 instead of cyt b5 and reductase  
13286	extr 12, E.R. 8, cyto 2, vacu 2, plas 1, cyto_nucl 1, pero 1, golg 1, cyto_mito 1	K00327: POR; NADPH-ferrihemoprotein reductase	VWS*

In [141]:
for r in sorted(model.genes.get_by_id('16591').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13729').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9240').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12843').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8671').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('16640').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13724').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8835').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

PSPPS 2.0 frdp_c --> ppi_c + psqldp_c 16591
SQLS 2.0 frdp_c + h_c + nadph_c --> nadp_c + 2.0 ppi_c + sql_c 16591
SQLSr 2.0 frdp_r + h_r + nadph_r --> nadp_r + 2.0 ppi_r + sql_r 16591
SS h_c + nadph_c + psqldp_c --> nadp_c + ppi_c + sql_c 16591
yli_R0755 2.0 frdp_c --> nadp_c + ppi_c + psqldp_c 16591
yli_R1451 2.0 frdp_r --> nadp_r + ppi_r + psqldp_r 16591
yli_R1452 h_r + nadph_r + psqldp_r --> nadp_r + ppi_r + sql_r 16591

SMO h_c + nadph_c + o2_c + sql_c --> Ssq23epx_c + h2o_c + nadp_c 13729
SQLEr h_r + nadph_r + o2_r + sql_r --> Ssq23epx_r + h2o_r + nadp_r 13729 or (13286 and 13729)
SQLErx h_r + nadh_r + o2_r + sql_r --> Ssq23epx_r + h2o_r + nad_r (13729 and 12471 and 16097) or (13729 and 13206 and 16097)

CAS Ssq23epx_c <=> cyart_c 9240
LNSTLS Ssq23epx_c --> lanost_c 9240
LNSTLSr Ssq23epx_r --> lanost_r 9240

LNS14DM 2.0 h_c + lanost_c + 3.0 nadph_c + 3.0 o2_c --> 44mctr_c + for_c + 4.0 h2o_c + 3.0 nadp_c 12843 or (12843 and 13286)
LNS14DMr 2.0 h_r + lanost_r + 3.0 nadph_r + 3.0 o2_

In [142]:
for r in sorted(model.genes.get_by_id('13060').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('10113').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8781').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('11716').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('10981').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12791').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13559').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

SAM24MT amet_c + zymst_c --> ahcys_c + fecost_c + h_c 13060
yli_R1449 amet_r + zymst_r --> ahcys_r + fecost_r + h_r 13060

C8STI fecost_c --> epist_c 10113
yli_R1454 fecost_r --> epist_r 10113

C5STDS epist_c + h_c + nadph_c + o2_c --> ergtrol_c + 2.0 h2o_c + nadp_c 8781
LSTO1r chlstol_r + h_r + nadph_r + o2_r --> ddsmsterol_r + 2.0 h2o_r + nadp_r 8781
LSTO2r h_r + lthstrl_r + nadph_r + o2_r --> 7dhchsterol_r + 2.0 h2o_r + nadp_r 8781
LTHSTRLOR h_c + lthstrl_c + nadph_c + o2_c --> 7dhchsterol_c + 2.0 h2o_c + nadp_c 8781
yli_R0758 epist_c + nadp_c --> h_c + nadph_c + yli_M03873_c 8781
yli_R1455 epist_r + nadp_r --> h_r + nadph_r + yli_M03873_r 8781

C22STDS ergtrol_c + h_c + nadph_c + o2_c --> ergtetrol_c + 2.0 h2o_c + nadp_c 11716 and 13286
C22STDSrx ergtrol_r + h_r + nadh_r + o2_r --> ergtetrol_r + 2.0 h2o_r + nad_r (11716 and 12471 and 16097) or (11716 and 13206 and 16097)
yli_R0759 nadp_c + yli_M03873_c --> ergtetrol_c + h_c + nadph_c 11716
yli_R1456 nadp_r + yli_M03873_r --> ergtet

In [143]:
# Remove cyto/redundant sterol reactions
# ERG27 is missing - potential function of succinate semialdehyde dehydrogenase in ERG27-missing organisms 
# https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2817430/
Remove = ['yli_R0755','PSPPS','SS','SQLS','yli_R1451','yli_R1452','SMO','CAS','LNSTLS','yli_R0744','yli_R0748',
          'yli_R0746','yli_R0742','yli_R1444','yli_R1447','yli_R1445','yli_R1442','LNS14DM','C14STR','C4STMO1',
          'C3STDH1','OBFOOLOR','MERGTROLR','44MZYMMO','4MZYMCODH','yli_R1449','yli_R1454','yli_R1455','yli_R1456',
          'yli_R0758','yli_R0759','ERGSTOLR_c','CHLSTD78I','LSTO2r','LTHSTRLOR']
model.remove_reactions(Remove, remove_orphans=True)
#C4STMO2r, C4STMO2Pr wrong -> replace with C4STMO2 and C3STDH2, and change to ER 
model.remove_reactions(['C4STMO2r','C4STMO2Pr'], remove_orphans=True)
# check nadh/nadph for reactions
model.reactions.get_by_id('SQLEr').gene_reaction_rule = '13286 and 13729'
model.reactions.get_by_id('LNS14DMr').gene_reaction_rule = '12843 and 13286'
# 12791 DnaJ homolog, 13559 C-8,7 sterol isomerase
model.reactions.get_by_id('CHLSTI_1').gene_reaction_rule = '13559'
# 8781 can convert chlstol to ddsmsterol -> keep LSTO1r
# Add ERG27 reactions from sce
r1 = sce.reactions.get_by_id('C3STKR1')
r2 = sce.reactions.get_by_id('C3STKR2')
r1.gene_reaction_rule = 'ERG27'
r2.gene_reaction_rule = 'ERG27'
model.add_reactions([r1,r2])
# Change cyto to ER
for x in ['C3STKR1','C4STMO2','C3STDH2','C3STKR2','SAM24MT','C8STI','C5STDS','C22STDS','CHLSTI_1']:
    r = model.reactions.get_by_id(x)
    r.id = r.id+'er'
    for m in r.metabolites:
        if not m.id.replace('_c','_r') in model.metabolites:
            m2 = m.copy()
            m2.id = m.id.replace('_c','_r')
            m2.compartment = 'r'
            model.add_metabolites([m2])
        r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})

In [144]:
for r in sorted(model.genes.get_by_id('13286').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

C22STDSer ergtrol_r + h_r + nadph_r + o2_r --> ergtetrol_r + 2.0 h2o_r + nadp_r 11716 and 13286
CYTP450R cytP450o_c + h_c + nadph_c <=> cytP450r_c + nadp_c 13286
CYTP450Rh cytP450o_h + h_h + nadph_h <=> cytP450r_h + nadp_h 13286
FADRx2 fad_c + h_c + nadph_c --> fadh2_c + nadp_c 13286 and 8744
FLVR h_c + nadph_c + ribflv_c --> nadp_c + rbflvrd_c 13286 and 8744
FMNRx2 fmn_c + h_c + nadph_c --> fmnh2_c + nadp_c 13286 and 8744
LNS14DMr 2.0 h_r + lanost_r + 3.0 nadph_r + 3.0 o2_r --> 44mctr_r + for_r + 4.0 h2o_r + 3.0 nadp_r 12843 and 13286
NFTYROX 2.0 Nfortyr_c + h_c + nadph_c --> Nbfortyr_c + nadp_c (13286 and 13361) or (13286 and 8979) or (13286 and 9188)
SQLEr h_r + nadph_r + o2_r + sql_r --> Ssq23epx_r + h2o_r + nadp_r 13286 and 13729
SULR 5.0 h_c + 3.0 nadph_c + so3_c --> 3.0 h2o_c + h2s_c + 3.0 nadp_c 10374 or 8744 or (PP_0860 and 8744) or (10374 and 8744) or (13286 and 8744)


13361	extr 7, E.R. 7, cyto 4, plas 3, golg 2, nucl 1, mito 1, pero 1, vacu 1, mito_nucl 1	K20660: CYP709; cytochrome P450, family 709	DEE*  
8979	plas 8, extr 5, cyto 3, pero 3, E.R. 3, cyto_pero 3	K00490: CYP4F; cytochrome P450, family 4, subfamily F	DEV*  
9188	nucl 8, pero 5, cyto_mito 5, mito 4.5, cyto 4.5, extr 4	K20495: CYP704B1; long-chain fatty acid omega-monooxygenase	KRA*  
10374	cyto 16, cyto_mito 11.333, cyto_nucl 9.833, mito 5.5, nucl 2.5	K00380: cysJ; sulfite reductase (NADPH) flavoprotein alpha-component	HTQ*  
8744	mito 9, cyto 8, nucl 4, extr 4	K00381: cysI; sulfite reductase (NADPH) hemoprotein beta-component	CPQ*  

9188 CYP704B1; long-chain fatty acid omega-monooxygenase expression is up in bisabolene strains at 48h, need to add reactions for this enzyme  

In [145]:
for r in sorted(model.genes.get_by_id('13361').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8979').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('9188').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('10374').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('8744').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)


NFTYROX 2.0 Nfortyr_c + h_c + nadph_c --> Nbfortyr_c + nadp_c (13286 and 13361) or (13286 and 8979) or (13286 and 9188)

NFTYROX 2.0 Nfortyr_c + h_c + nadph_c --> Nbfortyr_c + nadp_c (13286 and 13361) or (13286 and 8979) or (13286 and 9188)

NFTYROX 2.0 Nfortyr_c + h_c + nadph_c --> Nbfortyr_c + nadp_c (13286 and 13361) or (13286 and 8979) or (13286 and 9188)

SULR 5.0 h_c + 3.0 nadph_c + so3_c --> 3.0 h2o_c + h2s_c + 3.0 nadp_c 10374 or 8744 or (PP_0860 and 8744) or (10374 and 8744) or (13286 and 8744)

FADRx2 fad_c + h_c + nadph_c --> fadh2_c + nadp_c 13286 and 8744
FLVR h_c + nadph_c + ribflv_c --> nadp_c + rbflvrd_c 13286 and 8744
FMNRx2 fmn_c + h_c + nadph_c --> fmnh2_c + nadp_c 13286 and 8744
NO2R 6.0 fdxrd_h + no2_h --> 6.0 fdxox_h + 2.0 h2o_h + 4.0 h_h + nh4_h (CRv4_Au5_s16_g6229_t1 and 8744) or (CRv4_Au5_s17_g7064_t1 and 8744) or (CRv4_Au5_s3_g10824_t1 and 8744) or (CRv4_Au5_s6_g13523_t1 and 8744) or (CRv4_Au5_s7_g14133_t1 and 8744)
SELTORc 6.0 fdxrd_c + slnt_c --> 6.0 fdxox_c

In [146]:
# CYTP450 - NADPH reductase
model.reactions.get_by_id('SULR').gene_reaction_rule = '10374 and 8744'
# CYTP450R/h blocked, FADRx2 and FLVR by E. coli cysJ blocked
# Need to update Iron-sulfur cluster biosynthesis and riboflavin biosynthesis
# NFTYROX only present in iMM904 and involved in spore wall maturation, ortholog not found in IFO0880
model.remove_reactions(['CYTP450R','CYTP450Rh','FADRx2','FLVR','FMNRx2','NFTYROX'], remove_orphans=True)
model.remove_reactions(['NO2R','SELTORc','SELTORm','SULR_ferr'], remove_orphans=True)

Clean up mevalonate pathway  
ERG10	8678	mito 23.5, cyto_mito 14	K00626: E2.3.1.9, atoB; acetyl-CoA C-acetyltransferase	QRL*  
ERG13	12122	nucl 11, cyto_nucl 10, mito 9, cyto 7	K01641: E2.3.3.10; hydroxymethylglutaryl-CoA synthase	HVA*  
HMG2	9574	plas 16, E.R. 7, mito 3	K00021: HMGCR; hydroxymethylglutaryl-CoA reductase (NADPH)	GKD*  
HMGL	15693	mito 27	K01640: E4.1.3.4, HMGCL, hmgL; hydroxymethylglutaryl-CoA lyase	AKL*  
AACS	15276	plas 9, pero 6, mito 5, cyto 5, cyto_mito 5	K01907: AACS, acsA; acetoacetyl-CoA synthetase	SKL*  
ERG12	8758	cyto 13, extr 9, cyto_nucl 8.833, cyto_mito 7.833, nucl 3.5	K00869: E2.7.1.36, MVK, mvaK1; mevalonate kinase	VCA*  
ERG8	8702	cyto 16, cyto_mito 11.833, cyto_nucl 9.833, mito 6.5	K00938: E2.7.4.2, mvaK2; phosphomevalonate kinase	KKR*  
MVD1	11910	cyto 14.5, cyto_mito 13, mito 10.5	K01597: MVD, mvaD; diphosphomevalonate decarboxylase	SLK*  
IDI1	16203	cyto 11, cyto_nucl 10.5, nucl 8, cysk 5	K01823: idi, IDI; isopentenyl-diphosphate Delta-isomerase	IRM*  
#IDI2	16219	cyto 17, cyto_nucl 13, nucl 7	KOG0142: Isopentenyl pyrophosphate:dimethylallyl pyrophosphate isomerase	IRM* (only ~50% coverage)  
ERG20	12944	cyto 16.5, cyto_nucl 13.5, nucl 9.5	K00787: FDPS; farnesyl diphosphate synthase	RQK*  
BTS1	16503	cysk 9, cyto 8.5, cyto_nucl 7.5, nucl 5.5, pero 2	K00804: GGPS1; geranylgeranyl diphosphate synthase, type III	PKA*  
#PSY1	8589	plas 18, extr 3, mito 2, E.R. 2	HMMPfam:Squalene/phytoene synthase:PF00494,ProSitePatterns:Squalene and phytoene synthases signature 2.:PS01045,SUPERFAMILY::SSF48576,TIGRFAM:CarR_dom_SF: lycopene cyclase domain:TIGR03462	QAL*  
#CRTI	8585	mito 7, plas 5, cyto 4, extr 4, E.R. 4, golg 3	K02292: crtO; beta-carotene ketolase (CrtO type)	LRG*  
ERG9	16591	mito 16.5, cyto_mito 11.5, cyto 5.5, pero 3	K00801: FDFT1; farnesyl-diphosphate farnesyltransferase	KQLP  
ERG1	13729	plas 18, extr 3, E.R. 3, cyto 2	K00511: SQLE, ERG1; squalene monooxygenase	GQM*  
ERG7	9240	cysk 22, cyto 5	K01852: LSS, ERG7; lanosterol synthase	EGW*  
ERG11	12843	cyto 9, E.R. 5, mito 3, extr 3, plas 2, pero 2, vacu 2, mito_nucl 2	K05917: CYP51; sterol 14-demethylase	KVL*  
ERG24	8671	plas 13, mito 9, cyto 3	K00222: TM7SF2, ERG24; Delta14-sterol reductase	YVY*  
ERG25	16640	cyto 14, cyto_nucl 9.333, cyto_pero 8.833, mito 6, nucl 3.5	K07750: E1.14.13.72, SC4MOL, ERG25; methylsterol monooxygenase	KAQ*  
ERG26	8835	cyto 13, cyto_nucl 8.833, cyto_mito 8.833, pero 6, nucl 3.5, mito 3.5	K07748: E1.1.1.170, NSDHL, ERG26; sterol-4alpha-carboxylate 3-dehydrogenase (decarboxylating)	KKA*  
ERG26	13724	mito 16, nucl 5, cyto 3, pero 2	K07748: E1.1.1.170, NSDHL, ERG26; sterol-4alpha-carboxylate 3-dehydrogenase (decarboxylating)	VVA*  
ERG6	13060	mito 11, cyto 8, cyto_pero 7.333, cyto_nucl 5.833, pero 5.5	K00559: E2.1.1.41, SMT1, ERG6; sterol 24-C-methyltransferase	HSQ*  
ERG2	10113	plas 17, E.R. 5, mito 2, extr 2	K09829: ERG2; C-8 sterol isomerase	GKI*  
ERG3	8781	plas 26	K00227: SC5DL, ERG3; Delta7-sterol 5-desaturase	ARK*  
ERG5	11716	plas 14, E.R. 8, vacu 3	K09831: ERG5, CYP61A; sterol 22-desaturase	QDL*  
ERG4	10981	plas 18, E.R. 5, mito 3	K00223: ERG4; Delta24(24(1))-sterol reductase	YVF*

In [147]:
for x in ['8678','12122','9574','15693','15276','8758','8702','11910','16203','12944','16503','16591','13729','9240','12843','8671','16640','8835','13724','13060','10113','8781','11716','10981']:
    for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
        print(r.id, r.reaction, r.gene_reaction_rule)
    print()

ACACT10m 2maacoa_m + coa_m <=> accoa_m + ppcoa_m 8678
ACACT1m 2.0 accoa_m <=> aacoa_m + coa_m 8678
ACACT1r 2.0 accoa_c <=> aacoa_c + coa_c 8678

HMGCOAS coa_c + h_c + hmgcoa_c <=> aacoa_c + accoa_c + h2o_c 12122
HMGCOASm coa_m + h_m + hmgcoa_m <=> aacoa_m + accoa_m + h2o_m 12122
MHGS aacoa_c + accoa_c + h2o_c --> coa_c + hmgcoa_c 12122

HMGCOAR coa_c + mev__R_c + 2.0 nadp_c <=> 2.0 h_c + hmgcoa_c + 2.0 nadph_c 9574
HMGCOARr 2.0 h_r + hmgcoa_r + 2.0 nadph_r --> coa_r + mev__R_r + 2.0 nadp_r 9574
HMGCOARx 2.0 h_x + hmgcoa_x + 2.0 nadph_x --> coa_x + mev__R_x + 2.0 nadp_x 9574

HMGL hmgcoa_c --> acac_c + accoa_c 15693
HMGLm hmgcoa_m --> acac_m + accoa_m 15693
HMGLx hmgcoa_x --> acac_x + accoa_x 15693

AACOAT acac_c + atp_c + coa_c <=> aacoa_c + amp_c + ppi_c 15276

MEVK1 atp_c + mev__R_c --> 5pmev_c + adp_c + h_c 8758
MEVK1x atp_x + mev__R_x --> 5pmev_x + adp_x + h_x 8758
MEVK2 ctp_c + mev__R_c --> 5pmev_c + cdp_c + h_c 8758
MEVK3 gtp_c + mev__R_c --> 5pmev_c + gdp_c + h_c 8758
MEVK4 mev_

In [148]:
r = model.reactions.get_by_id('AACOAT')
r.id = 'AACOATx'
for m in r.metabolites:
    if not m.id.replace('_c','_x') in model.metabolites:
        m2 = m.copy()
        m2.id = m.id.replace('_c','_x')
        m2.compartment = 'x'
        model.add_metabolites([m2])
    r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_x'): r.get_coefficient(m.id)})
model.reactions.get_by_id('DMATT').gene_reaction_rule = '12944'
model.reactions.get_by_id('GRTT').gene_reaction_rule = '12944'
r = hsa2.reactions.get_by_id('FRDPtcr').copy()
model.add_reactions([r])
model.remove_reactions(['MHGS','HMGCOARx','HMGL','OCOAT1','MEVK1x','MEVK2','MEVK3','MEVK4','DPMVDx',
                        'IDIh','IPDDIx','DMATTx','FPPSh','GRTTx','UDCPDPS','yli_R0768'], remove_orphans=True)

13090	mito 27	K01027: OXCT; 3-oxoacid CoA-transferase, Succinyl-CoA:3-ketoacid coenzyme A transferase 1, mitochondrial	MDA*  
12795	cyto 16, mito 9, cyto_nucl 9	K11539: CBR4; carbonyl reductase 4	WTA*  
15180	cyto 13, cyto_nucl 8, cysk 8, mito 4	K00074: paaH, hbd, fadB, mmgB; 3-hydroxybutyryl-CoA dehydrogenase	YTAR  
11962	cyto 7.5, mito 6, cyto_nucl 6, cysk 6, extr 3, pero 3	K01555: FAH, fahA; fumarylacetoacetase	RMA*

In [149]:
for r in sorted(model.genes.get_by_id('13090').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('12795').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('15180').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)    

ACACCT acac_c + accoa_c --> aacoa_c + ac_c b2221 and 13090
BUTCT accoa_c + but_c --> ac_c + btcoa_c b2221 and 13090
HXCT accoa_c + hxa_c --> ac_c + hxcoa_c b2221 and 13090
OCOAT1m acac_m + succoa_m <=> aacoa_m + succ_m 13090

BDHm bhb_m + nad_m <=> acac_m + h_m + nadh_m 12795

HADPCOADH3 3hadpcoa_c + nad_c <=> h_c + nadh_c + oxadpcoa_c 15180
HBCO_nadp aacoa_c + h_c + nadph_c <=> 3hbcoa_c + nadp_c 15180


In [150]:
model.remove_reactions(['ACACCT','BUTCT','HXCT'], remove_orphans=True)

In [151]:
for r in sorted(model.metabolites.get_by_id('stcoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ACOADAGAT1601819Z180 12dgr1601819Z_c + stcoa_c --> coa_c + tag1601819Z180_c 16460
ACOADAGAT1801819Z180 12dgr1801819Z_c + stcoa_c --> coa_c + tag1801819Z180_c 16460
ACOADAGAT1819Z1819Z180 12dgr1819Z1819Z_c + stcoa_c --> coa_c + tag1819Z1819Z180_c 16460
AGAT_SC 0.01 1ag3p_SC_c + 0.02 dcacoa_c + 0.06 ddcacoa_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c --> coa_c + 0.01 pa_SC_c 10427
ECOAR h_c + nadph_c + od2coa_c --> nadp_c + stcoa_c 16241
EPISTAT_SC 0.01 epist_c + 0.655 hdcoa_c + 0.01 hexccoa_c + 0.27 odecoa_c + 0.02 pmtcoa_c + 0.03 stcoa_c + 0.015 tdcoa_c --> coa_c + 0.01 epistest_SC_c 11799
ERGSTAT_SC 0.01 ergst_c + 0.655 hdcoa_c + 0.01 hexccoa_c + 0.27 odecoa_c + 0.02 pmtcoa_c + 0.03 stcoa_c + 0.015 tdcoa_c --> coa_c + 0.01 ergstest_SC_c 11799
FA180COAabcp atp_x + h2o_x + stcoa_c --> adp_x + h_x + pi_x + stcoa_x 13167 or 9637
FACOAE180 h2o_c + stcoa_c --> coa_c + h_c + ocdca_c 8456
FACOAL180 atp_c + coa_c + ocdca_c --> amp_c + ppi_c 

In [152]:
model.remove_reactions(['yli_R0307','yli_R0308','yli_R0319','yli_R0335'], remove_orphans=True)

Fatty acid biosynthesis  
ACC1	8639	cyto 12.5, cyto_nucl 10, nucl 6.5, pero 5	K11262: ACACA; acetyl-CoA carboxylase / biotin carboxylase 1	ILA*  
ACH1	13797	mito 23, cyto 3	K01067: E3.1.2.1, ACH1; acetyl-CoA hydrolase	GWS*  
FAS1	8670	mito 22, cyto 3.5, cyto_nucl 2.5	K00668: FAS1; fatty acid synthase subunit beta, fungi type	GGL*  
FAS2	8777	cyto 15.5, cyto_nucl 9.5, mito 3, nucl 2.5, E.R. 2, plas 1, extr 1, pero 1, vacu 1	K00667: FAS2; fatty acid synthase subunit alpha, fungi type	AQK*  

FAA1	11167	cyto 11, cyto_nucl 8, mito 7, nucl 3, plas 3	K01897: ACSL, fadD; long-chain acyl-CoA synthetase	VYP*  
FAA2	12555	cyto 15.5, cyto_nucl 11.5, pero 6, nucl 4.5	K01897: ACSL, fadD; long-chain acyl-CoA synthetase	AKL*  
FAA4	15748	cyto 23, mito 3	K01897: ACSL, fadD; long-chain acyl-CoA synthetase	VYP*  

ELO2	16655	plas 22, E.R. 2, nucl 1, mito 1, vacu 1, mito_nucl 1	K10245: ELO2; fatty acid elongase 2	KTN*  
ELO3	11343	mito 13, extr 5, plas 4, E.R. 2, nucl 1, cyto 1, cyto_nucl 1, pero 1, cyto_pero 1	KOG3072: Long chain fatty acid elongase	KAN*, no rxn  
IFA38	16695	cyto 12, cyto_nucl 8.5, mito 7, pero 4, nucl 3	K10251: HSD17B12, KAR, IFA38; 17beta-estradiol 17-dehydrogenase / very-long-chain 3-oxoacyl-CoA reductase	KGE*  
PHS1	10677	mito 23, cyto 2	K10703: PHS1, PAS2; very-long-chain (3R)-3-hydroxyacyl-CoA dehydratase	KSE*  
TSC13	16241	mito 20.5, cyto_mito 12.333, cyto 3, cyto_nucl 2.833	K10258: TER, TSC13, CER10; very-long-chain enoyl-CoA reductase	FIA*  

Mitochondrial FA biosynthesis
Missing HFA1 (ACCOACrm), CEM1 (malACP + acACP -> acacACP + CO2), HTD2 (3hbACP -> b2ACP, CoA rxn can be done by ECOAH)  
MCT1	16542	mito 26	K00645: fabD; [acyl-carrier-protein] S-malonyltransferase	DGQ*  
ETR1	11837	mito 27	K07512: MECR, NRBF1; mitochondrial trans-2-enoyl-CoA reductase	KDE*  
ACP1	14261	mito 24, cyto 2	KOG1748: Acyl carrier protein/NADH-ubiquinone oxidoreductase, NDUFAB1/SDAP subunit	EAH*  
CBR4+HSD17B8 (OAR1, acetocetyl-ACP -> 3hbACP, CoA rxn can be done by HACD)  
CBR4	12795	cyto 16, mito 9, cyto_nucl 9	K11539: CBR4; carbonyl reductase 4	WTA*  
HSD17B8	11765	mito 19.5, cyto_mito 12, cyto 3.5, extr 2	K13370: HSD17B8; 17beta-estradiol 17-dehydrogenase / 3alpha(17beta)-hydroxysteroid dehydrogenase (NAD+)	TPT*  
AIM22	13899	mito 20, nucl 4, cyto 1, plas 1, pero 1, cyto_pero 1	K03800: lplA, lplJ; lipoate---protein ligase	GEM*  
LIP2	14821	mito 27	K03801: lipB; lipoyl(octanoyl) transferase	LGV*  
LIP5	14096	mito 25	K03644: lipA; lipoyl synthase	VRV*  

Many 3-oxoacyl-[acyl-carrier protein] reductase genes  
incomplete? remove them for now, and check later  
https://www.ncbi.nlm.nih.gov/pubmed/19028688  
https://www.ncbi.nlm.nih.gov/pubmed/27553474

Mitochondrial fatty acid synthesis is needed for lipoic acid biosynthesis in S. cer  
S. cer lacks mitochondrial beta-oxidation, therefore cannot make C8 FA needed for lipoate  
R. toruloides can degrade FA in mitochondria, check the Mito FAS in other fungi  

In [153]:
for x in ['8639','13797','8670','8777','11167','12555','15748','16655','16695','10677','16241',
          '16542','11837','14261','12795','13899','14821','14096',]:
    for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
        print(r.id, r.reaction, r.gene_reaction_rule)
    print()

ACCOAC accoa_c + atp_c + hco3_c --> adp_c + h_c + malcoa_c + pi_c 8639
ACCOACrm accoa_m + atp_m + hco3_m <=> adp_m + h_m + malcoa_m + pi_m 8639
FAS100 3.0 h_c + malcoa_c + 2.0 nadph_c + octa_c --> co2_c + coa_c + dca_c + h2o_c + 2.0 nadp_c (12826 and 8639 and 8670 and 8777) or (14471 and 8639 and 8670 and 8777)
FAS100COA 3.0 h_c + malcoa_c + 2.0 nadph_c + occoa_c --> co2_c + coa_c + dcacoa_c + h2o_c + 2.0 nadp_c (12826 and 8639 and 8670 and 8777) or (14471 and 8639 and 8670 and 8777)
FAS120 dca_c + 3.0 h_c + malcoa_c + 2.0 nadph_c --> co2_c + coa_c + ddca_c + h2o_c + 2.0 nadp_c (12826 and 8639 and 8670 and 8777) or (14471 and 8639 and 8670 and 8777)
FAS120COA dcacoa_c + 3.0 h_c + malcoa_c + 2.0 nadph_c --> co2_c + coa_c + ddcacoa_c + h2o_c + 2.0 nadp_c (12826 and 8639 and 8670 and 8777) or (14471 and 8639 and 8670 and 8777)
FAS140 ddca_c + 3.0 h_c + malcoa_c + 2.0 nadph_c --> co2_c + coa_c + h2o_c + 2.0 nadp_c + ttdca_c (12826 and 8639 and 8670 and 8777) or (14471 and 8639 and 8670 and

FAS161 4.0 h_c + malcoa_c + 3.0 nadph_c + o2_c + ttdca_c --> co2_c + coa_c + 3.0 h2o_c + hdcea_c + 3.0 nadp_c 16655
FAS181 4.0 h_c + hdca_c + malcoa_c + 3.0 nadph_c + o2_c --> co2_c + coa_c + 3.0 h2o_c + 3.0 nadp_c + ocdcea_c 16655
FAS240_L 9.0 h_c + 3.0 malcoa_c + 6.0 nadph_c + ocdca_c --> 3.0 co2_c + 3.0 coa_c + 3.0 h2o_c + 6.0 nadp_c + ttc_c (16241 and 16655) or (10677 and 16241 and 16695)
FAS260 3.0 h_c + malcoa_c + 2.0 nadph_c + ttc_c --> co2_c + coa_c + h2o_c + hexc_c + 2.0 nadp_c (16241 and 16655) or (10677 and 16241 and 16695)

FAS140COA ddcacoa_c + 3.0 h_c + malcoa_c + 2.0 nadph_c --> co2_c + coa_c + h2o_c + 2.0 nadp_c + tdcoa_c (10677 and 16241 and 16695) or (12826 and 8639 and 8670 and 8777) or (14471 and 8639 and 8670 and 8777)
FAS160COA 3.0 h_c + malcoa_c + 2.0 nadph_c + tdcoa_c --> co2_c + coa_c + h2o_c + 2.0 nadp_c + pmtcoa_c (10677 and 16241 and 16695) or (12826 and 8639 and 8670 and 8777) or (14471 and 8639 and 8670 and 8777)
FAS180 3.0 h_c + hdca_c + malcoa_c + 2.0 na

In [154]:
temp = ['13899','14821','14096']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
13899,"K03800: lplA, lplJ; lipoate---protein ligase",,AIM22,LIPT1,AIM22,LIPT1,Essential,"mito 20, nucl 4, cyto 1, plas 1, pero 1, cyto_...",GEM*
14821,K03801: lipB; lipoyl(octanoyl) transferase,S,LIP2,LIPT2,,,Not Essential,mito 27,LGV*
14096,K03644: lipA; lipoyl synthase,S,LIP5,LIAS,LIP5,LIAS,Essential,mito 25,VRV*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,


In [155]:
temp = ['12826','14471','8639','14261','16542','11837']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
12826,"K08762: DBI, ACBP; diazepam-binding inhibitor ...",,ACB1,"ACBD7,DBI",ACB1,DBI,Essential,"cyto 11, mito 8, cyto_nucl 8",LAA*
14471,"K08762: DBI, ACBP; diazepam-binding inhibitor ...",,,,ACB1,DBI,Not Essential,"mito 16, nucl 8, cyto 1, pero 1, vacu 1, cyto_...",VRR*
8639,K11262: ACACA; acetyl-CoA carboxylase / biotin...,,"HFA1,ACC1","ACACA,ACACB",ACC1,ACACB,Essential,"cyto 12.5, cyto_nucl 10, nucl 6.5, pero 5",ILA*
14261,KOG1748: Acyl carrier protein/NADH-ubiquinone ...,S,ACP1,NDUFAB1,,,Essential,"mito 24, cyto 2",EAH*
16542,K00645: fabD; [acyl-carrier-protein] S-malonyl...,,,MCAT,,,Not Essential,mito 26,DGQ*
11837,"K07512: MECR, NRBF1; mitochondrial trans-2-eno...",S,ETR1,MECR,ETR1,MECR,Not Essential,mito 27,KDE*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
14471,0.0630059,0.0458949,-0.162117,-0.0445085,-0.107805,-0.252623,-0.220519,-1.11705,-0.439887,-0.362255,0.227652
16542,-0.161755,-0.168402,-0.475257,0.334468,-0.474717,-0.326335,-0.154747,-1.28835,-0.424862,-0.0347438,-0.659424
11837,-0.00746914,-0.0836084,-0.0138207,-0.0768996,-0.241361,0.0511913,-0.0700867,0.287896,-0.04288,-0.327269,0.112745


In [156]:
# Remove FAS reactions with free fatty acid as product
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FAS') and not ('COA' in r.id or 'ACP' in r.id):
        print(r.id, r.reaction, r.gene_reaction_rule)
        model.remove_reactions([r])

FAS100 3.0 h_c + malcoa_c + 2.0 nadph_c + octa_c --> co2_c + coa_c + dca_c + h2o_c + 2.0 nadp_c (12826 and 8639 and 8670 and 8777) or (14471 and 8639 and 8670 and 8777)
FAS120 dca_c + 3.0 h_c + malcoa_c + 2.0 nadph_c --> co2_c + coa_c + ddca_c + h2o_c + 2.0 nadp_c (12826 and 8639 and 8670 and 8777) or (14471 and 8639 and 8670 and 8777)
FAS140 ddca_c + 3.0 h_c + malcoa_c + 2.0 nadph_c --> co2_c + coa_c + h2o_c + 2.0 nadp_c + ttdca_c (12826 and 8639 and 8670 and 8777) or (14471 and 8639 and 8670 and 8777)
FAS141 ddca_c + 4.0 h_c + malcoa_c + 3.0 nadph_c + o2_c --> co2_c + coa_c + 3.0 h2o_c + 3.0 nadp_c + ttdcea_c 16655
FAS160 3.0 h_c + malcoa_c + 2.0 nadph_c + ttdca_c --> co2_c + coa_c + h2o_c + hdca_c + 2.0 nadp_c (12826 and 8639 and 8670 and 8777) or (14471 and 8639 and 8670 and 8777)
FAS161 4.0 h_c + malcoa_c + 3.0 nadph_c + o2_c + ttdca_c --> co2_c + coa_c + 3.0 h2o_c + hdcea_c + 3.0 nadp_c 16655
FAS180 3.0 h_c + hdca_c + malcoa_c + 2.0 nadph_c --> co2_c + coa_c + h2o_c + 2.0 nadp_c 

In [157]:
# Remove cyto ACP reactions
model.remove_reactions(['ACOATA','FA120ACPHi','FA140ACPHi','FA141ACPHi','FA160ACPHi','FA161ACPHi','FA180ACPH',
                        'FA181ACPH','FA182ACPH','MCOATA','yli_R0163','yli_R0164','yli_R0165','yli_R0166',
                        'yli_R0167','yli_R0168','yli_R0169','yli_R0170','yli_R0171','yli_R0172','yli_R0175',
                        'yli_R0176','yli_R0177','yli_R0178','yli_R0179','yli_R0180'], remove_orphans=True)
# Remove mito FA ACP synthesis
model.remove_reactions(['ACCOACrm','FAS100ACPm','FAS120ACPm','FAS140ACPm','FAS141ACPm','FAS160ACPm','FAS161ACPm',
                        'FAS180ACPm','FAS181ACPm','FAS182ACPm','FAS80ACPm_L'], remove_orphans=True)
model.remove_reactions(['ECOAR','yli_R0126','yli_R0129','yli_R0140','MCOATA_1','ACOATAm','ACPS1_1'], remove_orphans=True)
for r in sorted(model.genes.get_by_id('14261').reactions, key=lambda x: x.id):
    if 'h' in r.compartments:
        model.remove_reactions([r])
    if 'b' in r.gene_reaction_rule:
        model.remove_reactions([r])

In [158]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('FAS') and 'COA' in r.id:
        r.gene_reaction_rule = '8670 and 8777'
        
r = sce.reactions.get_by_id('FAS240_L').copy()
r.id = 'FAS240COAer'
r.gene_reaction_rule = '(10677 and 11343 and 16241 and 16695) or (10677 and 16241 and 16655 and 16695)'
model.add_reactions([r])
r.add_metabolites({'ocdca_c': 1.0, 'stcoa_c': -1.0, 'ttc_c': -1.0, 'ttccoa_c': 1.0})
for m in r.metabolites:
    if not m.id.replace('_c','_r') in model.metabolites:
        m2 = m.copy()
        m2.id = m.id.replace('_c','_r')
        m2.compartment = 'r'
        model.add_metabolites([m2])
    r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})
    
r = sce.reactions.get_by_id('FAS260').copy()
r.id = 'FAS260COAer'
r.gene_reaction_rule = '10677 and 11343 and 16241 and 16695'
model.add_reactions([r])
r.add_metabolites({'ttc_c': 1.0, 'ttccoa_c': -1.0, 'hexc_c': -1.0, 'hexccoa_c': 1.0})
for m in r.metabolites:
    if not m.id.replace('_c','_r') in model.metabolites:
        m2 = m.copy()
        m2.id = m.id.replace('_c','_r')
        m2.compartment = 'r'
        model.add_metabolites([m2])
    r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})
    
for r in sorted(model.reactions, key=lambda x: x.id):
    if r.id.startswith('ACOAR'):
        r.gene_reaction_rule = '11837'
        
model.reactions.get_by_id('13GS').gene_reaction_rule = '15984'
model.reactions.get_by_id('MCOATAm').gene_reaction_rule = '14261 and 16542'

for x in ['LIPAMPL','LIPATPT','OCTNLL','LIPOCT','LIPOS']:
    r = model.reactions.get_by_id(x)
    r.id = r.id + 'm'
    for m in r.metabolites:
        if not m.id.replace('_c','_m') in model.metabolites:
            m2 = m.copy()
            m2.id = m.id.replace('_c','_m')
            m2.compartment = 'm'
            model.add_metabolites([m2])
        r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_m'): r.get_coefficient(m.id)})

TAG biosynthesis (E.R.)
GPD1	12154	cyto 23, pero 4	K00006: GPD1; glycerol-3-phosphate dehydrogenase (NAD+)	AHI*  
GPD2	14576	cyto 17.5, cyto_nucl 14, nucl 5.5	K00006: GPD1; glycerol-3-phosphate dehydrogenase (NAD+)	EGI*  
SCT1	15435	plas 8, cyto 7.5, cyto_nucl 6.5, nucl 4.5, mito 4	K13507: GAT; glycerol-3-phosphate O-acyltransferase / dihydroxyacetone phosphate acyltransferase	AQE*  
GNPAT	13369	cyto 12.5, cyto_nucl 10.5, nucl 7.5, mito 5	K00649: GNPAT; glyceronephosphate O-acyltransferase	AAL*  
AYR1	15575	plas 10, mito 8, cyto 5, E.R. 3	KOG1209: 1-Acyl dihydroxyacetone phosphate reductase and related dehydrogenases	KRR*  
SLC1	10427	mito 18, E.R. 4, extr 2, plas 1, golg 1, vacu 1	K13509: AGPAT1_2; lysophosphatidate acyltransferase	PSE*  
ALE1	16030	plas 18, E.R. 4, vacu 3	K13519: LPT1, ALE1; lysophospholipid acyltransferase	KEL*  
16030 is the blast hit of known lysoPC AT -> LPCAT_SC, TAZ1 is mitochondrial  
LOA1	16779	extr 11, mito 8, cyto 5, plas 2	KOG2898: Predicted phosphate acyltransferase, contains PlsC domain	KGR*  
TGL4,TGL5	9746	plas 22, nucl 2, mito 1, cyto 1, vacu 1, cyto_mito 1	KOG2214: Predicted esterase of the alpha-beta hydrolase superfamily	PWH*  
16779, 9746 oleoyl-CoA specific  
DPP1	13087	plas 23, mito 2	K18693: DPP1; diacylglycerol diphosphate phosphatase / phosphatidate phosphatase	GYY*  
PAH1	12485	nucl 12.5, cyto_nucl 10.5, cyto 7.5, mito 5	K15728: LPIN; phosphatidate phosphatase LPIN	PYL*  
DGK1	10156	mito 10, plas 8, cyto 3, pero 2, E.R. 2	K16368: DGK1; diacylglycerol kinase (CTP)	ILG*  
DGA1	16460	plas 12, mito 5, extr 3, E.R. 3, mito_nucl 3	K14457: MOGAT2, MGAT2; 2-acylglycerol O-acyltransferase 2	IIA*  
ARE1,ARE2	11799	plas 24, E.R. 2	K11155: DGAT1; diacylglycerol O-acyltransferase 1	LRY*  
LRO1	16477	cyto 9.5, cyto_nucl 9.5, nucl 8.5, pero 5, mito 3	K00679: E2.3.1.158; phospholipid:diacylglycerol acyltransferase	WEG*  
TAZ1	16474	mito 13, cyto_nucl 5, nucl 4.5, cyto 4.5, plas 2, pero 2	K13511: TAZ; monolysocardiolipin acyltransferase	TGW*  
TGL2	14317	mito 12, cyto 7.5, cyto_nucl 5, pero 3, extr 2	K01046: E3.1.1.3; triacylglycerol lipase	EGF*  
10393 best hit TGL2  
TGL4,TGL5	9746	plas 22, nucl 2, mito 1, cyto 1, vacu 1, cyto_mito 1	KOG2214: Predicted esterase of the alpha-beta hydrolase superfamily	PWH*  
TGL1	14617	plas 8, mito 6, nucl 4.5, cyto_nucl 4.5, cyto 3.5, pero 3	K01052: LIPA; lysosomal acid lipase/cholesteryl ester hydrolase	DRV*  
YEH1,YEH2	14247	plas 12, mito 5, golg 3, cyto_mito 3, nucl 2, KOG2624: Triglyceride lipase-cholesterol esterase	EGY*
NTE1	14309	extr 8, plas 7, cyto 6, nucl 2, mito 2, mito_nucl 2	K14676: NTE, NRE; lysophospholipid hydrolase	RLT*  
PLB1,PLB2,PLB3	12385	extr 27	K13333: PLB; lysophospholipase	LLL*  

LIPE	15065	extr 10, plas 5, mito 4, cyto 2, E.R. 2, golg 2	K07188: LIPE, HSL; hormone-sensitive lipase  
ATG15	9000	nucl 16.5, cyto_nucl 12.5, cyto 7.5	K17900: ATG15, AUT5; lipase ATG15	TYV*

In [159]:
temp = ['14317','9746','14617','14247','14309','12385','9000']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
14317,K01046: E3.1.1.3; triacylglycerol lipase,,TGL2,,TGL2,,Not Essential,"mito 12, cyto 7.5, cyto_nucl 5, pero 3, extr 2",EGF*
9746,KOG2214: Predicted esterase of the alpha-beta ...,,"TGL4,TGL5",,TGL5,,Not Essential,"plas 22, nucl 2, mito 1, cyto 1, vacu 1, cyto_...",PWH*
14617,K01052: LIPA; lysosomal acid lipase/cholestery...,,TGL1,"LIPF,LIPK,LIPA,LIPJ,LIPM",YEH2,LIPK,Not Essential,"plas 8, mito 6, nucl 4.5, cyto_nucl 4.5, cyto ...",DRV*
14247,KOG2624: Triglyceride lipase-cholesterol esterase,,"YEH1,YEH2",,YEH2,LIPA,Not Essential,"plas 12, mito 5, golg 3, cyto_mito 3, nucl 2, ...",EGY*
14309,"K14676: NTE, NRE; lysophospholipid hydrolase",A,NTE1,"PNPLA6,PNPLA7",NTE1,PNPLA,Not Essential,"extr 8, plas 7, cyto 6, nucl 2, mito 2, mito_n...",RLT*
12385,K13333: PLB; lysophospholipase,S,"PLB2,PLB1,PLB3",,PLB1,,Not Essential,extr 27,LLL*
9000,"K17900: ATG15, AUT5; lipase ATG15",,ATG15,,ATG15,,Not Essential,"nucl 16.5, cyto_nucl 12.5, cyto 7.5",TYV*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
9746,-0.0223147,-0.0041398,-0.068294,0.0642932,0.319812,-0.0303963,0.178785,0.0256476,0.146721,0.192792,0.0446445
14617,-0.0419116,-0.0971542,-0.0660963,0.0316563,0.0744906,0.202458,-0.111763,-0.368591,-0.188215,-0.313815,-0.0952425
14247,-0.0246227,-0.175606,0.00642496,-0.0811898,-0.822125,-0.476904,-0.140187,0.0623536,-0.188237,-0.0181106,-0.327808
14309,-0.0421803,-0.1025,-0.0699972,-0.446404,0.313427,-0.223928,0.170773,-0.526365,-0.067268,-0.221199,-0.447931
12385,0.168375,0.221694,0.0416109,0.0721808,-0.572405,0.258112,-0.0124852,0.0609392,-0.0540949,0.00780372,0.358722
9000,-0.169705,-0.259195,-0.196552,-0.197213,-0.273888,-0.414895,-0.470405,-0.990165,0.145391,0.110963,0.400523


In [160]:
for x in ['12154','14576','15435','13369','15575','10427','16030','9746','13087','12485','10156','16460','11799',
          '16477','16474','14317','14617','14247','14309','12385']:
    for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
        print(r.id, r.reaction, r.gene_reaction_rule)
    print()

G3PD1ir dhap_c + h_c + nadh_c --> glyc3p_c + nad_c 12154 or 14576

G3PD1ir dhap_c + h_c + nadh_c --> glyc3p_c + nad_c 12154 or 14576

GAT1_SC 0.02 dcacoa_c + 0.06 ddcacoa_c + glyc3p_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c --> 0.01 1ag3p_SC_c + coa_c 15435
GAT2_SC 0.02 dcacoa_c + 0.06 ddcacoa_c + dhap_c + 0.17 hdcoa_c + 0.09 ocdycacoa_c + 0.24 odecoa_c + 0.27 pmtcoa_c + 0.05 stcoa_c + 0.1 tdcoa_c --> 0.01 1agly3p_SC_c + coa_c 15435
yli_R1391 0.02 dccoa_r + 0.06 ddcacoa_r + glyc3p_r + 0.27 pmtcoa_r + 0.05 stcoa_r + 0.1 tdcoa_r + 0.17 yli_M04594_r + 0.09 yli_M04625_r + 0.24 yli_M04626_r --> 0.01 1ag3p_SC_r + coa_r 15435
yli_R1408 0.02 dccoa_r + 0.06 ddcacoa_r + dhap_r + 0.27 pmtcoa_r + 0.05 stcoa_r + 0.1 tdcoa_r + 0.17 yli_M04594_r + 0.09 yli_M04625_r + 0.24 yli_M04626_r --> 0.01 adhap_hs_r + coa_r 15435
yli_R1509 acoa_r + glyc3p_r --> 1ag3p_SC_r + coa_r 15435

DHAPA Rtotalcoa_c + dhap_c --> adhap_hs_c + coa_c 13369
DHAPAx Rtotalcoa

In [161]:
print('\''+'\',\''.join([r.id for r in sorted(model.genes.get_by_id('16474').reactions, key=lambda x: x.id)])+'\'')

'1AGPEAT1801819Z','1AGPEAT1801829Z12Z','1AGPEAT1819Z1819Z','1AGPEAT1819Z1829Z12Z','2AGPA120tipp','2AGPA140tipp','2AGPA141tipp','2AGPA160tipp','2AGPA161tipp','2AGPA180tipp','2AGPA181tipp','2AGPE120tipp','2AGPE140tipp','2AGPE141tipp','2AGPE160tipp','2AGPE161tipp','2AGPE180tipp','2AGPE181tipp','2AGPG120tipp','2AGPG140tipp','2AGPG141tipp','2AGPG160tipp','2AGPG161tipp','2AGPG180tipp','2AGPG181tipp','LPCAT_SC'


In [162]:
r = model.reactions.get_by_id('LPCAT_SC').copy()
r.id = 'LPCATm_RT'
model.add_reactions([r])
for m in r.metabolites:
    if not m.id.replace('_c','_m') in model.metabolites:
        m2 = m.copy()
        m2.id = m.id.replace('_c','_m')
        m2.compartment = 'm'
        model.add_metabolites([m2])
    r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_m'): r.get_coefficient(m.id)})

model.reactions.get_by_id('PCDAGAT').id = 'PCDAGAT_SC'
for x in ['GAT1_SC','GAT2_SC','ADHAPR_SC','AGAT_SC','DAGPYP_SC','LPP_SC','TRIGS_SC','EPISTAT_SC','ERGSTAT_SC',
          'FECOSTAT_SC','LANOSTAT_SC','ZYMSTAT_SC','PCDAGAT_SC','LPCAT_SC']:
    r = model.reactions.get_by_id(x)
    r.id = r.id.replace('_SC','er_RT')
    for m in r.metabolites:
        if not m.id.replace('_c','_r') in model.metabolites:
            m2 = m.copy()
            m2.id = m.id.replace('_c','_r')
            m2.compartment = 'r'
            model.add_metabolites([m2])
        r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})
model.reactions.get_by_id('yli_R1522').id = 'DAGKer_RT'
model.reactions.get_by_id('DAGKer_RT').add_metabolites({'dag_hs_r': 1.0, 'pa_EC_r': -1.0,
                                                        '12dgr_SC_r': -0.01, 'pa_SC_r': 0.01, 'h_r': 1.0})
model.reactions.get_by_id('GAT1er_RT').gene_reaction_rule = '15435'
model.reactions.get_by_id('GAT2er_RT').gene_reaction_rule = '13369 or 15435'
model.reactions.get_by_id('AGATer_RT').gene_reaction_rule = '10427 or 16030 or 16779 or 9746'
model.reactions.get_by_id('DAGPYPer_RT').gene_reaction_rule = '12485 or 13087'
model.reactions.get_by_id('TRIGSer_RT').gene_reaction_rule = '16460'
model.reactions.get_by_id('LPCATer_RT').gene_reaction_rule = '16030'

# 12385 PLB1 (Uniprot) sequentially removes both fatty acyl groups from diacylglycerophospholipids and
# therefore has both phospholipase A and lysophospholipase activities. Substrate preference is 
# phosphatidylserine > phosphatidylinositol >> phosphatidylcholine > phosphatidylethanolamine
# (SGD) deacylation of phosphatidylcholine and phosphatidylethanolamine but not phosphatidylinositol
# (BioCyc) Interestingly, the pH in the in vitro assays significantly changed the substrate preference of 
# the enzyme PLB1p. When tested at the optimum pH of 2.5 PLB1p the best substrates were phosphatidylserine 
# and phosphatidylinositol followed by phosphatidylcholine and phosphatidylethanolamine. At the physiological
# relevant pH of 5.5 the substrate preference changed to phosphatidylcholine and phosphatidylethanolamine,
# both catalyzed with almost the same activity, followed by phosphatidylinositol whereas phosphatidylserine
# was not cleaved at all.
# Add PLBPEe_RT
m1 = model.metabolites.get_by_id("pe_SC_c").copy()
m1.id = 'pe_SC_e'
m1.compartment = 'e'
m2 = model.metabolites.get_by_id("g3pe_c").copy()
m2.id = 'g3pe_e'
m2.compartment = 'e'
model.add_metabolites([m1,m2])

model.reactions.get_by_id("PLBPC_SCe").id = 'PLBPCe_RT'
model.reactions.get_by_id("PLBP1I_SCe").id = 'PLBP1Ie_RT'
r = model.reactions.get_by_id("PLBPCe_RT").copy()
r.id = 'PLBPEe_RT'
r.name = 'Phospholipase B phosphatidylethanolamine yeast specific extracellular'
model.add_reactions([r])
r.add_metabolites({'pc_SC_e': 0.005, 'g3pc_e': -0.5, 'pe_SC_e': -0.005, 'g3pe_e': 0.5})

temp = ['yli_R1391','yli_R1408','yli_R1509','DHAPA','DHAPAx','G3PAT120','G3PAT140','G3PAT141','G3PAT160','G3PAT161',
        'G3PAT180','G3PAT181','GPAM_hs','GPAMm_hs','AGPAT1','AGPAT120','AGPAT140','AGPAT141','AGPAT160','AGPAT161',
        'AGPAT180','AGPAT181','AGPATCOA1601819Z','AGPATCOA1801819Z','AGPATCOA1819Z160','AGPATCOA1819Z1819Z',
        'yli_R1392','yli_R1510','1AGPEAT1801819Z','1AGPEAT1801829Z12Z','1AGPEAT1819Z1819Z','1AGPEAT1819Z1829Z12Z',
        'yli_R1402','yli_R1517','yli_R1518','yli_R1523','yli_R1511','yli_R1512','yli_R1515','yli_R1516',
        'PAPA1601819Zh','PAPA160h','PAPA1801819Zh','PAPA1819Z160h','PAPA1819Z1619Zh','PAPA1819Z1819Zh','yli_R1513',
        'yli_R0309','yli_R1393','ACOADAGAT1601819Z160','ACOADAGAT1601819Z180','ACOADAGAT1601819Z1819Z',
        'ACOADAGAT1801819Z160','ACOADAGAT1801819Z180','ACOADAGAT1801819Z1819Z','ACOADAGAT1819Z1819Z160',
        'ACOADAGAT1819Z1819Z180','ACOADAGAT1819Z1819Z1819Z','CRETINOLPMTACT','DGAT','MOGAT','RETINOLACACT',
        'TRETINOLPMTACT','PLDAGAT1601819Z1601','PLDAGAT1601819Z1602','PLDAGAT1601819Z1819Z1','PLDAGAT1601819Z1819Z2',
        'PLDAGAT1601819Z1819Z3','PLDAGAT1601819Z1819Z4','PLDAGAT1801819Z1601','PLDAGAT1801819Z1602',
        'PLDAGAT1801819Z1819Z1','PLDAGAT1801819Z1819Z2','PLDAGAT1801819Z1819Z3','PLDAGAT1801819Z1819Z4',
        'PLDAGAT1819Z1819Z1601','PLDAGAT1819Z1819Z1602','PLDAGAT1819Z1819Z1819Z1','PLDAGAT1819Z1819Z1819Z2',
        'PLDAGAT1819Z1819Z1819Z3','PLDAGAT1819Z1819Z1819Z4','yli_R0311','yli_R1395','2AGPA120tipp','2AGPA140tipp',
        '2AGPA141tipp','2AGPA160tipp','2AGPA161tipp','2AGPA180tipp','2AGPA181tipp','2AGPE120tipp','2AGPE140tipp',
        '2AGPE141tipp','2AGPE160tipp','2AGPE161tipp','2AGPE180tipp','2AGPE181tipp','2AGPG120tipp','2AGPG140tipp',
        '2AGPG141tipp','2AGPG160tipp','2AGPG161tipp','2AGPG180tipp','2AGPG181tipp','EPISTESTH_SCe','ERGSTESTH_SCe',
        'FECOSTESTH_SCe','LANOSTESTH_SCe','LPSe','ZYMSTESTH_SCe','LPLPS1AGPE180','LPLPS1AGPE1819Z',
        'LPLPS1AGPE1829Z12Z','yli_R1519','yli_R1520']
model.remove_reactions(temp, remove_orphans=True)

In [163]:
# Make lipid droplet compartment and move lipid degradation
for x in ['triglyc_SC_r','12dgr_SC_r',
          'epistest_SC_r','ergstest_SC_r','fecostest_SC_r','lanostest_SC_r','zymstest_SC_r']:
    m = model.metabolites.get_by_id(x).copy()
    m.id = m.id.replace('SC_r','RT_d')
    m.compartment = 'd'
    model.add_metabolites([m])

r = model.reactions.get_by_id('FRDPtcr').copy()
r.id = 'TAGtrd'
r.name = 'Transport of triacylglyceride from E.R. to lipid droplet'
r.lower_bound = 0.0
model.add_reactions([r])
model.reactions.get_by_id('TAGtrd').add_metabolites({'frdp_c': 1.0, 'frdp_r': -1.0,
                                                     'triglyc_SC_r': -1, 'triglyc_RT_d': 1})
r = model.reactions.get_by_id('FRDPtcr').copy()
r.id = 'EPISTESTtrd'
r.name = 'Transport of episterol ester from E.R. to lipid droplet'
r.lower_bound = 0.0
model.add_reactions([r])
model.reactions.get_by_id('EPISTESTtrd').add_metabolites({'frdp_c': 1.0, 'frdp_r': -1.0,
                                                     'epistest_SC_r': -1, 'epistest_RT_d': 1})
r = model.reactions.get_by_id('FRDPtcr').copy()
r.id = 'ERGSTESTtrd'
r.name = 'Transport of ergosterol ester from E.R. to lipid droplet'
r.lower_bound = 0.0
model.add_reactions([r])
model.reactions.get_by_id('ERGSTESTtrd').add_metabolites({'frdp_c': 1.0, 'frdp_r': -1.0,
                                                     'ergstest_SC_r': -1.0, 'ergstest_RT_d': 1.0})
r = model.reactions.get_by_id('FRDPtcr').copy()
r.id = 'FECOSTESTtrd'
r.name = 'Transport of fecosterol ester from E.R. to lipid droplet'
r.lower_bound = 0.0
model.add_reactions([r])
model.reactions.get_by_id('FECOSTESTtrd').add_metabolites({'frdp_c': 1.0, 'frdp_r': -1.0,
                                                     'fecostest_SC_r': -1.0, 'fecostest_RT_d': 1.0})
r = model.reactions.get_by_id('FRDPtcr').copy()
r.id = 'LANOSTESTtrd'
r.name = 'Transport of lanosterol ester from E.R. to lipid droplet'
r.lower_bound = 0.0
model.add_reactions([r])
model.reactions.get_by_id('LANOSTESTtrd').add_metabolites({'frdp_c': 1.0, 'frdp_r': -1.0,
                                                     'lanostest_SC_r': -1.0, 'lanostest_RT_d': 1.0})
r = model.reactions.get_by_id('FRDPtcr').copy()
r.id = 'ZYMSTESTtrd'
r.name = 'Transport of zymosterol ester from E.R. to lipid droplet'
r.lower_bound = 0.0
model.add_reactions([r])
model.reactions.get_by_id('ZYMSTESTtrd').add_metabolites({'frdp_c': 1.0, 'frdp_r': -1.0,
                                                     'zymstest_SC_r': -1.0, 'zymstest_RT_d': 1.0})

model.reactions.get_by_id('TAGL_SC').add_metabolites({'triglyc_SC_c': 0.01, 'triglyc_RT_d': -0.01,
                                                      '12dgr_SC_c': -0.01, '12dgr_RT_d': 0.01})
model.reactions.get_by_id('TAGL_SC').gene_reaction_rule = '10393 or 14317 or 14617 or 15065 or 9746'
model.reactions.get_by_id('EPISTESTH_SC').add_metabolites({'epistest_SC_c': 0.01, 'epistest_RT_d': -0.01})
model.reactions.get_by_id('ERGSTESTH_SC').add_metabolites({'ergstest_SC_c': 0.01, 'ergstest_RT_d': -0.01})
model.reactions.get_by_id('FECOSTESTH_SC').add_metabolites({'fecostest_SC_c': 0.01, 'fecostest_RT_d': -0.01})
model.reactions.get_by_id('LANOSTESTH_SC').add_metabolites({'lanostest_SC_c': 0.01, 'lanostest_RT_d': -0.01})
model.reactions.get_by_id('ZYMSTESTH_SC').add_metabolites({'zymstest_SC_c': 0.01, 'zymstest_RT_d': -0.01})
model.reactions.get_by_id('PLBPC_SC').add_metabolites({'pc_SC_c': 0.005, 'pc_SC_r': -0.005})

for x in ['TAGL_SC','EPISTESTH_SC','ERGSTESTH_SC','FECOSTESTH_SC','LANOSTESTH_SC','ZYMSTESTH_SC','PLBPC_SC']:
    model.reactions.get_by_id(x).id = x.replace('_SC','_RT')

DAG -> MAG  
11739 DAGL sn1-specific diacylglycerol lipase  
15065 LIPE  
MAG -> glycerol + FA  
9728 YJU3/MGLL acylglycerol lipase  
14158 YJU3/MGLL acylglycerol lipase  
15065 LIPE

In [164]:
m = model.metabolites.get_by_id('12dgr_RT_d').copy()
m.id = 'mag_RT_d'
m.name = '2-Monoacylglycerol'
m.formula = 'C1920H3722O400'
model.add_metabolites([m])

r = model.reactions.get_by_id('TAGL_RT').copy()
r.id = 'DAGL_RT'
r.name = 'Diacylglycerol lipase'
model.add_reactions([r])
r.add_metabolites({'triglyc_RT_d': 0.01, '12dgr_RT_d': -0.02, 'mag_RT_d': 0.01})
r.gene_reaction_rule = '11739 or 15065'

r = model.reactions.get_by_id('TAGL_RT').copy()
r.id = 'MAGL_RT'
r.name = 'Monoacylglycerol lipase'
model.add_reactions([r])
r.add_metabolites({'triglyc_RT_d': 0.01, '12dgr_RT_d': -0.01, 'mag_RT_d': -0.01, 'glyc_c': 1})
r.gene_reaction_rule = '14158 or 15065 or 9728'

In [165]:
for r in sorted(model.reactions, key=lambda x: x.id):
    if 'lipase' in r.name:
        print(r.id, r.name, r.reaction, r.gene_reaction_rule)

DAGL_RT Diacylglycerol lipase 0.01 12dgr_RT_d + h2o_c --> 0.02 dca_c + 0.06 ddca_c + h_c + 0.27 hdca_c + 0.17 hdcea_c + 0.01 mag_RT_d + 0.05 ocdca_c + 0.24 ocdcea_c + 0.09 ocdcya_c + 0.1 ttdca_c 11739 or 15065
IPC124PLCg_RT Inositol phosphorylceramide  ceramide 1  24C  phospholipase C h2o_g + 0.01 ipc124_SC_g --> cer1_24_g + h_g + mi1p__D_g 15857
IPC126PLCg_RT Inositol phosphorylceramide  ceramide 1  26C  phospholipase C h2o_g + 0.01 ipc126_SC_g --> cer1_26_g + h_g + mi1p__D_g 15857
IPC224PLCg_RT Inositol phosphorylceramide  ceramide 2  24C  phospholipase C h2o_g + 0.01 ipc224_SC_g --> cer2_24_g + h_g + mi1p__D_g 15857
IPC226PLCg_RT Inositol phosphorylceramide  ceramide 2  26C  phospholipase C h2o_g + 0.01 ipc226_SC_g --> cer2_26_g + h_g + mi1p__D_g 15857
IPC324PLCg_RT Inositol phosphorylceramide  ceramide 3  24C  phospholipase C h2o_g + 0.01 ipc324_SC_g --> cer3_24_g + h_g + mi1p__D_g 15857
IPC326PLCg_RT Inositol phosphorylceramide  ceramide 3  26C  phospholipase C h2o_g + 0.01 ipc326

In [166]:
Annotation.loc[['15857','12385','14309','14089']]

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
15857,K12351: SMPD2; sphingomyelin phosphodiesterase 2,,ISC1,SMPD2,ISC1,SMPD2,Not Essential,"plas 14, mito 4, extr 4, cyto 3",WTR*
12385,K13333: PLB; lysophospholipase,S,"PLB2,PLB1,PLB3",,PLB1,,Not Essential,extr 27,LLL*
14309,"K14676: NTE, NRE; lysophospholipid hydrolase",A,NTE1,"PNPLA6,PNPLA7",NTE1,PNPLA,Not Essential,"extr 8, plas 7, cyto 6, nucl 2, mito 2, mito_n...",RLT*
14089,K19219: JMJD7; jumonji domain-containing prote...,,,JMJD7,,JMJD7,Not Essential,"mito 18.5, cyto_mito 12, cyto 4.5, nucl 2, pero 2",DEE*


In [167]:
model.remove_reactions(['PLPSA21801819Z','PLPSA21801829Z12Z','PLPSA21819Z1819Z','PLPSA21819Z1829Z12Z'], remove_orphans=True)

Remodeling  
SPO14	14023	nucl 18.5, cyto_nucl 12.5, cyto 5.5	K01115: PLD1_2; phospholipase D1/2	IYD*  

CDPDAG biosynthesis  
CDS1	12881	plas 21, E.R. 5	K00981: E2.7.7.41, CDS1, CDS2, cdsA; phosphatidate cytidylyltransferase	KPW*  
TAM41	11307	mito 21, nucl 2, cyto 2, cyto_nucl 2	K17807: TAM41, MMP37; mitochondrial translocator assembly and maintenance protein 41 (mito phosphotidate cytidylyltransferase)  
PI  
PIS1	15121	mito 11, extr 5, plas 4, E.R. 3, nucl 1, cyto 1, cyto_nucl 1, pero 1, golg 1, cyto_pero 1	K00999: CDIPT; CDP-diacylglycerol--inositol 3-phosphatidyltransferase	KKQ*  
TOR1,TOR2	12050	plas 11, cyto 7, nucl 4, pero 3, mito_nucl 3	K07203: MTOR, FRAP, TOR; serine/threonine-protein kinase mTOR	SSG*  
12050 is TOR, a homolog of PIK3  
3K nucl  
VPS34	8712	nucl 18, cyto_nucl 13.5, cyto 7	K00914: PIK3C3, VPS34; phosphatidylinositol 3-kinase	WRS*  
3,5K nucl  
FAB1	14139	nucl 13, cyto_nucl 11, cyto 7, mito 3, plas 1, pero 1, cysk 1, vacu 1	K00921: PIKFYVE, FAB1; 1-phosphatidylinositol-3-phosphate 5-kinase	AAF*  
4K E.R.  
STT4	15159	plas 10, nucl 4.5, cyto_nucl 4.5, mito 4, cyto 3.5, E.R. 2, vacu 2	K00888: PI4KA; phosphatidylinositol 4-kinase A	YVR*  
4K nucl  
PIK1	9782	extr 8, nucl 5.5, cyto_nucl 5.5, mito 5, cyto 4.5, pero 2	K19801: PI4KB; phosphatidylinositol 4-kinase B	ILI*  
LSB6	15086	nucl 17, cyto_nucl 13, cyto 7	K13711: PI4K2; phosphatidylinositol 4-kinase type 2	WPW*  
4,5K nucl  
MSS4	11878	mito 11, cyto 7.5, cyto_nucl 7, nucl 5.5	K00889: PIP5K; 1-phosphatidylinositol-4-phosphate 5-kinase	KVQ*  
3,5P, 4,5P E.R.  
INP52,INP53	15340	extr 10, cyto 4, mito 3, plas 3, cyto_nucl 3, nucl 2, E.R. 2	K20279: SYNJ; synaptojanin	LLS*  
3,5P E.R.  
SAC1	11381	plas 18, E.R. 5, vacu 2	KOG1889: Putative phosphoinositide phosphatase	KQE*  
FIG4	16619	mito 6, plas 6, nucl 5.5, cyto_nucl 5.5, cyto 4.5, pero 1, E.R. 1, golg 1, cysk 1, vacu 1	HMMPfam:SacI homology domain:PF02383,ProSiteProfiles:Sac phosphatase domain profile.:PS50275	AEG*  
4,5P nucl  
INP51	13609	nucl 16.5, cyto_nucl 13.333, cyto 9, cyto_mito 5.333	K01099: INPP5B_F; inositol polyphosphate 5-phosphatase INPP5B/F	LEE*  
INP54	15545	mito 8.5, cyto_mito 8, cyto 6.5, plas 5, nucl 4	KOG0565: Inositol polyphosphate 5-phosphatase and related proteins	ALL*  
3P nucl  
YMR1	14880	nucl 10.5, cyto_nucl 8.333, cyto 5, extr 5, cyto_mito 4.333, pero 3	KOG1089: Myotubularin-related phosphatidylinositol 3-phosphate 3-phosphatase MTM6	GLA*  
3P, 4P E.R.  
SAC1	11381	plas 18, E.R. 5, vacu 2	KOG1889: Putative phosphoinositide phosphatase	KQE*  
4,5 and 4 PL  
PLC1	12855	nucl 16, cyto_nucl 12, cyto 6, mito 2	K05857: PLCD; phosphatidylinositol phospholipase C, delta	ESS*  

PS  
CHO1	11583	mito 8, plas 7, extr 5, cyto 4, E.R. 2, cyto_pero 2	K17103: CHO1, pssA; CDP-diacylglycerol---serine O-phosphatidyltransferase	PKL*  

Cardiolipin biosynthesis (mito)  
PGS1	12596	mito 20, nucl 4, cyto 2	K00995: pgsA, PGS1; CDP-diacylglycerol--glycerol-3-phosphate 3-phosphatidyltransferase	TML*  
GEP4	10900	mito 18.5, cyto_mito 12.5, cyto 5.5	KOG2961: Predicted hydrolase (HAD superfamily)	VER*  
CRD1	16415	mito 18, extr 3, cyto 2, E.R. 2	K08744: CRLS; cardiolipin synthase (CMP-forming)	YIK*  
TAZ1	16474	mito 13, cyto_nucl 5, nucl 4.5, cyto 4.5, plas 2, pero 2	K13511: TAZ; monolysocardiolipin acyltransferase	TGW*  

EPT1	11351	plas 25	K00993: EPT1; ethanolaminephosphotransferase	GRA*  
CPT1	9088	plas 20, extr 3, E.R. 3	K00993: EPT1; ethanolaminephosphotransferase	KGE*  
OPI3	12748	extr 10, cyto 7, plas 6, mito 3	K00550: OPI3; methylene-fatty-acyl-phospholipid synthase	KEL*  
CHO2	10647	plas 21, E.R. 4	K16369: CHO2; phosphatidylethanolamine N-methyltransferase	KEG*  

PE  
14554	K01613: psd, PISD; phosphatidylserine decarbox...		PSD1 Essential	mito 12, plas 5, cyto 4, nucl 3, E.R. 2	IRQ*  
11446	K01613: psd, PISD; phosphatidylserine decarbox...		PSD2 Not Essential	nucl 14.5, cyto_nucl 13, cyto 8.5, mito 2	VGK*  
10504	K01613: psd, PISD; phosphatidylserine decarbox...		PSD2 Not Essential	mito 15, cyto 5, extr 2, nucl 1, plas 1, pero ...	AFV*

Uncharacterized  
9521	plas 16, mito 6, vacu 2, cyto 1, extr 1, E.R. 1	K13513: LCLAT1, AGPAT8; lysocardiolipin and lysophospholipid acyltransferase	TEL*  
human ortholog of 9521 has no activity for lysocardiolipin, but has lysoPI and lysoPG AT activities  
CST26	8422	plas 16, E.R. 5, vacu 3, mito 2	KOG1505: Lysophosphatidic acid acyltransferase LPAAT and related acyltransferases	VWR*  
PI

In [168]:
for x in ['14023','12881','15121','12050','8712','14139','15159','9782','15086','11878','15340',
          '11381','13609','15545','12855','11583','12596','10900','16415','16474',
          '11351','9088','12748','10647','14554','11446','10504']:
    for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
        print(r.id, r.reaction, r.gene_reaction_rule)
    print()

PCHOLP_hs h2o_c + pchol_hs_c --> chol_c + h_c + pa_hs_c 14023
PCHOLPg_hs h2o_g + pchol_hs_g --> chol_g + h_g + pa_hs_g 14023
PCHOLPm_hs h2o_m + pchol_hs_m --> chol_m + h_m + pa_hs_m 14023
PCHOLPr_hs h2o_r + pchol_hs_r --> chol_r + h_r + pa_hs_r 14023
PLD_SC h2o_c + 0.01 pc_SC_c --> chol_c + h_c + 0.01 pa_SC_c 14023
yli_R0318 100.0 h2o_c + pchol_cho_c --> 100.0 chol_c + 100.0 h_c + pa_EC_c 14023
yli_R0325 100.0 h2o_c + pe_SC_c --> 100.0 etha_c + 100.0 h_c + pa_EC_c 14023
yli_R1401 100.0 h2o_r + pchol_cho_r --> 100.0 chol_r + 100.0 h_r + pa_EC_r 14023
yli_R1406 100.0 h2o_r + pe_SC_r --> 100.0 etha_r + 100.0 h_r + pa_EC_r 14023

CDPDAGS1819Z160 ctp_c + 3.0 h_c + pa1819Z160_c --> cdp12dgr1819Z160_c + ppi_c 12881
CDPDAGS1819Z160h ctp_h + 2.0 h_h + pa1819Z160_h --> cdp12dgr1819Z160_h + ppi_h 12881
CDS ctp_c + h_c + pa_hs_c --> cdpdag_hs_c + ppi_c 12881
CDSm ctp_m + h_m + pa_hs_m --> cdpdag_hs_m + ppi_m 12881
DASYN_SC ctp_c + h_c + 0.01 pa_SC_c <=> 0.01 cdpdag_SC_c + ppi_c 12881
DASYNm_SC ctp

In [169]:
print('\''+'\',\''.join([r.id for r in sorted(model.genes.get_by_id('10504').reactions, key=lambda x: x.id)])+'\'')

'PSD120','PSD140','PSD141','PSD160','PSD161','PSD180','PSD181','PSERDg_SC','PSERDv_SC','yli_R0338','yli_R1411'


In [170]:
model.reactions.get_by_id('PIN4Kn_SC').id = 'PIN4Kn_RT'
model.reactions.get_by_id('CDPDGPm_SC').id = 'CDPDGPm_RT'
model.reactions.get_by_id('CDPDGPm_RT').lower_bound = 0.0
for x in ['DASYN_SC','PINOS_SC','PIN4K_SC','PI35BP5P_SC','PI45BP5P_SC','PSERS_SC','DAGCPT_SC','ETHAPT_SC',
          'PETOHM_SC','MFAPS_SC','PMETM_SC']:
    r = model.reactions.get_by_id(x)
    r.id = r.id.replace('_SC','er_RT')
    for m in r.metabolites:
        if not m.id.replace('_c','_r') in model.metabolites:
            m2 = m.copy()
            m2.id = m.id.replace('_c','_r')
            m2.compartment = 'r'
            model.add_metabolites([m2])
        r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_r'): r.get_coefficient(m.id)})
model.reactions.get_by_id('DASYNer_RT').lower_bound = 0.0
model.reactions.get_by_id('DASYNm_SC').id = 'DASYNm_RT'
model.reactions.get_by_id('DASYNm_RT').lower_bound = 0.0
model.reactions.get_by_id('DASYNm_RT').gene_reaction_rule = '11307'
model.reactions.get_by_id('ETHAPTer_RT').lower_bound = 0.0
model.reactions.get_by_id('PSERSer_RT').lower_bound = 0.0
model.reactions.get_by_id('PSERSm_SC').id = 'PSERSm_RT'
model.reactions.get_by_id('PSERSm_RT').lower_bound = 0.0

r = sce.reactions.get_by_id('PI45BP5P_SC').copy()
model.add_reactions([r])
for x in ['PLD_SC','PIN3K_SC','PI3P5K_SC','PI4P5K_SC','PI45BP5P_SC','PI45BPP_SC']:
    r = model.reactions.get_by_id(x)
    r.id = r.id.replace('_SC','n_RT')
    for m in r.metabolites:
        if not m.id.replace('_c','_n') in model.metabolites:
            m2 = m.copy()
            m2.id = m.id.replace('_c','_n')
            m2.compartment = 'n'
            model.add_metabolites([m2])
        r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_n'): r.get_coefficient(m.id)})
model.reactions.get_by_id('PIN3Kn_RT').gene_reaction_rule = '8712'
model.reactions.get_by_id('PIN4Ker_RT').gene_reaction_rule = '15159'
model.reactions.get_by_id('PIN4Kn_RT').gene_reaction_rule = '15086 or 9782'
model.reactions.get_by_id('PI35BP5Per_RT').gene_reaction_rule = '11381 or 15340 or 16619'
model.reactions.get_by_id('PI45BP5Per_RT').gene_reaction_rule = '15340'
model.reactions.get_by_id('PI45BP5Pn_RT').gene_reaction_rule = '13609 or 15545'
model.reactions.get_by_id('ETHAPTer_RT').gene_reaction_rule = '11351'

# Cardiolipin biosynthesis in mito
# http://jcb.rupress.org/content/192/1/7

r = hsa.reactions.get_by_id('GLYCtm').copy()
r.gene_reaction_rule = ''
model.add_reactions([r])

r = sce.reactions.get_by_id('PGPPAm_SC').copy()
r.gene_reaction_rule = '10900'
model.add_reactions([r])

r = hsa.reactions.get_by_id('GLYCtm').copy()
r.id = 'PAterm_RT'
r.name = 'Phosphatidate transport, from E.R. to mitochondria'
r.gene_reaction_rule = ''
model.add_reactions([r])
r.add_metabolites({'glyc_c': 1.0, 'glyc_m': -1.0, 'pa_SC_r': -1.0, 'pa_SC_m': 1.0})

r = hsa.reactions.get_by_id('GLYCtm').copy()
r.id = 'CDPDAGterm_RT'
r.name = 'CDP diacylglycerol transport, from E.R. to mitochondria'
r.gene_reaction_rule = ''
model.add_reactions([r])
r.add_metabolites({'glyc_c': 1.0, 'glyc_m': -1.0, 'cdpdag_SC_r': -1.0, 'cdpdag_SC_m': 1.0})

r = hsa.reactions.get_by_id('GLYCtm').copy()
r.id = 'PSterm_RT'
r.name = 'Phosphatidylserine transport, from E.R. to mitochondria'
r.gene_reaction_rule = ''
model.add_reactions([r])
r.add_metabolites({'glyc_c': 1.0, 'glyc_m': -1.0, 'ps_SC_r': -1.0, 'ps_SC_m': 1.0})

r = hsa.reactions.get_by_id('GLYCtm').copy()
r.id = 'PEterm_RT'
r.name = 'Phosphatidylethanolamine transport, from E.R. to mitochondria'
r.gene_reaction_rule = ''
model.add_reactions([r])
r.add_metabolites({'glyc_c': 1.0, 'glyc_m': -1.0, 'pe_SC_r': -1.0, 'pe_SC_m': 1.0})

# PI in E.R. and nucl

r = model.reactions.get_by_id('PI35BP5Per_RT').copy()
r.id = 'PIN3Per_RT'
r.gene_reaction_rule = '11381'
model.add_reactions([r])
model.reactions.get_by_id('PIN3Per_RT').add_metabolites({'ptd135bp_SC_r': 0.01, 'ptd3ino_SC_r': -0.02,
                                                         'ptd1ino_SC_r': 0.01})
r = model.reactions.get_by_id('PI45BP5Per_RT').copy()
r.id = 'PIN4Per_RT'
r.gene_reaction_rule = '11381'
model.add_reactions([r])
model.reactions.get_by_id('PIN4Per_RT').add_metabolites({'ptd145bp_SC_r': 0.01, 'ptd4ino_SC_r': -0.02,
                                                         'ptd1ino_SC_r': 0.01})
r = model.reactions.get_by_id('PI45BP5Pn_RT').copy()
r.id = 'PIN3Pn_RT'
r.gene_reaction_rule = '14880'
model.add_reactions([r])
model.reactions.get_by_id('PIN3Pn_RT').add_metabolites({'ptd145bp_SC_n': 0.01, 'ptd4ino_SC_n': -0.01,
                                                        'ptd3ino_SC_n': -0.01, 'ptd1ino_SC_n': 0.01})

model.reactions.get_by_id('PSERDg_SC').id = 'PSERDg_RT'
model.reactions.get_by_id('PSERDg_RT').gene_reaction_rule = '10504 or 11446'
model.reactions.get_by_id('PSERDv_SC').id = 'PSERDv_RT'
model.reactions.get_by_id('PSERDv_RT').gene_reaction_rule = '10504 or 11446'

r = model.reactions.get_by_id('PSERDm_SC').copy()
r.id = 'PSERDer_RT'
r.name = 'Phosphatidylserine decarboxylase yeast specific'
model.add_reactions([r])
for m in r.metabolites:
    if not m.id.replace('_m','_r') in model.metabolites:
        m2 = m.copy()
        m2.id = m.id.replace('_m','_r')
        m2.compartment = 'r'
        model.add_metabolites([m2])
    r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_m','_r'): r.get_coefficient(m.id)})

temp = ['PCHOLP_hs','PCHOLPg_hs','PCHOLPm_hs','PCHOLPr_hs','yli_R0318','yli_R0325','yli_R1401','yli_R1406',
        'CDPDAGS1819Z160','CDPDAGS1819Z160h','CDS','CDSm','yli_R0337','yli_R1410','CDIPT1819Z160',
        'CDIPTr','yli_R0346','yli_R1418','PI5P3K','PIK3','yli_R0352','PI4P5K','yli_R0355','PI3P4K','PI5P4K','PIK4',
        'yli_R0351','PI34P5K','PI3P5K','PIK5','yli_R0354','PI345P5P','PI45P5P','PI45PLC','PI4PLC','PIPLC','yli_R0353',
        'PSSA120','PSSA160','PSSA161','PSSA180','PSSA181','yli_R0334','yli_R1407','PGPPT','PSSA140','PSSA141',
        'yli_R0344','yli_R1416','yli_R1524','CLS_hs','PGPS1819Z160','PGPS1819Z160h','PGSA120','PGSA140','PGSA141',
        'PGSA160','PGSA161','PGSA180','PGSA181','yli_R0345','yli_R1417','CEPTC','CEPTE','ETHAPT1801819Z',
        'ETHAPT1819Z1819Z','yli_R0341','yli_R0342','yli_R1413','yli_R1414','PETOHMm_hs','PETOHMr_hs','yli_R0893',
        'yli_R1471','yli_R0339','yli_R1412','PSDm_hs','PSD120','PSD140','PSD141','PSD160','PSD161','PSD180','PSD181',
        'yli_R0338','yli_R1411']
model.remove_reactions(temp, remove_orphans=True)

16474 is Taz1 mitochondrial transacylase with substrate specificity for either lyso-PC or monolyso-CL  
In yeast, CLD1 mitochondrial cardiolipin-specific phospholipase deacylates cardiolipin, and  
Taz1 reacylates monolysocardiolipin with unsaturated fatty acids  
http://www.jbc.org/content/284/17/11572.long  
LPCATm_RT is incorrect, and no substrate specificity is known for this enzyme in R. toruloides  
Mitochondrial phospholipid/fatty acid composition is also unknown, remove LPCATm_RT for now

In [171]:
model.remove_reactions(['LPCATm_RT'], remove_orphans=True)

In [172]:
Annotation.loc[['9763','13609','15340','13260','15600']]

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
9763,"K07756: IP6K, IHPK; inositol-hexakisphosphate ...",,,"IP6K3,IP6K1",KCS1,IP6K3,Not Essential,"nucl 11.5, mito 9, cyto_nucl 8.5, cyto 4.5",STA*
13609,K01099: INPP5B_F; inositol polyphosphate 5-pho...,,,"INPP5B,OCRL",INP51,OCRL,Not Essential,"nucl 16.5, cyto_nucl 13.333, cyto 9, cyto_mito...",LEE*
15340,K20279: SYNJ; synaptojanin,,"INP52,INP53,INP51","SYNJ1,SYNJ2",INP53,SYNJ1,Not Essential,"extr 10, cyto 4, mito 3, plas 3, cyto_nucl 3, ...",LLS*
13260,"K00915: IPMK, IPK2; inositol-polyphosphate mul...",,ARG82,,ARG82,IP6K3,Essential,"cyto 19, cyto_nucl 11, extr 3, mito 2",EEA*
15600,K10572: IPPK; inositol-pentakisphosphate 2-kinase,,,IPPK,,IPPK,Essential,"cyto 18, nucl 5, mito 2, pero 2",SEA*


There are many reactions involving inositol phosphates in cyto  
Should move some of them to nucl?  
MI1345PP h2o_c + mi1345p_c --> mi134p_c + pi_c 13609 or 15340  

AMITKP: atp_c + h_c + mi1456p_c --> adp_c + mi13456p_c 9763 (wrong)  
AMITP: atp_c + mi145p_c --> adp_c + h_c + mi1456p_c 9763 (correct, but cyto)  
ITKK: atp_c + h_c + mi1346p_c --> adp_c + mi13456p_c 9763 (wrong, mi1346 in plant not yeast)  
PMI1346PS: atp_c + mi13456p_c --> adp_c + ppmi1346p_c 9763 (this rxn by IPMK)  
PMI12346PS: atp_c + minohp_c --> adp_c + ppmi12346p_c 9763  
PMI12346PH: h2o_c + ppmi12346p_c --> h_c + minohp_c + pi_c 9763 (wrong, 9763 is kinase)  
PMI1346PH: h2o_c + ppmi1346p_c --> h_c + mi13456p_c + pi_c 9763 (wrong, 9763 is kinase)  
MI1345PP: h2o_c + mi1345p_c --> mi134p_c + pi_c 13609 or 15340
MI145PP: h2o_c + mi145p_c --> mi14p_c + pi_c 13609 or 15340

AMITKPn: atp_n + h_n + mi1456p_n --> adp_n + mi13456p_n 9763 (wrong)  
MI145PKn: atp_n + mi145p_n --> adp_n + h_n + mi1345p_n 13260  
MI1345PKn: atp_n + mi1345p_n --> adp_n + h_n + mi13456p_n 13260  
MI145P6Kn: atp_n + mi145p_n --> adp_n + h_n + mi1456p_n 13260 or 9763  
MI1456PKn: atp_n + mi1456p_n --> adp_n + h_n + mi13456p_n 13260  
MI13456PKn: atp_n + mi13456p_n --> adp_n + h_n + minohp_n 15600  
PMI1346PHn: h2o_n + ppmi1346p_n --> h_n + mi13456p_n + pi_n 9763 (wrong, 9763 kinase)  
PMI12346PHn: h2o_n + ppmi12346p_n --> h_n + minohp_n + pi_n 9763 (wrong, 9763 kinase)

MI134PP: h2o_c + mi134p_c --> mi34p_c + pi_c 10673 (wrong, 10673 is cysQ)

In [173]:
for x in ['9763','13609','15340','13260','15600']:
    for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
        print(r.id, r.reaction, r.gene_reaction_rule)
    print()

AMITKP atp_c + h_c + mi1456p_c --> adp_c + mi13456p_c 9763
AMITKPn atp_n + h_n + mi1456p_n --> adp_n + mi13456p_n 9763
AMITP atp_c + mi145p_c --> adp_c + h_c + mi1456p_c 9763
ITKK atp_c + h_c + mi1346p_c --> adp_c + mi13456p_c 9763
MI145P6Kn atp_n + mi145p_n --> adp_n + h_n + mi1456p_n 13260 or 9763
PMI12346PH h2o_c + ppmi12346p_c --> h_c + minohp_c + pi_c 9763
PMI12346PHn h2o_n + ppmi12346p_n --> h_n + minohp_n + pi_n 9763
PMI12346PS atp_c + minohp_c --> adp_c + ppmi12346p_c 9763
PMI1346PH h2o_c + ppmi1346p_c --> h_c + mi13456p_c + pi_c 9763
PMI1346PHn h2o_n + ppmi1346p_n --> h_n + mi13456p_n + pi_n 9763
PMI1346PS atp_c + mi13456p_c --> adp_c + ppmi1346p_c 9763

MI1345PP h2o_c + mi1345p_c --> mi134p_c + pi_c 13609 or 15340
MI145PP h2o_c + mi145p_c --> mi14p_c + pi_c 13609 or 15340
PI45BP5Pn_RT h2o_n + 0.01 ptd145bp_SC_n --> pi_n + 0.01 ptd4ino_SC_n 13609 or 15545

MI1345PP h2o_c + mi1345p_c --> mi134p_c + pi_c 13609 or 15340
MI145PP h2o_c + mi145p_c --> mi14p_c + pi_c 13609 or 15340
P

In [174]:
model.reactions.get_by_id('MI145P6Kn').gene_reaction_rule = '13260'

for x in ['PMI12346PS','MI1345PP','MI145PP']:
    r = model.reactions.get_by_id(x)
    r.id = r.id+'n'
    for m in r.metabolites:
        if not m.id.replace('_c','_n') in model.metabolites:
            m2 = m.copy()
            m2.id = m.id.replace('_c','_n')
            m2.compartment = 'n'
            model.add_metabolites([m2])
        r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_n'): r.get_coefficient(m.id)})

temp = ['AMITKP','AMITKPn','AMITP','ITKK','PMI12346PH','PMI12346PHn','PMI1346PH','PMI1346PHn','PMI1346PS','MI134PP']
model.remove_reactions(temp, remove_orphans=True)

In [175]:
temp = Annotation.index[Annotation['Combined Annotations'].str.contains('lysophospholipase')]
#temp = Annotation.index[Annotation['Sc288c Orthologs'].str.contains('GPD1')]
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
8853,K13278: ASPG; 60kDa lysophospholipase,,,ASPG,,ASPG,Not Essential,"mito 10, cyto 7, mito_nucl 7, nucl 4",ANS*
9631,K06130: LYPLA2; lysophospholipase II,S,YLR118C,"LYPLA1,LYPLA2",YLR118C,LYPLA,Not Essential,"mito 11, extr 11, cyto 4.5, cyto_nucl 3",PSS*
12385,K13333: PLB; lysophospholipase,S,"PLB2,PLB1,PLB3",,PLB1,,Not Essential,extr 27,LLL*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
8853,0.273689,0.140262,0.370329,0.1627,0.395863,0.0424996,0.210741,0.248432,0.145345,-0.179415,0.0611695
9631,0.0650912,-0.239941,-0.0608276,-0.0299206,0.554819,-0.209271,-0.0284215,0.35474,0.210049,0.00650262,-0.747011
12385,0.168375,0.221694,0.0416109,0.0721808,-0.572405,0.258112,-0.0124852,0.0609392,-0.0540949,0.00780372,0.358722


In [176]:
for x in ['8853','9631','12385']:
    for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
        print(r.id, r.reaction, r.gene_reaction_rule)
    print()

ASNN asn__L_c + h2o_c --> asp__L_c + nh4_c 10043 or 13627 or 8853
ASPSE asn__L_x + h2o_x + h_x --> asp__L_x + nh4_x 8853

LIDOAMH h2o_e + lido_e --> 26dmani_e + degly_e + h_e 9631
MEGLYXYLAMH h2o_e + meglyxyl_e --> 26dmani_e + egly_e + h_e 9631

PLBP1Ie_RT h2o_e + 0.005 ptd1ino_SC_e --> 0.02 dca_e + 0.06 ddca_e + 0.5 g3pi_e + h_e + 0.27 hdca_e + 0.17 hdcea_e + 0.05 ocdca_e + 0.24 ocdcea_e + 0.09 ocdcya_e + 0.1 ttdca_e 12385
PLBPCe_RT h2o_e + 0.005 pc_SC_e --> 0.02 dca_e + 0.06 ddca_e + 0.5 g3pc_e + h_e + 0.27 hdca_e + 0.17 hdcea_e + 0.05 ocdca_e + 0.24 ocdcea_e + 0.09 ocdcya_e + 0.1 ttdca_e 12385
PLBPEe_RT h2o_e + 0.005 pe_SC_e --> 0.02 dca_e + 0.06 ddca_e + 0.5 g3pe_e + h_e + 0.27 hdca_e + 0.17 hdcea_e + 0.05 ocdca_e + 0.24 ocdcea_e + 0.09 ocdcya_e + 0.1 ttdca_e 12385



In [177]:
temp = Annotation.index[Annotation['Combined Annotations'].str.contains('asparaginase')]
#temp = Annotation.index[Annotation['Sc288c Orthologs'].str.contains('GPD1')]
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
10043,"K01424: E3.5.1.1, ansA, ansB; L-asparaginase",S,"YLR155C,ASP3-2,ASP3-3,ASP3-4,ASP1",,ASP3-4,,Not Essential,extr 26,SYY*
13627,"K01424: E3.5.1.1, ansA, ansB; L-asparaginase",S,"YLR155C,ASP3-2,ASP3-3,ASP3-4,ASP1",,ASP3-4,,Not Essential,"extr 24, cyto 2",GSS*
16759,HMMPfam:Peptide N-acetyl-beta-D-glucosaminyl a...,S,,,,,Not Essential,"extr 21, E.R. 3, mito 1, cyto 1, plas 1, cyto_...",RPG*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
10043,-0.00620802,0.188284,-0.110655,0.0219904,0.220112,0.0301343,-0.306927,-0.433964,0.0783407,0.208705,-0.052839
13627,-0.433261,-0.175238,-0.492188,-0.24719,-0.769979,-0.291801,-0.458407,-0.100953,-0.29205,-0.431563,-0.289214
16759,0.000805289,-0.115643,-0.0208313,0.117972,-0.154245,0.109194,-0.0966286,-0.0576095,-0.252818,-0.174171,-0.0749792


In [178]:
for r in sorted(model.metabolites.get_by_id('asn__L_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('asp__L_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('lido_e').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('26dmani_e').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('degly_e').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('egly_e').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

ASNtx asn__L_c + h_c <=> asn__L_x + 2.0 h_x 16187
ASPSE asn__L_x + h2o_x + h_x --> asp__L_x + nh4_x 8853

ASPSE asn__L_x + h2o_x + h_x --> asp__L_x + nh4_x 8853
ASPTAp akg_x + asp__L_x <=> glu__L_x + oaa_x 8936
ASPtx asp__L_x + h_x <=> asp__L_c + h_c 8766

LIDOAMH h2o_e + lido_e --> 26dmani_e + degly_e + h_e 9631

LIDOAMH h2o_e + lido_e --> 26dmani_e + degly_e + h_e 9631
MEGLYXYLAMH h2o_e + meglyxyl_e --> 26dmani_e + egly_e + h_e 9631

LIDOAMH h2o_e + lido_e --> 26dmani_e + degly_e + h_e 9631

MEGLYXYLAMH h2o_e + meglyxyl_e --> 26dmani_e + egly_e + h_e 9631


In [179]:
# 8853 ASPG has asparaginase activity as well as lysophospholipase and 
# 9631 LYPLA2 hydrolyzes fatty acids from S-acylated cysteine residues in proteins
model.remove_reactions(['ASPSE','LIDOAMH','MEGLYXYLAMH'], remove_orphans=True)

In [180]:
for r in sorted(model.metabolites.get_by_id('fadh2_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('fadh2_m').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

I2FE2SS fadh2_c + 2.0 fe2_c + 2.0 iscssh_c + iscu_c --> fad_c + 6.0 h_c + 2.0 iscs_c + iscu_2fe2s_c 10285 and 13740
I2FE2SS2 fadh2_c + 2.0 fe2_c + 2.0 iscssh_c + iscu_2fe2s_c --> fad_c + 6.0 h_c + 2.0 iscs_c + iscu_2fe2s2_c 10285 and 13740
I4FE4SR fadh2_c + 2.0 h_c + iscu_2fe2s2_c --> fad_c + iscu_4fe4s_c 10285

ACOAD10m 2mbcoa_m + fad_m --> 2mb2coa_m + fadh2_m 14070
ACOAD1m btcoa_m + fad_m --> b2coa_m + fadh2_m 12570 or 14070
ACOAD2m fad_m + hxcoa_m --> fadh2_m + hx2coa_m 12570 or 14070
ACOAD3m fad_m + occoa_m --> fadh2_m + oc2coa_m 12570
ACOAD4m dcacoa_m + fad_m --> dc2coa_m + fadh2_m 12570
ACOAD5m ddcacoa_m + fad_m --> dd2coa_m + fadh2_m 12570
ACOAD6m fad_m + tdcoa_m --> fadh2_m + td2coa_m 12570
ACOAD7m fad_m + pmtcoa_m --> fadh2_m + hdd2coa_m 12570
ACOAD8m fad_m + ivcoa_m --> 3mb2coa_m + fadh2_m 10012
ACOAD9m fad_m + ibcoa_m --> 2mp2coa_m + fadh2_m 14070
AHGDxm fad_m + s2hglut_m <=> akg_m + fadh2_m 10251
ARHGDxm fad_m + r2hglut_m <=> akg_m + fadh2_m 16643
CHOLD2m chol_m + fad_m -->

In [181]:
temp = ['10285','13740','11971']
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
10285,"K04488: iscU, nifU; nitrogen fixation protein ...",,"ISU2,ISU1",ISCU,ISU2,ISCU,Essential,"mito 21, cyto 6",GAV*
13740,"K04487: iscS, NFS1; cysteine desulfurase",S,NFS1,NFS1,NFS1,NFS1,Essential,"mito 22.5, cyto_mito 14, cyto 4.5",TQH*
11971,"K13628: iscA, ISCA1; iron-sulfur cluster assem...",S,ISA1,ISCA1,ISA1,ISCA1,Essential,"mito 24, nucl 2",FIV*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,


10285	mito 21, cyto 6	K04488: iscU, nifU; nitrogen fixation protein NifU and related proteins	GAV*  
13740	mito 22.5, cyto_mito 14, cyto 4.5	K04487: iscS, NFS1; cysteine desulfurase	TQH*  
11971	mito 24, nucl 2	K13628: iscA, ISCA1; iron-sulfur cluster assembly protein	FIV*

In [182]:
for r in sorted(model.genes.get_by_id('10285').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('13740').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.genes.get_by_id('11971').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

I2FE2SR 2fe1s_c + iscssh_c + iscu_c --> 4.0 h_c + iscs_c + iscu_2fe2s_c 10285 and 13740
I2FE2SS fadh2_c + 2.0 fe2_c + 2.0 iscssh_c + iscu_c --> fad_c + 6.0 h_c + 2.0 iscs_c + iscu_2fe2s_c 10285 and 13740
I2FE2SS2 fadh2_c + 2.0 fe2_c + 2.0 iscssh_c + iscu_2fe2s_c --> fad_c + 6.0 h_c + 2.0 iscs_c + iscu_2fe2s2_c 10285 and 13740
I2FE2ST 4.0 h_c + iscu_2fe2s_c --> 2fe2s_c + iscu_c 10285 and 11971
I4FE4SR fadh2_c + 2.0 h_c + iscu_2fe2s2_c --> fad_c + iscu_4fe4s_c 10285
I4FE4ST 4.0 h_c + iscu_4fe4s_c --> 4fe4s_c + iscu_c 10285 and 11971

I2FE2SR 2fe1s_c + iscssh_c + iscu_c --> 4.0 h_c + iscs_c + iscu_2fe2s_c 10285 and 13740
I2FE2SS fadh2_c + 2.0 fe2_c + 2.0 iscssh_c + iscu_c --> fad_c + 6.0 h_c + 2.0 iscs_c + iscu_2fe2s_c 10285 and 13740
I2FE2SS2 fadh2_c + 2.0 fe2_c + 2.0 iscssh_c + iscu_2fe2s_c --> fad_c + 6.0 h_c + 2.0 iscs_c + iscu_2fe2s2_c 10285 and 13740
ICYSDS cys__L_c + iscs_c --> ala__L_c + iscssh_c 13740
MOADSUx iscssh_c + moadamp_c + nadh_c --> amp_c + iscs_c + moadcosh_c + nad_c b

In [183]:
for x in ['I2FE2SR','I2FE2SS','I2FE2SS2','I2FE2ST','I4FE4SR','I4FE4ST','ICYSDS']:
    r = model.reactions.get_by_id(x)
    r.id = r.id + 'm'
    for m in r.metabolites:
        if not m.id.replace('_c','_m') in model.metabolites:
            m2 = m.copy()
            m2.id = m.id.replace('_c','_m')
            m2.compartment = 'm'
            model.add_metabolites([m2])
        r.add_metabolites({m.id: -r.get_coefficient(m.id), m.id.replace('_c','_m'): r.get_coefficient(m.id)})
model.remove_reactions(['MOADSUx','THZPSN3'], remove_orphans=True)

In [184]:
Annotation.loc[['14055']]

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
14055,"K08764: SCP2, SCPX; sterol carrier protein 2",,,SCP2,,SCP2,Not Essential,"cyto_mito 13.333, mito 13, cyto 11.5, cyto_nuc...",ANL*


In [185]:
# SCP2 reactions
for r in sorted(model.genes.get_by_id('14055').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

SCP21x phytcoa_c <=> phytcoa_x 14055
SCP22x dmnoncoa_c <=> dmnoncoa_x 14055
SCP2x coa_x + dhocholoylcoa_x --> dgcholcoa_x + ppcoa_x 14055
SCP3x coa_x + dhcholestancoa_x + o2_x --> dcholcoa_x + h2o_x + ppcoa_x 14055
SCPx cholcoaone_x + coa_x --> cholcoa_x + ppcoa_x 14055


In [186]:
for r in sorted(model.metabolites.get_by_id('phytcoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('phytcoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('dmnoncoa_c').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('dmnoncoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('dhcholestanate_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('dhcholestancoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('dhcholoylcoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('thcholoylcoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('dhocholoylcoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('dgcholcoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('dcholcoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()

for r in sorted(model.metabolites.get_by_id('thcholstoic_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('cholcoar_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('cholcoas_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('cholcoads_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('cholcoaone_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)
print()
for r in sorted(model.metabolites.get_by_id('cholcoa_x').reactions, key=lambda x: x.id):
    print(r.id, r.reaction, r.gene_reaction_rule)

FACOAL206 atp_c + coa_c + phyt_c --> amp_c + phytcoa_c + ppi_c 11167 or 15746 or 15748
SCP21x phytcoa_c <=> phytcoa_x 14055

SCP21x phytcoa_c <=> phytcoa_x 14055

SCP22x dmnoncoa_c <=> dmnoncoa_x 14055

CSNAT2x crn_x + dmnoncoa_x <=> coa_x + dmnoncrn_x 14245
SCP22x dmnoncoa_c <=> dmnoncoa_x 14055

VLCS2p atp_x + coa_x + dhcholestanate_x --> amp_x + dhcholestancoa_x + ppi_x 9912

AMACR2p dhcholestancoa_x + 0.5 o2_x --> dhcholoylcoa_x + h2o_x 8616
SCP3x coa_x + dhcholestancoa_x + o2_x --> dcholcoa_x + h2o_x + ppcoa_x 14055
VLCS2p atp_x + coa_x + dhcholestanate_x --> amp_x + dhcholestancoa_x + ppi_x 9912

ACOX22x dhcholoylcoa_x + fadh2_x + 0.5 o2_x --> fad_x + thcholoylcoa_x 12742 or 12752 or 9700
AMACR2p dhcholestancoa_x + 0.5 o2_x --> dhcholoylcoa_x + h2o_x 8616

ACOX22x dhcholoylcoa_x + fadh2_x + 0.5 o2_x --> fad_x + thcholoylcoa_x 12742 or 12752 or 9700
HSD17B42x h_x + nadph_x + o2_x + thcholoylcoa_x --> dhocholoylcoa_x + 2.0 h2o_x + nadp_x 11362

HSD17B42x h_x + nadph_x + o2_x + thch

In [187]:
temp = ['14245','9912','8616','11362','12742']
Annotation.loc[temp]

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
14245,K00624: E2.3.1.7; carnitine O-acetyltransferase,,CAT2,CRAT,CAT2,CRAT,Not Essential,"nucl 11.5, cyto_nucl 8, pero 6, mito 5, cyto 3.5",AKL*
9912,"K08745: SLC27A1_4, FATP1, FATP4; solute carrie...",S,FAT1,"SLC27A1,SLC27A4,SLC27A2,SLC27A5,SLC27A6",FAT1,SLC27,Not Essential,"extr 12, plas 8, pero 4, mito 1, cyto 1, E.R. ...",VRL*
8616,"K01796: E5.1.99.4, AMACR, mcr; alpha-methylacy...",S,,AMACR,,AMACR,Not Essential,"extr 10, cyto 5, mito 4, cyto_pero 4, pero 3, ...",SKL*
11362,K14729: FOX2; multifunctional beta-oxidation p...,,FOX2,HSD17B4,FOX2,HSD17,Not Essential,"cyto 15, mito 6, pero 4, mito_nucl 4",LAQ*
12742,"K00232: E1.3.3.6, ACOX1, ACOX3; acyl-CoA oxidase",,POX1,"ACOX1,ACOX2",POX1,ACOX1,Not Essential,"cyto 7, pero 6, cysk 6, nucl 5, cyto_mito 5",SKL*


In [188]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r.id, r.reaction, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

CSNAT2m coa_m + pcrn_m <=> crn_m + ppcoa_m 14245
CSNAT2x crn_x + dmnoncoa_x <=> coa_x + dmnoncrn_x 14245
CSNAT3x crn_x + ppcoa_x <=> coa_x + pcrn_x 14245
CSNATer accoa_r + crn_r <=> acrn_r + coa_r 14245
CSNATirm accoa_m + crn_m <=> acrn_m + coa_m 13580 or 14245
CSNATm acrn_m + coa_m <=> accoa_m + crn_m 14245
CSNATp accoa_x + crn_x <=> acrn_x + coa_x 14245
CSNATr accoa_c + crn_c <=> acrn_c + coa_c 13580 or 14245

FA240tp ttc_c --> ttc_x 9912
FA260tp hexc_c --> hexc_x 9912
FACOAL240p atp_x + coa_x + ttc_x --> amp_x + ppi_x + ttccoa_x 9912
FACOAL260p atp_x + coa_x + hexc_x --> amp_x + hexccoa_x + ppi_x 9912
FATP1t hdca_c + na1_c <=> hdca_e + na1_e 9912
FATP2t na1_c + ocdcea_c <=> na1_e + ocdcea_e 9912
FATP3t na1_c + ocdca_c <=> na1_e + ocdca_e 9912
FATP4t arach_c + na1_c <=> arach_e + na1_e 9912
FATP8t na1_c + ttc_c <=> na1_e + ttc_e 9912
FATP9t na1_c + nrvnc_c <=> na1_e + nrvnc_e 9912
VLCS2p atp_x + coa_x + dhcholestanate_x --> amp_x + dhcholestancoa_x + ppi_x 9912
VLCS2r atp_r + coa_r +

In [189]:
# Remove bile acid and estrogen/testosterone/cholesterol reactions?
# Leave them for now
#remove = ['VLCS2p','VLCS2r','VLCSp','VLCSr','AMACR2p','AMACR2r','AMACRp','AMACRr','ACOX22x','ACOX2x',
#          'HSD17B42x','HSD17B4x','HSB17B1','HSD17B2r','SCPx','SCP2x','SCP3x']
#model.remove_reactions(remove, remove_orphans=True)
remove = ['yli_R0200','yli_R0204','yli_R0208','yli_R0212']
model.remove_reactions(remove, remove_orphans=True)

In [190]:
# HSD17B42x and HSD17B4x related genes
Annotation.loc[['12564','11057','12565','13289']]

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
12564,K12343: SRD5A1; 3-oxo-5-alpha-steroid 4-dehydr...,,,SRD5A1,,,Not Essential,"mito 11, cyto 6.5, cyto_nucl 5.5, extr 4, nucl...",GVY*
11057,"K07422: CYP2U1; cytochrome P450, family 2, sub...",S,,"CYP1A1,CYP1A2,CYP2A13,CYP2A6,CYP2A7,CYP2B6,CYP...",,CYP2U,Not Essential,"mito 7, cyto 5, extr 5, plas 3, cyto_nucl 3, p...",AGK*
12565,K00512: CYP17A; steroid 17alpha-monooxygenase ...,S,,"CYP1A1,CYP1A2,CYP2A13,CYP2A6,CYP2A7,CYP2B6,CYP...",,CYP2U,Not Essential,"cyto 9, mito 5, extr 5, cyto_nucl 5, E.R. 4",YEV*
13289,"K12156: CYP83A1; cytochrome P450, family 83, s...",,,"CYP1A1,CYP1A2,CYP2A13,CYP2A6,CYP2A7,CYP2B6,CYP...",,CYP21,Not Essential,"cyto 11, mito 7, pero 5, nucl 4",FEE*


12564 SRD5A1 3-oxo-5-alpha-steroid 4-dehydrogenase 1 converts testosterone into 5-alpha-dihydrotestosterone and progesterone or corticosterone into their corresponding 5-alpha-3-oxosteroids  
11057 CYP2U1 catalyzes the hydroxylation of arachidonic acid, docosahexaenoic acid and other long chain fatty acids  
12565 CYP17A steroid 17-alpha-hydroxylase/17,20 lyase monooxygenases that catalyze synthesis of cholesterol, steroids and other lipids and are involved in drug metabolism, has both 17α-hydroxylase activity and 17,20-lyase activity  
13289 CYP83A1 metabolize aldoximes and produce corresponding thiohydroxamic acids, efficient in metabolizingaldoximes derived from chain-elongated methionine

In [191]:
for x in ['12564','11057','12565','13289']:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r.id, r.reaction, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

SR5AR2r andrstndn_r + h_r + nadph_r --> andrstandn_r + nadp_r 12564
SR5ARr h_r + nadph_r + tststerone_r --> 5adtststerone_r + nadp_r 12564

ACTLMO acetol_c + h_c + nadph_c + o2_c --> 2.0 h2o_c + mthgxl_c + nadp_c 11057 or 12565 or 13289
ACTNMO acetone_c + h_c + nadph_c + o2_c --> acetol_c + h2o_c + nadp_c 11057 or 12565 or 13289
ETOHMO etoh_c + h_c + nadph_c + o2_c --> acald_c + 2.0 h2o_c + nadp_c 11057 or 12565 or 13289
MELATNOX h_c + melatn_c + nadph_c + o2_c --> 6hoxmelatn_c + h2o_c + nadp_c 11057 or 12565 or 13289
P45017A1r h_r + nadph_r + o2_r + prgnlone_r --> 17ahprgnlone_r + h2o_r + nadp_r 11057 or 12565 or 13289
P45017A2r 17ahprgnlone_r + h_r + nadph_r + 0.5 o2_r --> acald_r + dhea_r + h2o_r + nadp_r 11057 or 12565 or 13289
P45017A3r h_r + nadph_r + o2_r + prgstrn_r --> 17ahprgstrn_r + h2o_r + nadp_r 11057 or 12565 or 13289
P45017A4r 17ahprgstrn_r + h_r + nadph_r + 0.5 o2_r --> acald_r + andrstndn_r + h2o_r + nadp_r 11057 or 12565 or 13289
P4501B1r estradiol_r + h_r + nadph_r +

In [192]:
model.reactions.get_by_id('P45017A1r').gene_reaction_rule = '12565'
model.reactions.get_by_id('P45017A2r').gene_reaction_rule = '12565'
model.reactions.get_by_id('P45017A3r').gene_reaction_rule = '12565'
model.reactions.get_by_id('P45017A4r').gene_reaction_rule = '12565'

remove = ['ACTLMO','ACTNMO','ETOHMO','MELATNOX','P4501B1r','P4502A6','P4502C18','P4502C19','P4502C8','P4502C9',
          'P4502C92','P4502C93','P4502C94','P4502E1','P4502F1','yli_R0467']
model.remove_reactions(remove, remove_orphans=True)

In [193]:
temp = Annotation.index[Annotation['Combined Annotations'].str.contains('P450')]
display(Annotation.loc[temp])
Show_Data(temp)

,Combined Annotations,Signal P,Sc288c Orthologs,Human Orthologs,Sc288 Best Hit,Human Blast,Essential,WolfPSort,C Terminal
RTO4_ID,,,,,,,,,
8979,"K00490: CYP4F; cytochrome P450, family 4, subf...",S,,,ERG5,CYP4F,Not Essential,"plas 8, extr 5, cyto 3, pero 3, E.R. 3, cyto_p...",DEV*
10304,"K07424: CYP3A; cytochrome P450, family 3, subf...",A,,,,CYP3A,Not Essential,"cyto 10.5, plas 6, cyto_nucl 6, E.R. 4, mito 2...",FNV*
11057,"K07422: CYP2U1; cytochrome P450, family 2, sub...",S,,"CYP1A1,CYP1A2,CYP2A13,CYP2A6,CYP2A7,CYP2B6,CYP...",,CYP2U,Not Essential,"mito 7, cyto 5, extr 5, plas 3, cyto_nucl 3, p...",AGK*
13289,"K12156: CYP83A1; cytochrome P450, family 83, s...",,,"CYP1A1,CYP1A2,CYP2A13,CYP2A6,CYP2A7,CYP2B6,CYP...",,CYP21,Not Essential,"cyto 11, mito 7, pero 5, nucl 4",FEE*
13361,"K20660: CYP709; cytochrome P450, family 709",A,,,,CYP46A,Not Essential,"extr 7, E.R. 7, cyto 4, plas 3, golg 2, nucl 1...",DEE*
13417,"HMMPfam:Cytochrome P450:PF00067,SUPERFAMILY::S...",,,,,,Not Essential,"mito 7, cyto 7, cyto_mito 7, extr 6",SAL*
13456,BLAST: cytochrome P450 [Coprinopsis cinerea ok...,,,,,,Not Essential,"cyto_nucl 8.833, cyto_mito 8.833, nucl 8.5, mi...",KLP*
13796,KOG0157: Cytochrome P450 CYP4/CYP19/CYP26 subf...,S,,,,CYP4V,Not Essential,"cyto 8, mito 5, extr 5, plas 3, pero 3, mito_n...",RAA*
15678,BLAST: cytochrome P450 3A4 [Neurospora crassa ...,,,,,,Not Essential,"mito 19, nucl 2, extr 2, pero 2",KKL*


,Glucose,Xylose,Arabinose,Acetate,Coumarate,Ferulate,YNB Oleic Acid,YNB Ricinoleic Acid,YNB Glucose,YNB Gluc DOC,YPD
proteinId,,,,,,,,,,,
8979,-0.139908,-0.128731,-0.0200666,-0.58432,-0.0562727,-0.677421,0.155185,0.551796,-0.141794,-1.29649,0.594217
10304,-0.266672,-0.167355,-0.198551,-0.200802,0.409827,0.0358239,-0.0205349,0.192346,0.127195,-0.353236,-0.334459
11057,0.152536,-0.048676,0.0334867,0.230004,0.162079,0.00816819,-0.0457452,0.0798432,0.0316417,0.0587281,-0.0606798
13289,0.108991,0.129432,-0.0555733,-0.117793,0.45196,-0.0219733,0.174766,0.488014,0.170954,0.219427,0.376429
13361,0.185102,-0.441905,-0.209129,-0.356887,-0.132086,0.12807,-0.128726,-0.311587,0.281459,0.482153,-0.598514
13456,-0.269833,-0.0592424,-0.31458,-0.390661,0.467561,-0.431006,0.262001,0.518352,0.576015,0.281951,-0.0152014
13796,0.240064,0.178266,0.162563,0.132045,0.0400877,-0.0678085,0.320418,0.312165,0.228206,-0.249937,-0.00371661
15678,-0.0504266,0.058232,0.112753,0.122631,0.274788,-0.0374203,0.0346931,-0.0101147,0.252964,0.109081,0.0991454
16504,0.0184393,-0.141547,-0.0573212,-0.0720826,-0.35482,-0.0151301,-0.288983,-0.232999,0.451238,0.0824068,0.461698


In [194]:
for x in temp:
    if x in model.genes:
        for r in sorted(model.genes.get_by_id(x).reactions, key=lambda x: x.id):
            print(r.id, r.reaction, r.gene_reaction_rule)
    else:
        print(x, 'no reactions')
    print()

8979 no reactions

10304 no reactions

11057 no reactions

13289 no reactions

13361 no reactions

13417 no reactions

13456 no reactions

13796 no reactions

15678 no reactions

16504 no reactions



In [195]:
print(len(model.genes))
print(len(model.reactions))
print(len(model.metabolites))
model

1296
2402
2735


Name,R. toruloides
Memory address,0x0102f395e80
Number of metabolites,2735
Number of reactions,2402
Number of groups,0
Objective expression,0
Compartments,"c, x, m, e, r, v, n, g, p, h, f, l, d"


In [196]:
model.repair()

In [197]:
for x in sorted(model.genes, key=lambda x: x.id):
    if not x.reactions:
        print(x)
print()
for x in sorted(model.metabolites, key=lambda x: x.id):
    if not x.reactions:
        print(x)

10449_AT1
12050
12791
12826
14471
15098
1892_AT1
3030_AT1
3032_AT1
3033_AT1
34_AT1
35_AT1
37_AT1
ACOX3
Acox3
CRv4_Au5_s10_g406_t1
CRv4_Au5_s16_g6129_t1
CRv4_Au5_s16_g6229_t1
CRv4_Au5_s17_g7064_t1
CRv4_Au5_s17_g7079_t1
CRv4_Au5_s17_g7486_t2
CRv4_Au5_s1_g1339_t1
CRv4_Au5_s22_g9840_t1
CRv4_Au5_s2_g9657_t1
CRv4_Au5_s3_g10824_t1
CRv4_Au5_s3_g11300_t1
CRv4_Au5_s4_g11616_t1
CRv4_Au5_s5_g12182_t1
CRv4_Au5_s6_g12531_t1
CRv4_Au5_s6_g13296_t1
CRv4_Au5_s6_g13391_t1
CRv4_Au5_s6_g13523_t1
CRv4_Au5_s7_g14133_t1
CRv4_Au5_s7_g14144_t1
CRv4_Au5_s9_g15253_t1
EHHADH
Ehhadh
HADHA
HADHB
Hadha
Hadhb
PP_0559
PP_0860
PP_1607
PP_1996
YBR036C
YER061C
YHR067W
YIL002C
YKL055C
YLR343W
YMR215W
YMR307W
YNL106C
YOL030W
YOL065C
YOL132W
YOR109W
YOR221C
b0181
b0185
b1090
b2316
b2836
b3255

12dgr_SC_c
1ag3p_SC_c
1agly3p_SC_c
1agpc_SC_c
1hdecg3p_h
1odec11eg3p_h
1odec9eg3p_h
1odecg3p_h
2fe1s_c
2fe2s_c
3hbutACP_h
3hcvac11eACP_h
3hddecACP_h
3hdecACP_h
3hhexACP_h
3hmrsACP_h
3hoctACP_h
3hoctaACP_h
3hpalmACP_h
3ocvac11eACP_h
3od

In [198]:
cobra.manipulation.remove_genes(model, [x for x in model.genes if not x.reactions])
model.remove_metabolites([x for x in model.metabolites if not x.reactions])

In [199]:
print(len(model.genes))
print(len(model.reactions))
print(len(model.metabolites))
model

1231
2402
2585


Name,R. toruloides
Memory address,0x0102f395e80
Number of metabolites,2585
Number of reactions,2402
Number of groups,0
Objective expression,0
Compartments,"c, x, m, e, r, v, n, g, p, h, f, l, d"


In [200]:
for x in sorted(model.genes, key=lambda x: x.id):
    if not x.reactions:
        print(x)
print()
for x in sorted(model.metabolites, key=lambda x: x.id):
    if not x.reactions:
        print(x)

In [201]:
cobra.io.save_json_model(model, "IFO0880_GPR_1c.json")

In [202]:
model_old = cobra.io.load_json_model("IFO0880_GPR_1b.json")
model_new = cobra.io.load_json_model("IFO0880_GPR_1c.json")

In [203]:
print('Removed reactions\n')
for r in sorted(model_old.reactions, key=lambda x: x.id):
    if r not in model_new.reactions:
        print(r)

Removed reactions

1AGPEAT1801819Z: 1agpe180_c + ocdce9coa_c --> coa_c + pe1801819Z_c
1AGPEAT1801829Z12Z: 1agpe180_c + lnlccoa_c --> coa_c + pe1801829Z12Z_c
1AGPEAT1801835Z9Z12Z: 1agpe180_c + pacoa_c --> coa_c + pe1801835Z9Z12Z_c
1AGPEAT1801845Z9Z12Z15Z: 1agpe180_c + cacoa_c --> coa_c + pe1801845Z9Z12Z15Z_c
1AGPEAT18111Z1819Z: 1agpe18111Z_c + ocdce9coa_c --> coa_c + pe18111Z1819Z_c
1AGPEAT18111Z1829Z12Z: 1agpe18111Z_c + lnlccoa_c --> coa_c + pe18111Z1829Z12Z_c
1AGPEAT18111Z1835Z9Z12Z: 1agpe18111Z_c + pacoa_c --> coa_c + pe18111Z1835Z9Z12Z_c
1AGPEAT18111Z1845Z9Z12Z15Z: 1agpe18111Z_c + cacoa_c --> coa_c + pe18111Z1845Z9Z12Z15Z_c
1AGPEAT1819Z1819Z: 1agpe1819Z_c + ocdce9coa_c --> coa_c + pe1819Z1819Z_c
1AGPEAT1819Z1829Z12Z: 1agpe1819Z_c + lnlccoa_c --> coa_c + pe1819Z1829Z12Z_c
1AGPEAT1819Z1835Z9Z12Z: 1agpe1819Z_c + pacoa_c --> coa_c + pe1819Z1835Z9Z12Z_c
1AGPEAT1819Z1845Z9Z12Z15Z: 1agpe1819Z_c + cacoa_c --> coa_c + pe1819Z1845Z9Z12Z15Z_c
1AGPEAT1829Z12Z1835Z9Z12Z: 1agpe1829Z12Z_c + pacoa_

In [204]:
print('Updated reactions\n')
for r in sorted(model_old.reactions, key=lambda x: x.id):
    if r in model_new.reactions:
        r2 = model_new.reactions.get_by_id(r.id)
        if (r.name == r2.name and r.reaction == r2.reaction and r.gene_reaction_rule == r2.gene_reaction_rule and
            r.lower_bound == r2.lower_bound and r.upper_bound == r2.upper_bound):
            pass
        else:
            print('Old', r, r.gene_reaction_rule)
            print('New', r2, r2.gene_reaction_rule)
            print()

Updated reactions

Old 13GS: udpg_c --> 13BDglcn_c + h_c + udp_c (YLR343W and 15984 and 16655) or (YMR215W and 15984 and 16655) or (YMR307W and 15984 and 16655) or (YOL030W and 15984 and 16655) or (YOL132W and 15984 and 16655)
New 13GS: udpg_c --> 13BDglcn_c + h_c + udp_c 15984

Old ACACT10m: 2maacoa_m + coa_m <=> accoa_m + ppcoa_m 13813 or 8678
New ACACT10m: 2maacoa_m + coa_m <=> accoa_m + ppcoa_m 8678

Old ACACT1m: 2.0 accoa_m --> aacoa_m + coa_m 8678 or 8885
New ACACT1m: 2.0 accoa_m <=> aacoa_m + coa_m 8678

Old ACACT1r: 2.0 accoa_c <=> aacoa_c + coa_c 13813 or 8678 or 8885
New ACACT1r: 2.0 accoa_c <=> aacoa_c + coa_c 8678

Old ACACT2m: accoa_m + btcoa_m --> 3ohcoa_m + coa_m 13813
New ACACT2m: accoa_m + btcoa_m <-- 3ohcoa_m + coa_m 8885

Old ACACT3m: accoa_m + hxcoa_m --> 3oocoa_m + coa_m 13813
New ACACT3m: accoa_m + hxcoa_m <-- 3oocoa_m + coa_m 8885

Old ACACT4m: accoa_m + occoa_m <-- 3odcoa_m + coa_m 13813 or 9065
New ACACT4m: accoa_m + occoa_m <-- 3odcoa_m + coa_m 8885

Old ACACT

Old P45017A3r: h_r + nadph_r + o2_r + prgstrn_r --> 17ahprgstrn_r + h2o_r + nadp_r 11057 or 12565 or 13289
New P45017A3r: h_r + nadph_r + o2_r + prgstrn_r --> 17ahprgstrn_r + h2o_r + nadp_r 12565

Old P45017A4r: 17ahprgstrn_r + h_r + nadph_r + 0.5 o2_r --> acald_r + andrstndn_r + h2o_r + nadp_r 11057 or 12565 or 13289
New P45017A4r: 17ahprgstrn_r + h_r + nadph_r + 0.5 o2_r --> acald_r + andrstndn_r + h2o_r + nadp_r 12565

Old PPCOAOm: fad_m + ppcoa_m --> fadh2_m + prpncoa_m 10408 or 12570 or 14070
New PPCOAOm: fad_m + ppcoa_m --> fadh2_m + prpncoa_m 14070

Old PRPNCOAHYDm: h2o_m + prpncoa_m --> 3hpcoa_m 11907 or 14805 or 16128 or (HADHA and HADHB) or (Hadha and Hadhb)
New PRPNCOAHYDm: h2o_m + prpncoa_m --> 3hpcoa_m 14805

Old SQLEr: h_r + nadph_r + o2_r + sql_r --> Ssq23epx_r + h2o_r + nadp_r 13729 or (13286 and 13729)
New SQLEr: h_r + nadph_r + o2_r + sql_r --> Ssq23epx_r + h2o_r + nadp_r 13286 and 13729

Old SQLErx: h_r + nadh_r + o2_r + sql_r --> Ssq23epx_r + h2o_r + nad_r (13206 an

In [205]:
print('Added reactions\n')
for r in sorted(model_new.reactions, key=lambda x: x.id):
    if r not in model_old.reactions:
        print(r)

Added reactions

4CMCOAS: T4hcinnm_c + atp_c + coa_c --> amp_c + coucoa_c + ppi_c
AACOATx: acac_x + atp_x + coa_x <=> aacoa_x + amp_x + ppi_x
ACACT1p: aacoa_x + coa_x --> 2.0 accoa_x
ACACT2p: 3ohcoa_x + coa_x --> accoa_x + btcoa_x
ACACT3p: 3oocoa_x + coa_x --> accoa_x + hxcoa_x
ACOAD1m: btcoa_m + fad_m --> b2coa_m + fadh2_m
ACOAD1p: btcoa_x + fad_x --> b2coa_x + fadh2_x
ACOAD2m: fad_m + hxcoa_m --> fadh2_m + hx2coa_m
ACOAD2p: fad_x + hxcoa_x --> fadh2_x + hx2coa_x
ACOAD3m: fad_m + occoa_m --> fadh2_m + oc2coa_m
ACOAD3p: fad_x + occoa_x --> fadh2_x + oc2coa_x
ACOAD4m: dcacoa_m + fad_m --> dc2coa_m + fadh2_m
ACOAD4p: dcacoa_x + fad_x --> dc2coa_x + fadh2_x
ACOAD5m: ddcacoa_m + fad_m --> dd2coa_m + fadh2_m
ACOAD5p: ddcacoa_x + fad_x --> dd2coa_x + fadh2_x
ACOAD6m: fad_m + tdcoa_m --> fadh2_m + td2coa_m
ACOAD6p: fad_x + tdcoa_x --> fadh2_x + td2coa_x
ACOAD7m: fad_m + pmtcoa_m --> fadh2_m + hdd2coa_m
ACOAD7p: fad_x + pmtcoa_x --> fadh2_x + hdd2coa_x
ACOAD8p: fad_x + stcoa_x --> fadh2_x + od